<a href="https://colab.research.google.com/github/yanavlall/DATASCI112/blob/main/DATASCI_112_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**NOTE: check variables in case change stuff around later**

When rummaging through data, we were perplexed to noice a surprisingly low correlation between masking and COVID cases and deaths per capita over the pandemic up to May 31, 2021. In other words, states that masked more didn't seem to have more or less COVID cases and deaths, bucking conventional narratives. Though preliminary, this result gets at the importance of data in public policy, as it allows us to see where the reality on the ground diverges with our preconceived intuitions.

For this project, we analyzed a basket of variables and their relationships to COVID, searching for which variables we might want to emphasize in combating a pandemic. This analysis is important for two reasons: first, because historical understanding is always useful for informing a mental model of the world; and second, because answering these questions could directly contribute to future pandemic preparedness, saving lives. There are lots of narratives floating around COVID; let's see what the data has to say.

Seeing prediction as a pathway to understanding, we decided to collect a basket of variables - as many measures potentially-important to COVID as possible. **Our research question then was - given this basket of variables, can we predict 2020 COVID deaths and cases per capita, and can we use our models to gain insight into COVID?** We aimed, in other words, towards an exploratory basket-of-variable analysis.

Several research teams have conducted basket-of-variable analyses on COVID. These were helpful; some of our variables were inspired from these studies. Surprisingly, our specific analysis is not covered in the literature; almost all analyses we could find used spring 2020 data and/or studied international or state-level data. (Only one group of researchers performed county-level analysis on COVID, and they focused on spring 2020, while our project tackles all 2020.) (Velasco et. al. 2021, https://pubmed.ncbi.nlm.nih.gov/33466900/; Riley et. al. 2022, https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0266330#sec006; Ziyadidegan et. al. 2022, https://link.springer.com/article/10.1007/s00477-021-02148-0; Karmakar, et. al. 2021, https://jamanetwork.com/journals/jamanetworkopen/fullarticle/2775732; Aabed et. al. 2020, https://www.sciencedirect.com/science/article/pii/S1319562X20306331; Chang et. al. 2022, https://www.nature.com/articles/s41598-022-09783-9)

We limited COVID data collection from the beginning of pandemic data collection (which varied by state, so we mass-downloaded data starting from January 1, 2020) up to December 31, 2020 (after which the appearance of COVID strains, the availability of vaccinations, and the non-uniform lifting of vaccine and masking mandates greatly complicates the analysis). We also collected pre-COVID data for baselines, mostly from between 2015 and 2019; our earliest such data, religious demographics, is from 2010.

**contiguous US only, excluded Puerto Rico, Alaska, Hawaii + other such states b/c very different admin landscapes, literal conditions (wather, demographics, sparsity, native american reservations, lack of information) - would multiply the time for data management without actually really increasing insight, not really worth it**

Our input variables fall into four broad categories. Our "baseline" variables measure a county's pre-COVID socioeconomic, physical, and mental "health"/vulnerabilities. (Think obesity rates, air pollution, inequality, demographics, etc.) This data is coarsely-grained; there is one data point per county. Our "politics" variables constitute similarly coarsely-grained data about 2020 proper. (Think political affiliation, political control, election results, and masking; no reliable, comprehensive week-by-week/month-by-month masking data exists.) Our "fluctuant" variables are measures that changed week-by-week during COVID, potentially influencing outcomes. (Think lockdown mobility, hospital capacity statistics, policies, etc.) This data is finely-grained; there are multiple data points per county. Lastly, our "spatial" variable measures the extent of COVID in a county's geographical neighbors, to attempt to incorporate disease spread into our models.

Our output variables are two - COVID cases per capita and COVID deaths per capita - predicted for the most part on a weekly basis. **(We also performed some low-granularity yearly analysis.) [CHECK THIS BEFORE SUBMITTING]**

**Step 1: Data Collection**

In [ ]:
# though some data can be read directly from URLs,
# for consistency's sake, we download all data as csv and then load them in
# store data in DATASCI_COVID_data in Google Drive

from google.colab import drive
drive.mount("/content/gdrive")

import pandas as pd

data_dir = "/content/gdrive/My Drive/DATASCI_COVID_data/"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# import masking data
# masking data on the county level is unreliable, spotty, and often nonexistent
# data processed from a two-week July 2020 of 250,000 Americans
# is by far the most reliable and granular data we could find
# data from https://github.com/nytimes/covid-19-data/tree/master/mask-use

masks = pd.read_csv(data_dir + "mask-use-by-county_2020.csv")
masks

,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
0,1001,0.053,0.074,0.134,0.295,0.444
1,1003,0.083,0.059,0.098,0.323,0.436
2,1005,0.067,0.121,0.120,0.201,0.491
3,1007,0.020,0.034,0.096,0.278,0.572
4,1009,0.053,0.114,0.180,0.194,0.459
...,...,...,...,...,...,...
3137,56037,0.061,0.295,0.230,0.146,0.268
3138,56039,0.095,0.157,0.160,0.247,0.340
3139,56041,0.098,0.278,0.154,0.207,0.264
3140,56043,0.204,0.155,0.069,0.285,0.287


In [ ]:
# import ACS from Social Explorer
# the American Community Survey (ACS) is collected by US Census Bureau
# tracks county-level socioeconomic and demographic data
# ACS collected in 1-year, 3-year, and 5-year increments
# longer timeframes are less granular but more accurate and comprehensive
# we obtained most baseline variables from the ACS, choosing 2015-2019 ACS,
# to maximize accuracy, comprehensiveness, and proximity to COVID
# (2020 data was notoriously inaccurate; Census Bureau itself warns against use)
# source URL - https://www.socialexplorer.com/tables/ACS2019_5yr/R13227257
# for more info about ACS, see:
# https://www.census.gov/programs-surveys/acs/guidance/comparing-acs-data.html
# https://www.census.gov/newsroom/press-releases/2022/acs-5-year-estimates.html

ACS = pd.read_csv(data_dir + "county_ACS_2015-2019.csv")
ACS

<ipython-input-3-9b19d600c097>:14: DtypeWarning: Columns (0,5,6,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,26

,FIPS,Geographic Identifier,Area Name,Qualifying Name,State Postal Abbreviation,Summary Level,Geographic Component,File identification,Logical Record Number,US,...,Own Children under 18 Years,Own Children under 18 Years: Children Living with Single Parents,Households,Households: 1-Person Household,Households: 2-Person Household,Households: 3-Person Household,Households: 4-Person Household,Households: 5-Person Household,Households: 6-Person Household,Households: 7-or-More Person Household
0,Geo_FIPS,Geo_GEOID,Geo_NAME,Geo_QName,Geo_STUSAB,Geo_SUMLEV,Geo_GEOCOMP,Geo_FILEID,Geo_LOGRECNO,Geo_US,...,SE_A10065_001,SE_A10065_002,SE_A10066_001,SE_A10066_002,SE_A10066_003,SE_A10066_004,SE_A10066_005,SE_A10066_006,SE_A10066_007,SE_A10066_008
1,01001,05000US01001,Autauga County,"Autauga County, Alabama",al,050,00,ACSSF,0000013,NaN,...,11694,2817,21397,5473,7473,3671,2659,1462,527,132
2,01003,05000US01003,Baldwin County,"Baldwin County, Alabama",al,050,00,ACSSF,0000014,NaN,...,39564,8432,80930,23955,31304,10710,8820,4180,1481,480
3,01005,05000US01005,Barbour County,"Barbour County, Alabama",al,050,00,ACSSF,0000015,NaN,...,4150,2186,9345,2983,3066,1417,1179,469,152,79
4,01007,05000US01007,Bibb County,"Bibb County, Alabama",al,050,00,ACSSF,0000016,NaN,...,3361,893,6891,1963,2385,1058,923,313,121,128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3216,72145,05000US72145,Vega Baja Municipio,"Vega Baja Municipio, Puerto Rico",pr,50,0,ACSSF,86,NaN,...,8343,4512,18721,4717,6322,3783,2845,816,151,87
3217,72147,05000US72147,Vieques Municipio,"Vieques Municipio, Puerto Rico",pr,50,0,ACSSF,87,NaN,...,1524,941,2258,839,810,350,62,138,43,16
3218,72149,05000US72149,Villalba Municipio,"Villalba Municipio, Puerto Rico",pr,50,0,ACSSF,88,NaN,...,3506,2464,7908,1645,2457,1756,1299,472,210,69
3219,72151,05000US72151,Yabucoa Municipio,"Yabucoa Municipio, Puerto Rico",pr,50,0,ACSSF,89,NaN,...,4975,3534,11541,3281,4188,2079,1315,444,177,57


In [ ]:
# import religion data from Social Explorer
# most recent Census Bureau data of county religiosity, congregations, etc.
# comes from 2010 census
# source URL - https://www.socialexplorer.com/tables/RCMS_2010/R13227268

religion = pd.read_csv(data_dir + "county_religion_2010.csv")
religion

<ipython-input-4-f3b57e046ab6>:6: DtypeWarning: Columns (0,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,2

,FIPS,Name of Area,Qualifying Name,Population (2010),All Religious Congregations,All Religious Adherents,All Religious Adherence Rate of Total Population,Total Congregations - Major Religions,Evangelical Protestant Congregations,Black Protestant Congregations,...,Reconstructionist Judaism.1,Reform Judaism.1,Shinto.1,Sikh.1,Swedenborgian Church.1,Tao.1,Union of Messianic Jewish Congregations.2,Unitarian Universalist Association of Congregations.2,Unity Churches,Zoroastrian.1
0,Geo_FIPS,Geo_Name,Geo_QName,RCMS10_T001_001,RCMS10_T002_001,RCMS10_NV001_001,RCMS10_NV009_001,RCMS10_T003_001,RCMS10_T003_002,RCMS10_T003_003,...,RCMS10_NV016_021,RCMS10_NV016_022,RCMS10_NV016_023,RCMS10_NV016_024,RCMS10_NV016_025,RCMS10_NV016_026,RCMS10_NV016_027,RCMS10_NV016_028,RCMS10_NV016_029,RCMS10_NV016_030
1,01001,Autauga County,"Autauga County, Alabama",54571,106,36938,67.687888889,106,79,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.005
2,01003,Baldwin County,"Baldwin County, Alabama",182265,271,96918,53.174,271,178,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01005,Barbour County,"Barbour County, Alabama",27457,89,15101,54.999,89,51,21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01007,Bibb County,"Bibb County, Alabama",22915,81,11430,49.88,81,63,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3145,56037,Sweetwater County,"Sweetwater County, Wyoming",43806,66,20905,47.722,66,35,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3146,56039,Teton County,"Teton County, Wyoming",21294,20,5544,26.036,20,8,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3147,56041,Uinta County,"Uinta County, Wyoming",21118,47,12815,60.683,47,16,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3148,56043,Washakie County,"Washakie County, Wyoming",8533,25,4026,47.181889,25,15,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# import health data from Social Explorer from 2019
# (2020 overlaps with COVID, and as such won't serve to establish a baseline)
# source URL - https://www.socialexplorer.com/tables/HD2019/R13324268

health = pd.read_csv(data_dir + "county_healthdata_2019.csv")
health

,FIPS,Name of Area,Qualifying Name,Nation,State,County,Physically Unhealthy Days per Month (Persons 18 Years and Over),Mentally Unhealthy Days per Month (Persons 18 Years and Over),Percent of Adults That Report Fair or Poor Health (Persons 18 Years and Over),Percent of Low Birthweight Births (<2.5kg),...,Food Environment Index,Air Pollution Particulate Matter Average Daily Pm2.5,Presence Of Drinking Water Violations,Percentage Of Households with Severe Housing Problems,Percentage Of Households with High Housing Costs,Percentage Of Households with Overcrowding,Percentage of Households with Lack of Kitchen or Plumbing Facilities,Percent Of Driving Alone to Work,Long Commute Driving Alone Workers Who Drive Alone,Percent Of Long Commute Driving Alone Workers Who Drive Alone
0,Geo_FIPS,Geo_NAME,Geo_QNAME,Geo_NATION,Geo_STATE,Geo_COUNTY,SE_T001_001,SE_T001_002,SE_T002_001,SE_T003_001,...,SE_T013_001,SE_T016_001,SE_T016_002,SE_T016_003,SE_T016_004,SE_T016_005,SE_T016_006,SE_T016_007,SE_T016_008,SE_T016_009
1,01001,Autauga County,"Autauga County, Alabama",00,01,001,4.2005779826,4.3067392835,18.411124355,8.4757194245,...,7.2,11.7,No,14.954645747,13.205222961,2.4515812699,0.6374111302,85.965056526,20911,38.3
2,01003,Baldwin County,"Baldwin County, Alabama",00,01,003,4.0987477691,4.2496487805,18.060457821,8.3386827577,...,8,10.3,Yes,13.831725255,12.569278139,1.0728021051,0.6072464746,84.719423478,74415,40.5
3,01005,Barbour County,"Barbour County, Alabama",00,01,005,5.0674383044,4.634994147,25.773415631,10.952623535,...,5.6,11.5,No,15.455531453,13.67426348,2.0065075922,0.8134490239,83.404353334,7242,33.8
4,01007,Bibb County,"Bibb County, Alabama",00,01,007,4.3633772783,4.3157100192,19.996911856,11.105002749,...,7.6,11.2,No,10.960854093,10.808080808,0.1992882562,0.2846975089,86.365902293,6930,48.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3216,72145,Vega Baja Municipio,"Vega Baja Municipio, Puerto Rico",00,72,145,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3217,72147,Vieques Municipio,"Vieques Municipio, Puerto Rico",00,72,147,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3218,72149,Villalba Municipio,"Villalba Municipio, Puerto Rico",00,72,149,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3219,72151,Yabucoa Municipio,"Yabucoa Municipio, Puerto Rico",00,72,151,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# import CDC baseline health data
# import CDC 2018 sleep deprivation data (% sleep-deprived among adults, age-adjusted)
# for all data, CDC defines adults as 18+ years old
# more recent county-level data isn't available
# source URL - https://www.cdc.gov/sleep/data-and-statistics/adults.html

sleep = pd.read_csv(data_dir + "county_sleepdeprived_2018.csv")
sleep

,CountyFIPS,County,State Abbreviation,State,Age-Adjusted Prevalence (%),95% Confidence Interval,Quartile,Unnamed: 7
0,37185,Warren,NC,North Carolina,41.6,40.3 - 42.8,39.7 - 49.1,NaN
1,45003,Aiken,SC,South Carolina,37.3,36.0 - 38.5,36.8 - 39.6,NaN
2,45017,Calhoun,SC,South Carolina,39.3,38.0 - 40.6,36.8 - 39.6,NaN
3,45031,Darlington,SC,South Carolina,41.9,40.6 - 43.1,39.7 - 49.1,NaN
4,45041,Florence,SC,South Carolina,41.0,39.6 - 42.3,39.7 - 49.1,NaN
...,...,...,...,...,...,...,...,...
3137,21043,Carter,KY,Kentucky,44.7,42.8 - 46.4,39.7 - 49.1,NaN
3138,28033,DeSoto,MS,Mississippi,36.9,35.5 - 38.2,36.8 - 39.6,NaN
3139,36085,Richmond,NY,New York,43.6,42.3 - 44.8,39.7 - 49.1,NaN
3140,20169,Saline,KS,Kansas,32.5,31.0 - 33.8,25.6 - 33.7,NaN


In [ ]:
# import CDC risk factors & cardiovascular disease data
# source URL - https://nccd.cdc.gov/DHDSPAtlas/
# use the left panel to select relevant measures one-by-one, then click export
# data downloaded on March 13, 2023

# import CDC 2020 smokers data (% adults smoking)
smokers = pd.read_csv(data_dir + "county_smokers_2020.csv")
smokers
# import CDC 2019 diabetes data (% diagnosed with diabetes, 20+ years old)
diabetes = pd.read_csv(data_dir + "county_diabetes_2019.csv")
diabetes
# import CDC 2019 obesity data (% obese, 20+ years old)
obesity = pd.read_csv(data_dir + "county_obesity_2019.csv")
obesity
# import CDC 2019 physical inactivity (age-adjusted, % inactive, 20+ years old)
inactive = pd.read_csv(data_dir + "county_physinactive_2019.csv")
inactive
# import CDC 2017-2019 death rate from heart disease, all ages
heart = pd.read_csv(data_dir + "county_heartdeaths_2017-2019.csv")
heart
# import CDC 2017-2019 death rate from strokes, all ages
strokes = pd.read_csv(data_dir + "county_strokedeaths_2017-2019.csv")
strokes
# import CDC 2017-2019 death rate from high blood pressure, all ages
hypertension = pd.read_csv(data_dir + "county_hypertension_2017-2019.csv")
hypertension

,cnty_fips,display_name,Value,theme_range
0,2013,"""Aleutians East, (AK)""",70.0,12.3 - 92.2 (644)
1,2016,"""Aleutians West, (AK)""",80.6,12.3 - 92.2 (644)
2,2020,"""Anchorage, (AK)""",84.2,12.3 - 92.2 (644)
3,2050,"""Bethel, (AK)""",136.2,116.8 - 141.0 (643)
4,2060,"""Bristol Bay, (AK)""",-1.0,NaN
...,...,...,...,...
3221,72151,"""Yabucoa, (PR)""",180.8,174.7 - 554.9 (641)
3222,72153,"""Yauco, (PR)""",211.7,174.7 - 554.9 (641)
3223,78010,"""Saint Croix (County Equivalent), (VI)""",-1.0,NaN
3224,78020,"""Saint John (County Equivalent), (VI)""",-1.0,NaN


In [ ]:
# import other baseline data from CDC atlas (physical and social environment)

# import county air quality (annual PM2.5 level, 2016)
AQI = pd.read_csv(data_dir + "county_AQI_2016.csv")
AQI
# import county broadband (% without broadband coverage, 2016-2020)
broadband = pd.read_csv(data_dir + "county_broadband_2016-2020.csv")
broadband
# import extent of primary care (population per primary care physician, 2019)
doctors = pd.read_csv(data_dir + "county_PCP_2019.csv")
doctors

,cnty_fips,display_name,Value,theme_range
0,2013,"""Aleutians East, (AK)""",-1.0,NaN
1,2016,"""Aleutians West, (AK)""",2.8,2.5 - 3.5 (547)
2,2020,"""Anchorage, (AK)""",0.8,0.2 - 1.3 (723)
3,2050,"""Bethel, (AK)""",0.9,0.2 - 1.3 (723)
4,2060,"""Bristol Bay, (AK)""",-1.0,NaN
...,...,...,...,...
3221,72151,"""Yabucoa, (PR)""",6.5,3.6 - 28.3 (591)
3222,72153,"""Yauco, (PR)""",0.8,0.2 - 1.3 (723)
3223,78010,"""Saint Croix (County Equivalent), (VI)""",-1.0,NaN
3224,78020,"""Saint John (County Equivalent), (VI)""",-1.0,NaN


In [ ]:
# import insurers data from Kaiser Family Foundation
# source URL - https://www.kff.org/private-insurance/issue-brief/insurer-participation-on-the-aca-marketplaces-2014-2021/
# data imported from article as .xlsx, manually converted to csv

insurers = pd.read_csv(data_dir + "county_insurers_2014-2021.csv")
insurers

,FIPS,ST,County Name,listissuers2014,listissuers2015,listissuers2016,listissuers2017,listissuers2018,listissuers2019,listissuers2020,...,2020,2021,Insurer Category.2014,Insurer Category.2015,Insurer Category.2016,Insurer Category.2017,Insurer Category.2018,Insurer Category.2019,Insurer Category.2020,Insurer Category.2021
0,1001,AL,Autauga,BCBS Of AL,"BCBS Of AL, Unitedhealth","BCBS Of AL, Unitedhealth",BCBS Of AL,BCBS Of AL,BCBS Of AL,BCBS Of AL,...,1,1,One,Two,Two,One,One,One,One,One
1,1003,AL,Baldwin,BCBS Of AL,"BCBS Of AL, Unitedhealth","BCBS Of AL, Unitedhealth",BCBS Of AL,BCBS Of AL,BCBS Of AL,BCBS Of AL,...,1,1,One,Two,Two,One,One,One,One,One
2,1005,AL,Barbour,BCBS Of AL,"BCBS Of AL, Unitedhealth","BCBS Of AL, Unitedhealth",BCBS Of AL,BCBS Of AL,BCBS Of AL,BCBS Of AL,...,1,1,One,Two,Two,One,One,One,One,One
3,1007,AL,Bibb,BCBS Of AL,"BCBS Of AL, Unitedhealth","BCBS Of AL, Unitedhealth",BCBS Of AL,BCBS Of AL,BCBS Of AL,BCBS Of AL,...,1,1,One,Two,Two,One,One,One,One,One
4,1009,AL,Blount,BCBS Of AL,"BCBS Of AL, Unitedhealth","BCBS Of AL, Humana, Unitedhealth",BCBS Of AL,BCBS Of AL,BCBS Of AL,BCBS Of AL,...,1,1,One,Two,Three or More,One,One,One,One,One
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,56037,WY,Sweetwater,"BCBS Of WY, WINhealth","BCBS Of WY, WINhealth",BCBS Of WY,BCBS Of WY,BCBS Of WY,BCBS Of WY,BCBS Of WY,...,1,2,Two,Two,One,One,One,One,One,Two
3138,56039,WY,Teton,"BCBS Of WY, WINhealth","BCBS Of WY, WINhealth",BCBS Of WY,BCBS Of WY,BCBS Of WY,BCBS Of WY,BCBS Of WY,...,1,2,Two,Two,One,One,One,One,One,Two
3139,56041,WY,Uinta,"BCBS Of WY, WINhealth","BCBS Of WY, WINhealth",BCBS Of WY,BCBS Of WY,BCBS Of WY,BCBS Of WY,BCBS Of WY,...,1,2,Two,Two,One,One,One,One,One,Two
3140,56043,WY,Washakie,"BCBS Of WY, WINhealth","BCBS Of WY, WINhealth",BCBS Of WY,BCBS Of WY,BCBS Of WY,BCBS Of WY,BCBS Of WY,...,1,2,Two,Two,One,One,One,One,One,Two


In [ ]:
# import hospital capacity data from HealthData.gov
# source URL - https://healthdata.gov/Hospital/COVID-19-Reported-Patient-Impact-and-Hospital-Capa/anag-cw7u/data
# dataset spans late 2019 to early 2023, very granular (hospital-level) + rich

hospitals = pd.read_csv(data_dir + "hospital_capacity_2019-2023.csv")
hospitals

<ipython-input-10-f9c8417e33eb>:5: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  hospitals = pd.read_csv(data_dir + "hospital_capacity_2019-2023.csv")


,hospital_pk,collection_week,state,ccn,hospital_name,address,city,zip,hospital_subtype,fips_code,...,previous_day_admission_pediatric_covid_confirmed_unknown_7_day_sum,staffed_icu_pediatric_patients_confirmed_covid_7_day_avg,staffed_icu_pediatric_patients_confirmed_covid_7_day_coverage,staffed_icu_pediatric_patients_confirmed_covid_7_day_sum,staffed_pediatric_icu_bed_occupancy_7_day_avg,staffed_pediatric_icu_bed_occupancy_7_day_coverage,staffed_pediatric_icu_bed_occupancy_7_day_sum,total_staffed_pediatric_icu_beds_7_day_avg,total_staffed_pediatric_icu_beds_7_day_coverage,total_staffed_pediatric_icu_beds_7_day_sum
0,100217,2020/07/10,FL,100217,SEBASTIAN RIVER MEDICAL CENTER,13695 US HWY 1,SEBASTIAN,32978.0,Short Term,12061.0,...,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN
1,100232,2020/07/03,FL,100232,HCA FLORIDA PUTNAM HOSPITAL,611 ZEAGLER DR,PALATKA,32177.0,Short Term,12107.0,...,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN
2,102029,2020/05/01,FL,102029,SELECT SPECIALTY HOSPITAL-FORT MYERS,3050 CHAMPION RING ROAD,FORT MYERS,33905.0,Long Term,12071.0,...,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN
3,102030,2020/06/05,FL,102030,SELECT SPECIALTY HOSPITAL DAYTONA BEACH,301 MEMORIAL MEDICAL PARKWAY,DAYTONA BEACH,32117.0,Long Term,12127.0,...,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN
4,110107,2020/07/31,GA,110107,ATRIUM HEALTH NAVICENT THE MEDICAL CENTER,777 HEMLOCK STREET,MACON,31201.0,Short Term,13021.0,...,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
761856,400012,2022/06/24,PR,400012,HOSPITAL ONCOLOGICO DR ISAAC GONZALEZ MARTINEZ,BO. MONACILLOS CARR 22 CENTRO MEDICO DE PUERTO...,SAN JUAN,935.0,Short Term,72127.0,...,0.0,0.0,5,0.0,0.0,5,0.0,0.0,5,0.0
761857,451386,2020/12/11,TX,451386,MEMORIAL HOSPITAL,224 E SECOND STREET,DUMAS,79029.0,Critical Access Hospitals,48341.0,...,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN
761858,500124,2022/11/18,WA,500124,EVERGREENHEALTH MEDICAL CENTER,12040 NE 128TH STREET,KIRKLAND,98034.0,Short Term,53033.0,...,0.0,0.0,7,0.0,0.0,7,0.0,0.0,7,0.0
761859,670128,2021/10/15,TX,670128,BAYLOR SCOTT & WHITE MEDICAL CENTER – PFLUGERV...,2600 EAST PFLUGERVILLE PARKWAY,PFLUGERVILLE,78660.0,Short Term,48453.0,...,NaN,NaN,0,NaN,0.0,7,0.0,0.0,7,0.0


In [ ]:
# import election data
# data obtained from MIT Election Data and Science Lab with Harvard Dataverse
# source URL - https://dataverse.harvard.edu/file.xhtml?fileId=6689930&version=11.0

elections = pd.read_csv(data_dir + "county_pres_2000-2020.csv")
elections

,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode
0,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,AL GORE,DEMOCRAT,4942,17208,20220315,TOTAL
1,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,GEORGE W. BUSH,REPUBLICAN,11993,17208,20220315,TOTAL
2,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,RALPH NADER,GREEN,160,17208,20220315,TOTAL
3,2000,ALABAMA,AL,AUTAUGA,1001.0,US PRESIDENT,OTHER,OTHER,113,17208,20220315,TOTAL
4,2000,ALABAMA,AL,BALDWIN,1003.0,US PRESIDENT,AL GORE,DEMOCRAT,13997,56480,20220315,TOTAL
...,...,...,...,...,...,...,...,...,...,...,...,...
72612,2020,WYOMING,WY,WASHAKIE,56043.0,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,3245,4032,20220315,TOTAL
72613,2020,WYOMING,WY,WESTON,56045.0,US PRESIDENT,JOSEPH R BIDEN JR,DEMOCRAT,360,3560,20220315,TOTAL
72614,2020,WYOMING,WY,WESTON,56045.0,US PRESIDENT,JO JORGENSEN,LIBERTARIAN,46,3560,20220315,TOTAL
72615,2020,WYOMING,WY,WESTON,56045.0,US PRESIDENT,OTHER,OTHER,47,3560,20220315,TOTAL


In [ ]:
# import state political control data
# source URL - https://documents.ncsl.org/wwwncsl/Elections/Legis_Control_2020_April%201.pdf
# from https://www.ncsl.org/about-state-legislatures/state-partisan-composition
# using Adobe, exported to spreadsheet, then manually converted to csv
# partisan composition of legislatures & governorships as of April 1, 2020

state_politics = pd.read_csv(data_dir + "state_political_control_2020.csv")
state_politics

,"2020 State & Legislative Partisan Composition (April 1st, 2020)",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,STATE,Total\nSeats,Total\nSenate,Senate\nDem.,Senate\nRep.,Senate\nother,Total\nHouse,House\nDem.,House\nRep.,House\nother,Legis.\nControl,Gov.\nParty,State\nControl
1,Alabama,140,35,8,27,NaN,105,28,77,NaN,Rep,Rep,Rep
2,Alaska,60,20,7,13,NaN,40,15,23,2,Rep,Rep,Rep
3,Arizona,90,30,13,17,NaN,60,29,31,NaN,Rep,Rep,Rep
4,Arkansas,135,35,9,26,NaN,100,24,76,NaN,Rep,Rep,Rep
5,California,120,40,29,10,1v,80,61,18,1,Dem,Dem,Dem
6,Colorado,100,35,19,16,NaN,65,41,24,NaN,Dem,Dem,Dem
7,Connecticut,187,36,22,14,NaN,151,91,60,NaN,Dem,Dem,Dem
8,Delaware,62,21,12,9,NaN,41,26,15,NaN,Dem,Dem,Dem
9,Florida,160,40,17,23,NaN,120,47,73,NaN,Rep,Rep,Rep


In [ ]:
# import GDP by county data from Bureau of Economic Analysis
# source URL - https://www.bea.gov/data/gdp/gdp-county-metro-and-other-areas
# go to source tables to download data as spreadsheet; manually converted to csv
# 2021 report contains 2018-2021 GDP figures
# BEA retroactively refines GDP figures; 2021 report is better than 2020 report

# GDP = pd.read_csv(data_dir + "county_GDP_2018-2021.csv")
# GDP

# # clean GDP data
# # we only want county GDP before 2021, not relative rankings
# cleanGDP = GDP.iloc[2:, :4]
# cleanGDP.iloc[0, 0] = "County"

# # setting first row as header
# cleanGDP.columns = cleanGDP.iloc[0]
# cleanGDP = cleanGDP.add_suffix("_GDP")
# cleanGDP = cleanGDP.iloc[2:]

# # though the BEA data is horribly-produced
# # (why are there no FIPS numbers!)
# # there is one redeeming feature - there is one blank line between each state,
# # and the state's location is below said line
# # this is the only way to access states in cleanGDP,
# # because states share names with counties
# # we will use states to convert county names into qualifying names,
# # which is necessary to make them unique

# # get row indexes of states
# state_locs = cleanGDP[cleanGDP["County_GDP"].isna()].index + 1

# # append the state's name to all counties between it and the next state
# for i in range(len(state_locs) - 1):
#   state_1 = cleanGDP["County_GDP"].loc[state_locs[i]]
#   start = state_locs[i] + 1
#   end = state_locs[(i + 1)] - 1
#   cleanGDP["County_GDP"].loc[start:end] += " County, " + state_1
# # (need to handle the last state separately to avoid out-of-bounds error on for loop)
# state_50 = cleanGDP["County_GDP"].loc[state_locs[50]]
# start = state_locs[50] + 1
# end = 3221
# cleanGDP["County_GDP"].loc[start:end] += ", " + state_50
# cleanGDP

# # drop blank space rows
# cleanGDP = cleanGDP.dropna()

# # drop states to keep counties only
# cleanGDP = cleanGDP.drop(state_locs, axis=0)
# cleanGDP

# # even this cleaning ultimately doesn't work,
# # because the county GDP dataset contains only the county's "core name",
# # and not its suffix (county, city, etc.), making it difficult
# # to reconstruct the proper qualifying name
# # and even more, Virginia reports GDP data in combination areas,
# # mixes of cities and counties from which it's impossible to recover
# # more granular distinctions

# an extension of this project would be more intensive cleaning
# that would allow for us to recover GDP data

In [ ]:
# import fluctuant air quality data (daily AQI over 2020) from EPA
# source URL - https://aqs.epa.gov/aqsweb/airdata/download_files.html#AQI

daily_AQI = pd.read_csv(data_dir + "county_daily_AQI_2020.csv")
daily_AQI

,State Name,county Name,State Code,County Code,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting
0,Alabama,Baldwin,1,3,2020-01-01,48,Good,PM2.5,01-003-0010,1
1,Alabama,Baldwin,1,3,2020-01-04,13,Good,PM2.5,01-003-0010,1
2,Alabama,Baldwin,1,3,2020-01-07,14,Good,PM2.5,01-003-0010,1
3,Alabama,Baldwin,1,3,2020-01-10,39,Good,PM2.5,01-003-0010,1
4,Alabama,Baldwin,1,3,2020-01-13,29,Good,PM2.5,01-003-0010,1
...,...,...,...,...,...,...,...,...,...,...
324333,Wyoming,Weston,56,45,2020-12-27,32,Good,Ozone,56-045-0003,1
324334,Wyoming,Weston,56,45,2020-12-28,30,Good,Ozone,56-045-0003,1
324335,Wyoming,Weston,56,45,2020-12-29,33,Good,Ozone,56-045-0003,1
324336,Wyoming,Weston,56,45,2020-12-30,33,Good,Ozone,56-045-0003,1


In [ ]:
# import monthly climate data for non-Hawaiian counties from NOAA
# source URL - https://www.ncei.noaa.gov/access/monitoring/climate-at-a-glance/county/mapping/
# one csv must be downloaded per measure per month
# to download by hand, exploit the fact that download URL pathways are regular
# unfortunately, pd.read_csv can't handle the csv files' non-tabular shapes,
# so I have to manually delete the first three rows on all files

# import monthly precipitation

# file directory for precipitation
precip_dir = data_dir + "Climate_2020/110-pcp-"

# join all precipitation data to one dataframe
# begin by loading in January 2020 precipitation data
precip = pd.DataFrame()
precip[["Location ID", "Location", "Value", "Rank", "Anomaly (1901-2000 base period)",
        "1901-2000 Mean"]] = pd.read_csv(precip_dir + "202001-1.csv")

# using January 2020 data as scaffolding, merge in all other precipitation datasets
# these datasets all contain same number of counties; inner join works fine
# both Location ID and Location are independently unique identifiers for datasets;
# join on both to avoid unnecessary duplicates of one of the columns
for i in range(2, 10):
  df_new = pd.DataFrame()
  df_new[["Location ID", "Location", "Value", "Rank", "Anomaly (1901-2000 base period)",
          "1901-2000 Mean"]] = pd.read_csv(precip_dir + "2020" + "0" + str(i) + "-1.csv")
  precip = precip.merge(df_new, on=["Location ID", "Location"], suffixes=("_20200" + str(i-1), "_20200" + str(i)), validate="one_to_one")
for i in range(10, 13):
  df_new = pd.DataFrame()
  df_new[["Location ID", "Location", "Value", "Rank", "Anomaly (1901-2000 base period)",
          "1901-2000 Mean"]] = pd.read_csv(precip_dir + "2020" + str(i) + "-1.csv")
  precip = precip.merge(df_new, on=["Location ID", "Location"], suffixes=("_2020" + str(i-1), "_2020" + str(i)), validate="one_to_one")

precip

,Location ID,Location,Value_202001,Rank_202001,Anomaly (1901-2000 base period)_202001,1901-2000 Mean_202001,Value_202002,Rank_202002,Anomaly (1901-2000 base period)_202002,1901-2000 Mean_202002,...,Anomaly (1901-2000 base period)_202010,1901-2000 Mean_202010,Value_202011,Rank_202011,Anomaly (1901-2000 base period)_202011,1901-2000 Mean_202011,Value_202012,Rank_202012,Anomaly (1901-2000 base period)_202012,1901-2000 Mean_202012
0,AL-001,Autauga County,7.54,114,2.44,5.10,10.85,126,5.58,5.27,...,3.10,2.57,3.66,73,0.00,3.66,3.09,24,-2.00,5.09
1,AL-003,Baldwin County,5.09,86,0.30,4.79,5.04,70,0.05,4.99,...,0.94,3.35,2.23,41,-1.50,3.73,3.85,45,-1.06,4.91
2,AL-005,Barbour County,8.93,123,4.22,4.71,7.44,113,2.51,4.93,...,0.29,2.52,3.11,66,-0.23,3.34,2.34,15,-2.36,4.70
3,AL-007,Bibb County,7.33,102,1.85,5.48,12.66,128,7.35,5.31,...,3.79,2.80,3.17,54,-0.68,3.85,3.47,35,-1.74,5.21
4,AL-009,Blount County,7.30,102,1.80,5.50,11.62,127,6.40,5.22,...,3.61,3.03,2.15,23,-1.89,4.04,5.47,74,0.21,5.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3132,AK-230,Skagway Municipality,6.60,41,-1.31,7.91,8.32,74,1.66,6.66,...,-5.60,12.76,7.98,37,-2.68,10.66,30.89,98,20.88,10.01
3133,AK-240,Southeast Fairbanks Census Area,1.04,76,0.22,0.82,0.66,47,-0.12,0.78,...,0.72,1.32,0.68,29,-0.24,0.92,0.40,11,-0.53,0.93
3134,AK-275,Wrangell City and Borough,14.70,54,1.41,13.29,15.19,84,4.95,10.24,...,-8.03,20.77,21.29,80,5.00,16.29,31.63,98,16.48,15.15
3135,AK-282,Yakutat City and Borough,10.40,20,-5.71,16.11,16.03,69,2.36,13.67,...,-9.74,30.71,15.40,31,-6.04,21.44,27.63,82,7.78,19.85


In [ ]:
# import average monthly temperature

# file directory for average temperature
temp_dir = data_dir + "Climate_2020/110-tavg-"

temp = pd.DataFrame()
temp[["Location ID", "Location", "Value", "Rank", "Anomaly (1901-2000 base period)",
        "1901-2000 Mean"]] = pd.read_csv(temp_dir + "202001-1.csv")

# using January 2020 data as scaffolding, merge in all other precipitation datasets
# these datasets all contain same number of counties; inner join works fine
# both Location ID and Location are independently unique identifiers for datasets;
# join on both to avoid unnecessary duplicates of one of the columns
for i in range(2, 10):
  df_new = pd.DataFrame()
  df_new[["Location ID", "Location", "Value", "Rank", "Anomaly (1901-2000 base period)",
          "1901-2000 Mean"]] = pd.read_csv(temp_dir + "2020" + "0" + str(i) + "-1.csv")
  temp = temp.merge(df_new, on=["Location ID", "Location"], suffixes=("_20200" + str(i-1), "_20200" + str(i)), validate="one_to_one")
for i in range(10, 13):
  df_new = pd.DataFrame()
  df_new[["Location ID", "Location", "Value", "Rank", "Anomaly (1901-2000 base period)",
          "1901-2000 Mean"]] = pd.read_csv(temp_dir + "2020" + str(i) + "-1.csv")
  temp = temp.merge(df_new, on=["Location ID", "Location"], suffixes=("_2020" + str(i-1), "_2020" + str(i)), validate="one_to_one")

temp

,Location ID,Location,Value_202001,Rank_202001,Anomaly (1901-2000 base period)_202001,1901-2000 Mean_202001,Value_202002,Rank_202002,Anomaly (1901-2000 base period)_202002,1901-2000 Mean_202002,...,Anomaly (1901-2000 base period)_202010,1901-2000 Mean_202010,Value_202011,Rank_202011,Anomaly (1901-2000 base period)_202011,1901-2000 Mean_202011,Value_202012,Rank_202012,Anomaly (1901-2000 base period)_202012,1901-2000 Mean_202012
0,AL-001,Autauga County,50.7,111,4.4,46.3,52.5,101,3.3,49.2,...,3.8,64.6,59.6,122,5.3,54.3,46.8,57,-0.4,47.2
1,AL-003,Baldwin County,54.8,106,3.8,51.0,56.6,100,3.0,53.6,...,3.2,68.0,64.1,123,5.8,58.3,51.0,49,-1.3,52.3
2,AL-005,Barbour County,51.6,103,3.8,47.8,52.5,88,1.9,50.6,...,2.3,65.6,60.4,120,4.6,55.8,47.2,41,-1.6,48.8
3,AL-007,Bibb County,48.6,109,4.3,44.3,49.7,91,2.4,47.3,...,2.5,63.6,57.3,119,4.5,52.8,44.8,49,-0.8,45.6
4,AL-009,Blount County,47.1,114,5.5,41.6,48.1,100,3.6,44.5,...,3.3,61.5,55.6,117,5.0,50.6,43.5,63,0.4,43.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3132,AK-230,Skagway Municipality,9.5,28,-2.6,12.1,18.6,69,3.2,15.4,...,-0.7,29.9,17.1,26,-3.1,20.2,21.9,88,6.9,15.0
3133,AK-240,Southeast Fairbanks Census Area,-16.6,21,-9.7,-6.9,-2.9,35,-2.6,-0.3,...,0.3,21.8,1.2,31,-3.0,4.2,4.9,85,9.2,-4.3
3134,AK-275,Wrangell City and Borough,23.6,27,-2.1,25.7,30.3,70,2.2,28.1,...,-1.7,40.0,30.4,31,-1.4,31.8,33.3,91,6.0,27.3
3135,AK-282,Yakutat City and Borough,11.8,14,-5.4,17.2,22.8,60,2.4,20.4,...,0.3,33.0,21.5,27,-2.5,24.0,25.4,82,5.8,19.6


In [ ]:
# import weekly 2020 COVID data from Social Explorer
# compiles weekly statistics on cases, deaths,
# and (more sparsely) testing and COVID hospitalizations
# source URL - https://www.socialexplorer.com/tables/COVID19/R13227100

COVID = pd.read_csv(data_dir + "county_COVID_2020.csv")
COVID

<ipython-input-17-ee298dc863f7>:6: DtypeWarning: Columns (0,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,

,FIPS,Name of Area,Qualifying Name,Nation,State,County,Metropolitan Statistical Area,Cumulative Number of Confirmed Cases as of Dec 31 2020,Cumulative Number of Deaths as of Dec 31 2020,Number of Currently Hospitalized as of Dec 31 2020,...,Change in Cumulative Confirmed Cases as of Jan 21 2020 from Previous Month,Percent Change in Cumulative Confirmed Cases as of Jan 21 2020 from Previous Month,Change in Cumulative Deaths as of Jan 21 2020 from Previous Week,Percent Change in Cumulative Deaths as of Jan 21 2020 from Previous Week,Change in Cumulative Deaths as of Jan 21 2020 from Previous Month,Percent Change in Cumulative Deaths as of Jan 21 2020 from Previous Month,Cumulative Percent of Population with Confirmed Cases as of Jan 21 2020,Cumulative Percent of Population Deaths as of Jan 21 2020,"Cumulative Confirmed Cases Rate per 100,000 as of Jan 21 2020","Cumulative Death Rate per 100,000 as of Jan 21 2020"
0,Geo_FIPS,Geo_NAME,Geo_QNAME,Geo_NATION,Geo_STATE,Geo_COUNTY,Geo_MSA,ORG_A123120,ORG_B123120,ORG_C123120,...,ORG_K012120,ORG_L012120,ORG_M012120,ORG_N012120,ORG_O012120,ORG_P012120,ORG_R012120,ORG_S012120,ORG_T012120,ORG_U012120
1,01001,Autauga County,"Autauga County, Alabama",00,01,001,NaN,4190,48,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01003,Baldwin County,"Baldwin County, Alabama",00,01,003,NaN,13601,161,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01005,Barbour County,"Barbour County, Alabama",00,01,005,NaN,1514,32,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01007,Bibb County,"Bibb County, Alabama",00,01,007,NaN,1834,46,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3276,72999,Unknown County,"Unknown County, Puerto Rico",0,72,999,NaN,3875.0,1503.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3277,78010,St. Croix County,"St. Croix County, Virgin Islands",0,78,10,NaN,829.0,7.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3278,78020,St. John County,"St. John County, Virgin Islands",0,78,20,NaN,174.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3279,78030,St. Thomas County,"St. Thomas County, Virgin Islands",0,78,30,NaN,1028.0,15.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# import county adjacency file from Census Bureau
# enables simple spatial autoregression (SAR)
# source URL - https://www.census.gov/geographies/reference-files/2010/geo/county-adjacency.html
# more info on SAR: https://www.sciencedirect.com/topics/earth-and-planetary-sciences/spatial-autoregressive-model
# before reading .txt file, manually changed encoding from ANSI to UTF-8
# no good adjacency matrix exists
# (matrices on Github contain a suspiciously low number of rows/counties)

# given a line of text, isolates FIPS numeric codes
# returns FIPS codes as list of strings
def find_FIPS(line):
  fips = []
  for word in line.split():
    if word.isdigit():
      fips.append(word)
  return fips

# first pass: read in file line-by-line
# build set of all counties in the adjacency file
counties = set()
with open(data_dir + "county_adjacency.txt", encoding="UTF-8") as county_adjacency:
  for line in county_adjacency:
    for FIPS in find_FIPS(line):
      counties.add(FIPS)

# convert set of counties to ordered list
counties = sorted(counties)
counties

# create adjacency matrix dataframe
# we want both columns and rows (index) to be the list of counties
adjacency = pd.DataFrame(columns=counties, index=counties)
adjacency = adjacency.fillna(0)
adjacency

# second pass: read in file line-by-line, again
# update dataframe values for 1 when counties are bordering
# a county, for our purposes, does not border itself

# initialize row of county, set of columns of counties
columns = set()
row = adjacency.index[0]

with open(data_dir + "county_adjacency.txt", encoding="UTF-8") as county_adjacency:
  # loop through all lines in file, isolating FIPS codes
  for line in county_adjacency:
    county_list = find_FIPS(line)
    # if more than 1 FIPS codes in the line, the line is a header
    # first FIPS code represents new row
    if len(county_list) > 1:
      # reassign all previous row, column pairs in adjacency to 1
      # (convert set of columns to sorted list to apply to dataframe)
      adjacency.loc[row, sorted(columns)] = 1
      # reset row
      row = county_list[0]
      # reset columns set
      columns.clear()
      # add the second FIPS code to columns set if it doesn't equal the row
      if county_list[1] != row:
        columns.add(county_list[1])
    else:
      # for lines with 1 FIPS code, add FIPS to set if it doesn't equal the row
      # (ensures no county marked as bordering itself)
      if county_list[0] != row:
        columns.add(county_list[0])

adjacency

,01001,01003,01005,01007,01009,01011,01013,01015,01017,01019,...,72141,72143,72145,72147,72149,72151,72153,78010,78020,78030
01001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
01003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
01005,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
01007,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
01009,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72151,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
72153,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
78010,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
78020,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
# import monthly 2020 COVID data from Social Explorer
# compiles weekly statistics on cases, deaths,
# labor force participation and mobility (from satellite images)
# and (more sparsely) testing and COVID hospitalizations
# source URL - https://www.socialexplorer.com/tables/COVID19_INSIGHTS/R13227245
# mobility data was not used, since it's very sparse
# (different counties miss different values for different months)
# and not adjusted for seasonality (values given are relative to January baseline,
# and not absolute)

COVID_insights = pd.read_csv(data_dir + "county_COVID_insights_2020.csv")
COVID_insights

<ipython-input-19-2276d725b7ea>:11: DtypeWarning: Columns (0,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263

,FIPS,Name of Area,Qualifying Name,State,County,New Confirmed Cases for December 2020,"New Confirmed Cases for December 2020 per 100,000",New Confirmed Deaths for December 2020,"New Confirmed Deaths for December 2020 per 100,000","Google Mobility in Retail and Recreational Places (percent change from trend, monthly average)",...,Total Population.11,Total Population: Under 18 Years.11,Total Population: 18 to 34 Years.11,Total Population: 35 to 64 Years.11,Total Population: 65 and Over.11,Total Population: White Alone.11,Total Population: Black or African American Alone.11,Total Population: Hispanic Or Latino.11,Median Household Income.11,Median Family Income.11
0,Geo_FIPS,Geo_NAME,Geo_QName,Geo_STATE,Geo_COUNTY,TT_T012_001,TT_T012_002,TT_T012_003,TT_T012_004,TT_T012_005,...,TT_T001_005,TT_T001_006,TT_T001_007,TT_T001_008,TT_T001_009,TT_T001_010,TT_T001_011,TT_T001_012,TT_T001_013,TT_T001_014
1,01001,Autauga County,"Autauga County, Alabama",01,001,1410,2554.35,6,10.87,-1.87096774193548,...,55200,13369,11729,22052,8050,42437,10565,1528,58786,73530
2,01003,Baldwin County,"Baldwin County, Alabama",01,003,4711,2263.74,63,30.27,-8.2258064516129,...,208107,45677,38767,82998,40665,179526,19764,9353,55962,71951
3,01005,Barbour County,"Barbour County, Alabama",01,005,336,1303.23,21,81.45,-4.89285714285714,...,25782,5436,5848,9864,4634,12216,12266,1106,34186,44339
4,01007,Bibb County,"Bibb County, Alabama",01,007,638,2832.16,29,128.73,NaN,...,22527,4659,5163,9044,3661,17268,5018,547,45340,54840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3216,72145,Vega Baja Municipio,"Vega Baja Municipio, Puerto Rico",72,145,501,938.71,0,0.0,NaN,...,53371,11018,11822,20568,9963,45626,2725,51797,19096,24186
3217,72147,Vieques Municipio,"Vieques Municipio, Puerto Rico",72,147,69,786.68,0,0.0,NaN,...,8771,1786,1730,3429,1826,5296,856,8247,15539,22014
3218,72149,Villalba Municipio,"Villalba Municipio, Puerto Rico",72,149,96,417.52,0,0.0,NaN,...,22993,5184,5318,8844,3647,12293,647,22936,19855,23736
3219,72151,Yabucoa Municipio,"Yabucoa Municipio, Puerto Rico",72,151,155,453.89,0,0.0,NaN,...,34149,6922,7237,13650,6340,10487,22748,34122,16013,19978


In [ ]:
# # import policies data from CUSP
# # specifically, examine COVID prevention policies
# # due to time constraints, we ultimately will not examine COVID prevention policies
# # (we've kept this data here since we anyway collected it);
# # a future extension of this project would be to do so

# # import state closure policy data
# # source URL - https://statepolicies.com/data/graphs/reopening-2/
# closures = pd.read_csv(data_dir + "state_closures.csv")
# closures

# # import mask mandate policy data
# # source URL - https://statepolicies.com/data/graphs/face-masks/
# mask_mandates = pd.read_csv(data_dir + "state_mask_mandates.csv")
# mask_mandates

# # import interstate travel quarantine policy data
# # source URL - https://statepolicies.com/data/graphs/interstate-travel-quarantines/
# interstate_quarantines = pd.read_csv(data_dir + "state_interstate_quarantines.csv")
# interstate_quarantines

# # import stay-at-home order policy data
# # source URL - https://statepolicies.com/data/graphs/stay-at-home-order/
# stay_at_home = pd.read_csv(data_dir + "state_stay-at-home.csv")
# stay_at_home

**Step 2: Cleaning & Aggregating Coarse-Grained Data**

In [ ]:
# ACS data is highly-redundant, extract variables we deem necessary
# we seek - geographic identifiers; population; age; sex; race; poverty; inequality;
# work commute; education; home ownership
# some relevant columns (ex. age-by-sex data) occur in large ranges,
# so we extract that data numerically

# geography, population, inequality, work commute, home ownership
some_cols = ["FIPS", "Area Name", "Qualifying Name", "State Postal Abbreviation", "Total Population",
                    "Population Density (Per Sq. Mile)", "Total Population: Male", "Total Population: Female",
                    "Median Age:", "Gini Index", "Average Commute to Work (In Min)", "Owner Occupied Housing Units", "Housing Units"]

ACScols = list(ACS.columns)

# age
age_start = ACScols.index("Total Population: Under 5 Years")
age_end = ACScols.index("Total Population: 85 Years and Over")
age = ACScols[age_start: age_end + 1]

# race
race_start = ACScols.index("Total Population: Not Hispanic or Latino")
race_end = ACScols.index("Total Population: Hispanic or Latino: Two or More Races")
race = ACScols[race_start: race_end + 1]

# poverty
poverty_start = ACScols.index("Population for Whom Poverty Status Is Determined:")
poverty_end = ACScols.index("Population for Whom Poverty Status Is Determined: 2.00 and Over")
poverty = ACScols[poverty_start: poverty_end + 1]

# education
educ_start = ACScols.index("Population 25 Years and Over: Less than High School")
educ_end = ACScols.index("Population 25 Years and Over: Doctorate Degree")
educ = ACScols[educ_start: educ_end + 1]

# extract relevant variables
relevant_cols = some_cols + age + race + poverty + educ
cleanACS = ACS[relevant_cols]

# one row per US county
# (exclude Puerto Rico, Alaska, Hawaii & eliminate double header)
cleanACS = cleanACS.iloc[1:]
cleanACS = cleanACS[cleanACS["State Postal Abbreviation"] != "pr"]
cleanACS = cleanACS[cleanACS["State Postal Abbreviation"] != "ak"]
cleanACS = cleanACS[cleanACS["State Postal Abbreviation"] != "hi"]

# some datasets store FIPS as strings with leading zeroes,
# while some store numerically
# add numeric storage of FIPS to cleanACS to facilitate future merges
cleanACS["FIPS_num"] = pd.to_numeric(cleanACS["FIPS"])

cleanACS

,FIPS,Area Name,Qualifying Name,State Postal Abbreviation,Total Population,Population Density (Per Sq. Mile),Total Population: Male,Total Population: Female,Median Age:,Gini Index,...,Population for Whom Poverty Status Is Determined: 1.50 to 1.99,Population for Whom Poverty Status Is Determined: 2.00 and Over,Population 25 Years and Over: Less than High School,Population 25 Years and Over: High School Graduate (Includes Equivalency),Population 25 Years and Over: Some College,Population 25 Years and Over: Bachelor's Degree,Population 25 Years and Over: Master's Degree,Population 25 Years and Over: Professional School Degree,Population 25 Years and Over: Doctorate Degree,FIPS_num
1,01001,Autauga County,"Autauga County, Alabama",al,55380,93.16273,26934,28446,38.2,0.4542,...,4199,37887,4291,12551,10596,6019,2875,499,536,1001
2,01003,Baldwin County,"Baldwin County, Alabama",al,212830,133.8703,103496,109334,43,0.4587,...,20926,150048,13893,41797,47274,31801,11812,2912,1623,1003
3,01005,Barbour County,"Barbour County, Alabama",al,25361,28.65624,13421,11940,40.4,0.4883,...,2621,10169,4812,6396,4676,1367,495,128,90,1005
4,01007,Bibb County,"Bibb County, Alabama",al,22493,36.13558,12150,10343,40.9,0.4487,...,3001,11905,3386,7256,3848,1043,488,80,67,1007
5,01009,Blount County,"Blount County, Alabama",al,57681,89.45139,28495,29186,40.7,0.457,...,6364,36423,7763,13299,13519,3432,1393,266,119,1009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,56037,Sweetwater County,"Sweetwater County, Wyoming",wy,43521,4.173883,22471,21050,35.3,0.4065,...,3567,32136,2017,9239,10415,4136,1563,356,236,56037
3139,56039,Teton County,"Teton County, Wyoming",wy,23280,5.824592,12325,10955,39.3,0.5003,...,1878,18316,834,2577,4037,6747,2399,401,328,56039
3140,56041,Uinta County,"Uinta County, Wyoming",wy,20479,9.837535,10406,10073,35.8,0.3861,...,1478,14627,941,5383,4562,1433,557,63,25,56041
3141,56043,Washakie County,"Washakie County, Wyoming",wy,8027,3.585605,4065,3962,42.9,0.3868,...,673,5538,568,1650,2031,854,337,87,19,56043


In [ ]:
# create coarse dataframe for all coarsely-grained data
# (i.e. data where there's one observation per county)
# merge coarse-grained dataframes into coarse after cleaning

# first, renaming masks columns for clarity
masks_ = masks.add_suffix("_masks")

In [ ]:
# second, perform the merge
# masks dataframe stores FIPS numerically
# number of rows has been preserved = no lost counties
coarse = cleanACS.merge(masks_, left_on="FIPS_num", right_on="COUNTYFP_masks",
                        validate="one_to_one").drop("COUNTYFP_masks", axis=1)
coarse

,FIPS,Area Name,Qualifying Name,State Postal Abbreviation,Total Population,Population Density (Per Sq. Mile),Total Population: Male,Total Population: Female,Median Age:,Gini Index,...,Population 25 Years and Over: Bachelor's Degree,Population 25 Years and Over: Master's Degree,Population 25 Years and Over: Professional School Degree,Population 25 Years and Over: Doctorate Degree,FIPS_num,NEVER_masks,RARELY_masks,SOMETIMES_masks,FREQUENTLY_masks,ALWAYS_masks
0,01001,Autauga County,"Autauga County, Alabama",al,55380,93.16273,26934,28446,38.2,0.4542,...,6019,2875,499,536,1001,0.053,0.074,0.134,0.295,0.444
1,01003,Baldwin County,"Baldwin County, Alabama",al,212830,133.8703,103496,109334,43,0.4587,...,31801,11812,2912,1623,1003,0.083,0.059,0.098,0.323,0.436
2,01005,Barbour County,"Barbour County, Alabama",al,25361,28.65624,13421,11940,40.4,0.4883,...,1367,495,128,90,1005,0.067,0.121,0.120,0.201,0.491
3,01007,Bibb County,"Bibb County, Alabama",al,22493,36.13558,12150,10343,40.9,0.4487,...,1043,488,80,67,1007,0.020,0.034,0.096,0.278,0.572
4,01009,Blount County,"Blount County, Alabama",al,57681,89.45139,28495,29186,40.7,0.457,...,3432,1393,266,119,1009,0.053,0.114,0.180,0.194,0.459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3103,56037,Sweetwater County,"Sweetwater County, Wyoming",wy,43521,4.173883,22471,21050,35.3,0.4065,...,4136,1563,356,236,56037,0.061,0.295,0.230,0.146,0.268
3104,56039,Teton County,"Teton County, Wyoming",wy,23280,5.824592,12325,10955,39.3,0.5003,...,6747,2399,401,328,56039,0.095,0.157,0.160,0.247,0.340
3105,56041,Uinta County,"Uinta County, Wyoming",wy,20479,9.837535,10406,10073,35.8,0.3861,...,1433,557,63,25,56041,0.098,0.278,0.154,0.207,0.264
3106,56043,Washakie County,"Washakie County, Wyoming",wy,8027,3.585605,4065,3962,42.9,0.3868,...,854,337,87,19,56043,0.204,0.155,0.069,0.285,0.287


In [ ]:
# clean CDC risk factors data (extracting and renaming relevant columns)
# when CDC lacks data for a county, it assigns a value of -1 (effectively a "null")
# for most dataframes, no obvious way to fill in "nulls"
# counties that don't report heartdeaths, for example, are likely underresourced
# and thus probably have heartdeaths above the average,
# but we do not know by how much above the average
# so we filter out all counties without this data
# (especially since counties without CDC data are few,
# and data for them is likely to be missing in other datasets as well)

# but there's one caveat to this, as for some dataframes,
# CDC is bizarrely missing data from all NJ, due to an administrative error,
# not a reflection of underlying data paucity
# we will fill in NJ data with the average of all counties in adjacent states (NY, PA, DE)

# first, extract all NJ counties
NJ_counties = coarse[coarse["State Postal Abbreviation"] == "nj"]["FIPS_num"]
one_NJ_county = NJ_counties.iloc[0]
NJ_counties, one_NJ_county

# now, extract all counties in states adjacent to NJ
states_adjacent_NJ = coarse[(coarse["State Postal Abbreviation"] == "ny") | (coarse["State Postal Abbreviation"] == "pa") | (coarse["State Postal Abbreviation"] == "de")]["FIPS_num"]
states_adjacent_NJ

def fill_CDC_nulls(df):
  # if one NJ county is missing, all are missing
  # in the below if statement, we produce a boolean series,
  # then use .iloc[0] to extract its one value (separate from index)
  if (df[df["cnty_fips"] == one_NJ_county]["Value"] == -1).iloc[0]:
    # take average value of all counties in states adjacent to NJ
    adj_states_avg = df[df["cnty_fips"].isin(states_adjacent_NJ)]["Value"].mean()
    # fill NJ counties with that value
    # some Python idiosyncrasies - you need to use .loc[] to reassign dataframe values
    row_loc = df[df["cnty_fips"].isin(NJ_counties)].index
    df.loc[row_loc, "Value"] = adj_states_avg
  # for all non-NJ counties, drop rows with -1 values for reasons outlined earlier
  df = df[df["Value"] != -1]
  return df

In [ ]:
# apply above function to clean data

smokers = fill_CDC_nulls(smokers)
smokers = smokers[["cnty_fips", "Value"]].rename({"Value": "smokers"}, axis=1)

diabetes = fill_CDC_nulls(diabetes)
diabetes = diabetes[["cnty_fips", "Value"]].rename({"Value": "diabetes"}, axis=1)

obesity = fill_CDC_nulls(obesity)
obesity = obesity[["cnty_fips", "Value"]].rename({"Value": "obesity"}, axis=1)

inactive = fill_CDC_nulls(inactive)
inactive = inactive[["cnty_fips", "Value"]].rename({"Value": "inactive"}, axis=1)

heart = fill_CDC_nulls(heart)
heart = heart[["cnty_fips", "Value"]].rename({"Value": "heartdeaths"}, axis=1)

strokes = fill_CDC_nulls(strokes)
strokes = strokes[["cnty_fips", "Value"]].rename({"Value": "strokedeaths"}, axis=1)

hypertension = fill_CDC_nulls(hypertension)
hypertension = hypertension[["cnty_fips", "Value"]].rename({"Value": "hypertension"}, axis=1)

# CDC risk factors dataframes store FIPS numerically
# perform inner-merge - we want to keep counties present across all datasets
coarse = coarse.merge(smokers, left_on="FIPS_num", right_on="cnty_fips",
                      validate="one_to_one").drop("cnty_fips", axis=1)
coarse = coarse.merge(diabetes, left_on="FIPS_num", right_on="cnty_fips",
                      validate="one_to_one").drop("cnty_fips", axis=1)
coarse = coarse.merge(obesity, left_on="FIPS_num", right_on="cnty_fips",
                      validate="one_to_one").drop("cnty_fips", axis=1)
coarse = coarse.merge(inactive, left_on="FIPS_num", right_on="cnty_fips",
                      validate="one_to_one").drop("cnty_fips", axis=1)
coarse = coarse.merge(heart, left_on="FIPS_num", right_on="cnty_fips",
                      validate="one_to_one").drop("cnty_fips", axis=1)
coarse = coarse.merge(strokes, left_on="FIPS_num", right_on="cnty_fips",
                      validate="one_to_one").drop("cnty_fips", axis=1)
coarse = coarse.merge(hypertension, left_on="FIPS_num", right_on="cnty_fips",
                      validate="one_to_one").drop("cnty_fips", axis=1)

coarse
# based on number of rows, we've retained NJ counties!

,FIPS,Area Name,Qualifying Name,State Postal Abbreviation,Total Population,Population Density (Per Sq. Mile),Total Population: Male,Total Population: Female,Median Age:,Gini Index,...,SOMETIMES_masks,FREQUENTLY_masks,ALWAYS_masks,smokers,diabetes,obesity,inactive,heartdeaths,strokedeaths,hypertension
0,01001,Autauga County,"Autauga County, Alabama",al,55380,93.16273,26934,28446,38.2,0.4542,...,0.134,0.295,0.444,18.0,9.6,30.9,26.6,198.0,55.0,76.9
1,01003,Baldwin County,"Baldwin County, Alabama",al,212830,133.8703,103496,109334,43,0.4587,...,0.098,0.323,0.436,16.1,7.9,27.7,22.8,188.6,42.6,74.0
2,01005,Barbour County,"Barbour County, Alabama",al,25361,28.65624,13421,11940,40.4,0.4883,...,0.120,0.201,0.491,24.8,11.3,24.8,27.5,278.5,49.6,110.9
3,01007,Bibb County,"Bibb County, Alabama",al,22493,36.13558,12150,10343,40.9,0.4487,...,0.096,0.278,0.572,22.4,10.2,39.3,27.8,256.1,57.8,76.8
4,01009,Blount County,"Blount County, Alabama",al,57681,89.45139,28495,29186,40.7,0.457,...,0.180,0.194,0.459,21.0,9.5,35.8,25.5,225.6,49.6,66.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3100,56037,Sweetwater County,"Sweetwater County, Wyoming",wy,43521,4.173883,22471,21050,35.3,0.4065,...,0.230,0.146,0.268,17.5,7.6,33.8,21.4,160.1,29.2,142.8
3101,56039,Teton County,"Teton County, Wyoming",wy,23280,5.824592,12325,10955,39.3,0.5003,...,0.160,0.247,0.340,12.2,4.2,15.4,12.0,94.9,26.2,29.9
3102,56041,Uinta County,"Uinta County, Wyoming",wy,20479,9.837535,10406,10073,35.8,0.3861,...,0.154,0.207,0.264,18.6,7.3,28.3,23.8,180.6,27.1,115.4
3103,56043,Washakie County,"Washakie County, Wyoming",wy,8027,3.585605,4065,3962,42.9,0.3868,...,0.069,0.285,0.287,16.7,8.8,24.5,19.8,160.9,33.8,127.6


In [ ]:
# check that we've retained NJ state data
coarse[coarse["FIPS_num"] == one_NJ_county]

,FIPS,Area Name,Qualifying Name,State Postal Abbreviation,Total Population,Population Density (Per Sq. Mile),Total Population: Male,Total Population: Female,Median Age:,Gini Index,...,SOMETIMES_masks,FREQUENTLY_masks,ALWAYS_masks,smokers,diabetes,obesity,inactive,heartdeaths,strokedeaths,hypertension
1737,34001,Atlantic County,"Atlantic County, New Jersey",nj,266105,479.0241,128791,137314,41.7,0.4716,...,0.03,0.176,0.734,15.9,8.624242,29.705303,22.781818,198.0,30.6,146.6


In [ ]:
# for the following dataframes, we can fill in all "nulls"
# counties without such data are remote and underresourced
# (no administrative flaws with NJ)
# likely have great air quality and lack doctors & broadband
# fill these in with 0
AQI = AQI[["cnty_fips", "Value"]].rename({"Value": "AQI"}, axis=1)
AQI = AQI.replace(to_replace=-1, value=0)
doctors = doctors[["cnty_fips", "Value"]].rename({"Value": "doctors"}, axis=1)
doctors = doctors.replace(to_replace=-1, value=0)
# broadband measures % without access, so fill in with 100
broadband = broadband[["cnty_fips", "Value"]].rename({"Value": "broadband"}, axis=1)
broadband = broadband.replace(to_replace=-1, value=100)

# complete merge
coarse = coarse.merge(AQI, left_on="FIPS_num", right_on="cnty_fips",
                      validate="one_to_one").drop("cnty_fips", axis=1)
coarse = coarse.merge(broadband, left_on="FIPS_num", right_on="cnty_fips",
                      validate="one_to_one").drop("cnty_fips", axis=1)
coarse = coarse.merge(doctors, left_on="FIPS_num", right_on="cnty_fips",
                      validate="one_to_one").drop("cnty_fips", axis=1)
coarse

,FIPS,Area Name,Qualifying Name,State Postal Abbreviation,Total Population,Population Density (Per Sq. Mile),Total Population: Male,Total Population: Female,Median Age:,Gini Index,...,smokers,diabetes,obesity,inactive,heartdeaths,strokedeaths,hypertension,AQI,broadband,doctors
0,01001,Autauga County,"Autauga County, Alabama",al,55380,93.16273,26934,28446,38.2,0.4542,...,18.0,9.6,30.9,26.6,198.0,55.0,76.9,10.4,17.3,2.2
1,01003,Baldwin County,"Baldwin County, Alabama",al,212830,133.8703,103496,109334,43,0.4587,...,16.1,7.9,27.7,22.8,188.6,42.6,74.0,7.2,14.9,1.4
2,01005,Barbour County,"Barbour County, Alabama",al,25361,28.65624,13421,11940,40.4,0.4883,...,24.8,11.3,24.8,27.5,278.5,49.6,110.9,9.4,35.4,2.7
3,01007,Bibb County,"Bibb County, Alabama",al,22493,36.13558,12150,10343,40.9,0.4487,...,22.4,10.2,39.3,27.8,256.1,57.8,76.8,10.0,23.9,1.7
4,01009,Blount County,"Blount County, Alabama",al,57681,89.45139,28495,29186,40.7,0.457,...,21.0,9.5,35.8,25.5,225.6,49.6,66.0,10.6,20.4,4.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3100,56037,Sweetwater County,"Sweetwater County, Wyoming",wy,43521,4.173883,22471,21050,35.3,0.4065,...,17.5,7.6,33.8,21.4,160.1,29.2,142.8,5.0,13.3,2.2
3101,56039,Teton County,"Teton County, Wyoming",wy,23280,5.824592,12325,10955,39.3,0.5003,...,12.2,4.2,15.4,12.0,94.9,26.2,29.9,4.5,10.9,0.9
3102,56041,Uinta County,"Uinta County, Wyoming",wy,20479,9.837535,10406,10073,35.8,0.3861,...,18.6,7.3,28.3,23.8,180.6,27.1,115.4,6.2,8.6,2.2
3103,56043,Washakie County,"Washakie County, Wyoming",wy,8027,3.585605,4065,3962,42.9,0.3868,...,16.7,8.8,24.5,19.8,160.9,33.8,127.6,4.1,17.2,1.6


In [ ]:
# clean insurers data (extracting and renaming relevant columns)
insurers_num = insurers.loc[:, "2014":"2020"]
insurers_num = insurers_num.add_suffix("_insurers")
insurers_num["FIPS"] = insurers["FIPS"]

# join to coarse data
coarse = coarse.merge(insurers_num, left_on="FIPS_num", right_on="FIPS",
                      validate="one_to_one", suffixes=(None, "_y")).drop("FIPS_y", axis=1)
coarse

,FIPS,Area Name,Qualifying Name,State Postal Abbreviation,Total Population,Population Density (Per Sq. Mile),Total Population: Male,Total Population: Female,Median Age:,Gini Index,...,AQI,broadband,doctors,2014_insurers,2015_insurers,2016_insurers,2017_insurers,2018_insurers,2019_insurers,2020_insurers
0,01001,Autauga County,"Autauga County, Alabama",al,55380,93.16273,26934,28446,38.2,0.4542,...,10.4,17.3,2.2,1,2,2,1,1,1,1
1,01003,Baldwin County,"Baldwin County, Alabama",al,212830,133.8703,103496,109334,43,0.4587,...,7.2,14.9,1.4,1,2,2,1,1,1,1
2,01005,Barbour County,"Barbour County, Alabama",al,25361,28.65624,13421,11940,40.4,0.4883,...,9.4,35.4,2.7,1,2,2,1,1,1,1
3,01007,Bibb County,"Bibb County, Alabama",al,22493,36.13558,12150,10343,40.9,0.4487,...,10.0,23.9,1.7,1,2,2,1,1,1,1
4,01009,Blount County,"Blount County, Alabama",al,57681,89.45139,28495,29186,40.7,0.457,...,10.6,20.4,4.4,1,2,3,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3100,56037,Sweetwater County,"Sweetwater County, Wyoming",wy,43521,4.173883,22471,21050,35.3,0.4065,...,5.0,13.3,2.2,2,2,1,1,1,1,1
3101,56039,Teton County,"Teton County, Wyoming",wy,23280,5.824592,12325,10955,39.3,0.5003,...,4.5,10.9,0.9,2,2,1,1,1,1,1
3102,56041,Uinta County,"Uinta County, Wyoming",wy,20479,9.837535,10406,10073,35.8,0.3861,...,6.2,8.6,2.2,2,2,1,1,1,1,1
3103,56043,Washakie County,"Washakie County, Wyoming",wy,8027,3.585605,4065,3962,42.9,0.3868,...,4.1,17.2,1.6,2,2,1,1,1,1,1


In [ ]:
# clean elections data (extracting and renaming relevant data)
# election results are meant to track political affiliation
# since Trump caused such an electoral shift in American politics,
# election data pre-2016 isn't really relevant
elections = elections[(elections["year"] == 2016) | (elections["year"] == 2020)]
elections = elections[["year", "county_fips", "party", "candidatevotes", "totalvotes"]]

# get percentages for each party
trump2016 = elections[(elections["party"] == "REPUBLICAN") & (elections["year"] == 2016)]
elections["Trump2016"] = trump2016["candidatevotes"] / trump2016["totalvotes"] * 100

clinton2016 = elections[(elections["party"] == "DEMOCRAT") & (elections["year"] == 2016)]
elections["Clinton2016"] = clinton2016["candidatevotes"] / clinton2016["totalvotes"] * 100

trump2020 = elections[(elections["party"] == "REPUBLICAN") & (elections["year"] == 2020)]
elections["Trump2020"] = trump2020["candidatevotes"] / trump2020["totalvotes"] * 100

biden2020 = elections[(elections["party"] == "DEMOCRAT") & (elections["year"] == 2020)]
elections["Biden2020"] = biden2020["candidatevotes"] / biden2020["totalvotes"] * 100

# group by county
election_results = elections.groupby("county_fips").sum().loc[:, "Trump2016":"Biden2020"].reset_index()

# calculate 3rd-party voters (faster than aggregating across America's many 3rd-parties)
election_results["Other2016"] = 100 - (election_results["Trump2016"] + election_results["Clinton2016"])
election_results["Other2020"] = 100 - (election_results["Trump2020"] + election_results["Biden2020"])
election_results

,county_fips,Trump2016,Clinton2016,Trump2020,Biden2020,Other2016,Other2020
0,1001.0,72.766588,23.769671,71.436802,27.018365,3.463741,1.544833
1,1003.0,76.545712,19.385601,76.171373,22.409030,4.068687,1.419597
2,1005.0,52.096666,46.527844,53.451226,45.788173,1.375490,0.760601
3,1007.0,76.403220,21.249575,78.426264,20.698280,2.347205,0.875456
4,1009.0,89.334844,8.425825,89.571553,9.569378,2.239331,0.859069
...,...,...,...,...,...,...,...
3150,56037.0,70.951547,18.861646,73.236316,22.894957,10.186807,3.868727
3151,56039.0,31.052507,57.923497,29.356868,66.599040,11.023996,4.044093
3152,56041.0,72.656434,14.191263,79.247278,16.819960,13.152302,3.932762
3153,56043.0,76.324069,13.948610,80.481151,16.145833,9.727320,3.373016


In [ ]:
# merge election results into coarse
coarse = coarse.merge(election_results, left_on="FIPS_num", right_on="county_fips",
                      validate="one_to_one")
coarse

,FIPS,Area Name,Qualifying Name,State Postal Abbreviation,Total Population,Population Density (Per Sq. Mile),Total Population: Male,Total Population: Female,Median Age:,Gini Index,...,2018_insurers,2019_insurers,2020_insurers,county_fips,Trump2016,Clinton2016,Trump2020,Biden2020,Other2016,Other2020
0,01001,Autauga County,"Autauga County, Alabama",al,55380,93.16273,26934,28446,38.2,0.4542,...,1,1,1,1001.0,72.766588,23.769671,71.436802,27.018365,3.463741,1.544833
1,01003,Baldwin County,"Baldwin County, Alabama",al,212830,133.8703,103496,109334,43,0.4587,...,1,1,1,1003.0,76.545712,19.385601,76.171373,22.409030,4.068687,1.419597
2,01005,Barbour County,"Barbour County, Alabama",al,25361,28.65624,13421,11940,40.4,0.4883,...,1,1,1,1005.0,52.096666,46.527844,53.451226,45.788173,1.375490,0.760601
3,01007,Bibb County,"Bibb County, Alabama",al,22493,36.13558,12150,10343,40.9,0.4487,...,1,1,1,1007.0,76.403220,21.249575,78.426264,20.698280,2.347205,0.875456
4,01009,Blount County,"Blount County, Alabama",al,57681,89.45139,28495,29186,40.7,0.457,...,1,1,1,1009.0,89.334844,8.425825,89.571553,9.569378,2.239331,0.859069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3100,56037,Sweetwater County,"Sweetwater County, Wyoming",wy,43521,4.173883,22471,21050,35.3,0.4065,...,1,1,1,56037.0,70.951547,18.861646,73.236316,22.894957,10.186807,3.868727
3101,56039,Teton County,"Teton County, Wyoming",wy,23280,5.824592,12325,10955,39.3,0.5003,...,1,1,1,56039.0,31.052507,57.923497,29.356868,66.599040,11.023996,4.044093
3102,56041,Uinta County,"Uinta County, Wyoming",wy,20479,9.837535,10406,10073,35.8,0.3861,...,1,1,1,56041.0,72.656434,14.191263,79.247278,16.819960,13.152302,3.932762
3103,56043,Washakie County,"Washakie County, Wyoming",wy,8027,3.585605,4065,3962,42.9,0.3868,...,1,1,1,56043.0,76.324069,13.948610,80.481151,16.145833,9.727320,3.373016


In [ ]:
# clean state politics data (extracting and renaming relevant data)

# setting first row as header
state_politics.columns = state_politics.iloc[0]
states_pol = state_politics.drop(0, axis=0)

# extracting relevant data and naming it properly
# chiefly, converting categorical to quantitative, and then cleaning up
states_pol = states_pol.loc[:50, "Legis.\nControl":]
naming = {"Legis.\nControl_Dem": "Legisl_D", "Legis.\nControl_Rep": "Legisl_R",
          "Gov.\nParty_Dem": "Gov_D", "Gov.\nParty_Rep": "Gov_R"}
states_pol = pd.get_dummies(states_pol).rename(
    naming, axis=1)[["Legisl_D", "Legisl_R", "Gov_D", "Gov_R"]]
states_pol["State"] = state_politics["STATE"]

# Nebraska's legislature is technically nonpartisan but basically Republican
states_pol["Legisl_R"][27] = 1

states_pol

<ipython-input-31-8b5d080da22b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  states_pol["Legisl_R"][27] = 1


,Legisl_D,Legisl_R,Gov_D,Gov_R,State
1,0,1,0,1,Alabama
2,0,1,0,1,Alaska
3,0,1,0,1,Arizona
4,0,1,0,1,Arkansas
5,1,0,1,0,California
6,1,0,1,0,Colorado
7,1,0,1,0,Connecticut
8,1,0,1,0,Delaware
9,0,1,0,1,Florida
10,0,1,0,1,Georgia


In [ ]:
# extract county states from their qualifying names in coarse
# (prepares to merge with states_pol dataframe)
def get_state(name):
  first_letter = name.find(",") + 2
  return name[first_letter:]

coarse["States"] = [get_state(name) for name in coarse["Qualifying Name"]]
coarse

,FIPS,Area Name,Qualifying Name,State Postal Abbreviation,Total Population,Population Density (Per Sq. Mile),Total Population: Male,Total Population: Female,Median Age:,Gini Index,...,2019_insurers,2020_insurers,county_fips,Trump2016,Clinton2016,Trump2020,Biden2020,Other2016,Other2020,States
0,01001,Autauga County,"Autauga County, Alabama",al,55380,93.16273,26934,28446,38.2,0.4542,...,1,1,1001.0,72.766588,23.769671,71.436802,27.018365,3.463741,1.544833,Alabama
1,01003,Baldwin County,"Baldwin County, Alabama",al,212830,133.8703,103496,109334,43,0.4587,...,1,1,1003.0,76.545712,19.385601,76.171373,22.409030,4.068687,1.419597,Alabama
2,01005,Barbour County,"Barbour County, Alabama",al,25361,28.65624,13421,11940,40.4,0.4883,...,1,1,1005.0,52.096666,46.527844,53.451226,45.788173,1.375490,0.760601,Alabama
3,01007,Bibb County,"Bibb County, Alabama",al,22493,36.13558,12150,10343,40.9,0.4487,...,1,1,1007.0,76.403220,21.249575,78.426264,20.698280,2.347205,0.875456,Alabama
4,01009,Blount County,"Blount County, Alabama",al,57681,89.45139,28495,29186,40.7,0.457,...,1,1,1009.0,89.334844,8.425825,89.571553,9.569378,2.239331,0.859069,Alabama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3100,56037,Sweetwater County,"Sweetwater County, Wyoming",wy,43521,4.173883,22471,21050,35.3,0.4065,...,1,1,56037.0,70.951547,18.861646,73.236316,22.894957,10.186807,3.868727,Wyoming
3101,56039,Teton County,"Teton County, Wyoming",wy,23280,5.824592,12325,10955,39.3,0.5003,...,1,1,56039.0,31.052507,57.923497,29.356868,66.599040,11.023996,4.044093,Wyoming
3102,56041,Uinta County,"Uinta County, Wyoming",wy,20479,9.837535,10406,10073,35.8,0.3861,...,1,1,56041.0,72.656434,14.191263,79.247278,16.819960,13.152302,3.932762,Wyoming
3103,56043,Washakie County,"Washakie County, Wyoming",wy,8027,3.585605,4065,3962,42.9,0.3868,...,1,1,56043.0,76.324069,13.948610,80.481151,16.145833,9.727320,3.373016,Wyoming


In [ ]:
# merge coarse with states_pol
# since many counties correspond to one states,
# this is a many-to-one merge
# inner-merge with states_pol drops DC data, since it isn't a state
# we aim to analyze COVID in the US' contiguous states, so this outcome is desirable)

coarse = coarse.merge(states_pol, left_on="States", right_on="State",
             validate="many_to_one").drop("State", axis=1)
coarse

,FIPS,Area Name,Qualifying Name,State Postal Abbreviation,Total Population,Population Density (Per Sq. Mile),Total Population: Male,Total Population: Female,Median Age:,Gini Index,...,Clinton2016,Trump2020,Biden2020,Other2016,Other2020,States,Legisl_D,Legisl_R,Gov_D,Gov_R
0,01001,Autauga County,"Autauga County, Alabama",al,55380,93.16273,26934,28446,38.2,0.4542,...,23.769671,71.436802,27.018365,3.463741,1.544833,Alabama,0,1,0,1
1,01003,Baldwin County,"Baldwin County, Alabama",al,212830,133.8703,103496,109334,43,0.4587,...,19.385601,76.171373,22.409030,4.068687,1.419597,Alabama,0,1,0,1
2,01005,Barbour County,"Barbour County, Alabama",al,25361,28.65624,13421,11940,40.4,0.4883,...,46.527844,53.451226,45.788173,1.375490,0.760601,Alabama,0,1,0,1
3,01007,Bibb County,"Bibb County, Alabama",al,22493,36.13558,12150,10343,40.9,0.4487,...,21.249575,78.426264,20.698280,2.347205,0.875456,Alabama,0,1,0,1
4,01009,Blount County,"Blount County, Alabama",al,57681,89.45139,28495,29186,40.7,0.457,...,8.425825,89.571553,9.569378,2.239331,0.859069,Alabama,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3099,56037,Sweetwater County,"Sweetwater County, Wyoming",wy,43521,4.173883,22471,21050,35.3,0.4065,...,18.861646,73.236316,22.894957,10.186807,3.868727,Wyoming,0,1,0,1
3100,56039,Teton County,"Teton County, Wyoming",wy,23280,5.824592,12325,10955,39.3,0.5003,...,57.923497,29.356868,66.599040,11.023996,4.044093,Wyoming,0,1,0,1
3101,56041,Uinta County,"Uinta County, Wyoming",wy,20479,9.837535,10406,10073,35.8,0.3861,...,14.191263,79.247278,16.819960,13.152302,3.932762,Wyoming,0,1,0,1
3102,56043,Washakie County,"Washakie County, Wyoming",wy,8027,3.585605,4065,3962,42.9,0.3868,...,13.948610,80.481151,16.145833,9.727320,3.373016,Wyoming,0,1,0,1


In [ ]:
# extract relevant variables from health dataframe
# unhealthy days physically + mentally; % insured; food environment index
# data on uninsured rate in the population is outdated, but the best we've got

health = health[["Qualifying Name", "Physically Unhealthy Days per Month (Persons 18 Years and Over)", "Mentally Unhealthy Days per Month (Persons 18 Years and Over)",
                 "Percent of Persons Without Insurance (Population Under 65 Years, 2013 est.)", "Food Environment Index"]]
health = health.drop(0, axis=0)

# it turns out that original ACS FIPS data was inconsistently encoded,
# sometimes as a string, sometimes as an integer,
# so inner-merging by FIPS causes counties to be unpredictably dropped
# to avoid this, we merge on Qualifying Name (county, state),
# which is always a string
# we do NOT merge on county name itself, because county names are not unique
coarse = coarse.merge(health, on="Qualifying Name", validate="one_to_one")
coarse

,FIPS,Area Name,Qualifying Name,State Postal Abbreviation,Total Population,Population Density (Per Sq. Mile),Total Population: Male,Total Population: Female,Median Age:,Gini Index,...,Other2020,States,Legisl_D,Legisl_R,Gov_D,Gov_R,Physically Unhealthy Days per Month (Persons 18 Years and Over),Mentally Unhealthy Days per Month (Persons 18 Years and Over),"Percent of Persons Without Insurance (Population Under 65 Years, 2013 est.)",Food Environment Index
0,01001,Autauga County,"Autauga County, Alabama",al,55380,93.16273,26934,28446,38.2,0.4542,...,1.544833,Alabama,0,1,0,1,4.2005779826,4.3067392835,10.961103842,7.2
1,01003,Baldwin County,"Baldwin County, Alabama",al,212830,133.8703,103496,109334,43,0.4587,...,1.419597,Alabama,0,1,0,1,4.0987477691,4.2496487805,13.570310753,8
2,01005,Barbour County,"Barbour County, Alabama",al,25361,28.65624,13421,11940,40.4,0.4883,...,0.760601,Alabama,0,1,0,1,5.0674383044,4.634994147,16.542239686,5.6
3,01007,Bibb County,"Bibb County, Alabama",al,22493,36.13558,12150,10343,40.9,0.4487,...,0.875456,Alabama,0,1,0,1,4.3633772783,4.3157100192,12.297318992,7.6
4,01009,Blount County,"Blount County, Alabama",al,57681,89.45139,28495,29186,40.7,0.457,...,0.859069,Alabama,0,1,0,1,4.5127526482,4.7015992588,15.564604172,8.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3099,56037,Sweetwater County,"Sweetwater County, Wyoming",wy,43521,4.173883,22471,21050,35.3,0.4065,...,3.868727,Wyoming,0,1,0,1,3.536555838,3.5435462345,15.270086912,7.7
3100,56039,Teton County,"Teton County, Wyoming",wy,23280,5.824592,12325,10955,39.3,0.5003,...,4.044093,Wyoming,0,1,0,1,3.1663157057,3.0722529868,16.043395022,8.3
3101,56041,Uinta County,"Uinta County, Wyoming",wy,20479,9.837535,10406,10073,35.8,0.3861,...,3.932762,Wyoming,0,1,0,1,3.677537986,3.6999219286,15.073191133,7.3
3102,56043,Washakie County,"Washakie County, Wyoming",wy,8027,3.585605,4065,3962,42.9,0.3868,...,3.373016,Wyoming,0,1,0,1,3.6016870855,3.4796941981,19.59012781,8.2


In [ ]:
# from religion dataframe, we really think the population's religiosity is chiefly important,
# a supposition which the literature on epidemiology mostly supports
# inner-merging on religion would drop 3 counties;
# we think it's not worth losing those counties' data because of a missing value
# in one probably tangentially-important feature, so we left-merge,
# then fill values with national average of US 2010

# first, find nationwide religious adherence

relig_adhere = religion.iloc[1:][["Population (2010)", "All Religious Adherents"]]

relig_adhere["Population (2010)"] = pd.to_numeric(relig_adhere["Population (2010)"])
relig_adhere["All Religious Adherents"] = pd.to_numeric(relig_adhere["All Religious Adherents"])

relig_adhere = relig_adhere["All Religious Adherents"].sum() / relig_adhere["Population (2010)"].sum()
relig_adhere

# this adherence figure, at 49%, seems low
# but that's because the definition for adherence means "being in a congregation,"
# rather than being vaguely religious (https://www.usreligioncensus.org/faq)
# this is the better figure, since effects of religion on epidemiology would stem from
# increased community (both increased disease spread and increased resiliency / support)

0.48776993823308307

In [ ]:
# second, before we left-merge and fill nulls with religious adherence, [HERE]
# we must check dataframe for other nulls

# there are indeed 20 counties with some null values
nulls = coarse[coarse.isnull().any(axis=1)]

# counties are missing null values in food environment & average work commute columns
null_cols = nulls.isnull().any(axis=0)
null_cols[null_cols]

# counties without food environment values are likely underresourced,
# and likely don't have ready access to food
# so we can fill in food environment nulls with 0
coarse["Food Environment Index"] = coarse["Food Environment Index"].fillna(0)

# there is one county without an average commute time, with a FIPS of 48301
nulls[nulls["Average Commute to Work (In Min)"].isna()]
row_loc = coarse[coarse["FIPS_num"] == 48301].index

# fill in average commute time nulls with the average of neighboring counties' commute times
# get adjacent counties from adjacency matrix
adjacents_48301 = adjacency[adjacency["48301"] == 1]["48301"]
adjacents_48301 = adjacents_48301.reset_index()
adjacents_48301["index"] = pd.to_numeric(adjacents_48301["index"])

# get county commute times
commute = coarse[["FIPS_num", "Average Commute to Work (In Min)"]]

# get average of adjacent counties' commute times only
commute = commute[coarse["FIPS_num"].isin(adjacents_48301["index"])]
commute_time = commute["Average Commute to Work (In Min)"].mean()

# fill null in coarse
coarse["Average Commute to Work (In Min)"].iloc[row_loc] = commute_time

# no null cells left
# to find whether cells are null, sum across rows, then columns
coarse.isna().sum().sum()

<ipython-input-36-244c11689f74>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coarse["Average Commute to Work (In Min)"].iloc[row_loc] = commute_time


0

In [ ]:
# third, left-merge religion data and fill nulls
coarse = coarse.merge(religion[["Qualifying Name", "All Religious Adherence Rate of Total Population"]],
                      on="Qualifying Name", how="left", validate="one_to_one")
coarse["All Religious Adherence Rate of Total Population"] = coarse["All Religious Adherence Rate of Total Population"].fillna(relig_adhere)
coarse

,FIPS,Area Name,Qualifying Name,State Postal Abbreviation,Total Population,Population Density (Per Sq. Mile),Total Population: Male,Total Population: Female,Median Age:,Gini Index,...,States,Legisl_D,Legisl_R,Gov_D,Gov_R,Physically Unhealthy Days per Month (Persons 18 Years and Over),Mentally Unhealthy Days per Month (Persons 18 Years and Over),"Percent of Persons Without Insurance (Population Under 65 Years, 2013 est.)",Food Environment Index,All Religious Adherence Rate of Total Population
0,01001,Autauga County,"Autauga County, Alabama",al,55380,93.16273,26934,28446,38.2,0.4542,...,Alabama,0,1,0,1,4.2005779826,4.3067392835,10.961103842,7.2,67.687888889
1,01003,Baldwin County,"Baldwin County, Alabama",al,212830,133.8703,103496,109334,43,0.4587,...,Alabama,0,1,0,1,4.0987477691,4.2496487805,13.570310753,8,53.174
2,01005,Barbour County,"Barbour County, Alabama",al,25361,28.65624,13421,11940,40.4,0.4883,...,Alabama,0,1,0,1,5.0674383044,4.634994147,16.542239686,5.6,54.999
3,01007,Bibb County,"Bibb County, Alabama",al,22493,36.13558,12150,10343,40.9,0.4487,...,Alabama,0,1,0,1,4.3633772783,4.3157100192,12.297318992,7.6,49.88
4,01009,Blount County,"Blount County, Alabama",al,57681,89.45139,28495,29186,40.7,0.457,...,Alabama,0,1,0,1,4.5127526482,4.7015992588,15.564604172,8.5,65.162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3099,56037,Sweetwater County,"Sweetwater County, Wyoming",wy,43521,4.173883,22471,21050,35.3,0.4065,...,Wyoming,0,1,0,1,3.536555838,3.5435462345,15.270086912,7.7,47.722
3100,56039,Teton County,"Teton County, Wyoming",wy,23280,5.824592,12325,10955,39.3,0.5003,...,Wyoming,0,1,0,1,3.1663157057,3.0722529868,16.043395022,8.3,26.036
3101,56041,Uinta County,"Uinta County, Wyoming",wy,20479,9.837535,10406,10073,35.8,0.3861,...,Wyoming,0,1,0,1,3.677537986,3.6999219286,15.073191133,7.3,60.683
3102,56043,Washakie County,"Washakie County, Wyoming",wy,8027,3.585605,4065,3962,42.9,0.3868,...,Wyoming,0,1,0,1,3.6016870855,3.4796941981,19.59012781,8.2,47.181889


In [ ]:
# check that coarse contains no nulls
# to access all cells, sum over rows then columns
coarse.isnull().sum().sum()

0

In [ ]:
# clean and merge sleep data
# sleep dataframe has numeric county FIPS labels
cleanSleep = sleep.rename({"Age-Adjusted Prevalence (%)": "Sleep Deprivation"}, axis=1)
cleanSleep = cleanSleep[["CountyFIPS", "Sleep Deprivation"]]
coarse = coarse.merge(cleanSleep, left_on="FIPS_num", right_on="CountyFIPS",
                      validate="one_to_one").drop("CountyFIPS", axis=1)
coarse

,FIPS,Area Name,Qualifying Name,State Postal Abbreviation,Total Population,Population Density (Per Sq. Mile),Total Population: Male,Total Population: Female,Median Age:,Gini Index,...,Legisl_D,Legisl_R,Gov_D,Gov_R,Physically Unhealthy Days per Month (Persons 18 Years and Over),Mentally Unhealthy Days per Month (Persons 18 Years and Over),"Percent of Persons Without Insurance (Population Under 65 Years, 2013 est.)",Food Environment Index,All Religious Adherence Rate of Total Population,Sleep Deprivation
0,01001,Autauga County,"Autauga County, Alabama",al,55380,93.16273,26934,28446,38.2,0.4542,...,0,1,0,1,4.2005779826,4.3067392835,10.961103842,7.2,67.687888889,38.4
1,01003,Baldwin County,"Baldwin County, Alabama",al,212830,133.8703,103496,109334,43,0.4587,...,0,1,0,1,4.0987477691,4.2496487805,13.570310753,8,53.174,36.4
2,01005,Barbour County,"Barbour County, Alabama",al,25361,28.65624,13421,11940,40.4,0.4883,...,0,1,0,1,5.0674383044,4.634994147,16.542239686,5.6,54.999,40.9
3,01007,Bibb County,"Bibb County, Alabama",al,22493,36.13558,12150,10343,40.9,0.4487,...,0,1,0,1,4.3633772783,4.3157100192,12.297318992,7.6,49.88,40.1
4,01009,Blount County,"Blount County, Alabama",al,57681,89.45139,28495,29186,40.7,0.457,...,0,1,0,1,4.5127526482,4.7015992588,15.564604172,8.5,65.162,38.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3099,56037,Sweetwater County,"Sweetwater County, Wyoming",wy,43521,4.173883,22471,21050,35.3,0.4065,...,0,1,0,1,3.536555838,3.5435462345,15.270086912,7.7,47.722,35.4
3100,56039,Teton County,"Teton County, Wyoming",wy,23280,5.824592,12325,10955,39.3,0.5003,...,0,1,0,1,3.1663157057,3.0722529868,16.043395022,8.3,26.036,27.1
3101,56041,Uinta County,"Uinta County, Wyoming",wy,20479,9.837535,10406,10073,35.8,0.3861,...,0,1,0,1,3.677537986,3.6999219286,15.073191133,7.3,60.683,35.9
3102,56043,Washakie County,"Washakie County, Wyoming",wy,8027,3.585605,4065,3962,42.9,0.3868,...,0,1,0,1,3.6016870855,3.4796941981,19.59012781,8.2,47.181889,32.4


In [ ]:
COVID_insights["Employment to Population Ratio (percent)"]
# I want to get all of these

0            TT_T012_016
1       54.4712889088662
2       48.8500165544642
3       45.0516840595479
4       44.5368558382257
              ...       
3216            5.201545
3217           82.595072
3218           39.172047
3219           28.953319
3220           39.512812
Name: Employment to Population Ratio (percent), Length: 3221, dtype: object

In [ ]:
to_keep = []

for column in COVID_insights.columns:
  if column == "Employment to Population Ratio (percent)":
    to_keep.append(column)
to_keep

['Employment to Population Ratio (percent)']

In [ ]:
# extract labor force participation rate by looping through COVID_insights columns
fields = list(COVID_insights)[:3]
for i in range(20, len(list(COVID_insights)), 26):
  fields.append(list(COVID_insights)[i])
employed = pd.DataFrame(COVID_insights, columns=fields).iloc[1:]

# convert all labor force participation rate columns to numerical data
employed = employed.iloc[:, 3:].astype(float)
# aggregate for the entire year
employed = employed.mean(axis=1)
# create a final dataframe with location & labor force data
employed_final = pd.DataFrame(COVID_insights.iloc[1:, 2])
employed_final["labor_force_participation"] = employed
employed_final

,Qualifying Name,labor_force_participation
1,"Autauga County, Alabama",53.997691
2,"Baldwin County, Alabama",49.839271
3,"Barbour County, Alabama",41.776637
4,"Bibb County, Alabama",43.724920
5,"Blount County, Alabama",51.132421
...,...,...
3216,"Vega Baja Municipio, Puerto Rico",3.827320
3217,"Vieques Municipio, Puerto Rico",62.023022
3218,"Villalba Municipio, Puerto Rico",29.514307
3219,"Yabucoa Municipio, Puerto Rico",21.458789


In [ ]:
# merge labor force participation data into course
coarse = coarse.merge(employed_final, validate="one_to_one", on="Qualifying Name")

In [ ]:
# download cleaned coarse data!
# download from Colab into computer, then upload to Google Drive
coarse.to_csv("coarse.csv")

**Step 3: Cleaning & Aggregating Fine-Grained Data**

In [ ]:
# merge precip and temp on the basis of Location ID
comb_temp_precip = pd.merge(temp, precip, on='Location ID', validate="one_to_one", suffixes=('_temp', '_precip'))

# use regex to keep only values (filter out anomaly, rank, and mean data)
comb_temp_precip = comb_temp_precip[comb_temp_precip.columns.drop(list(comb_temp_precip.filter(regex='(Anomaly|Rank|Mean_)')))]
comb_temp_precip = comb_temp_precip.drop("Location_precip", axis=1)
comb_temp_precip

,Location ID,Location_temp,Value_202001_temp,Value_202002_temp,Value_202003_temp,Value_202004_temp,Value_202005_temp,Value_202006_temp,Value_202007_temp,Value_202008_temp,...,Value_202003_precip,Value_202004_precip,Value_202005_precip,Value_202006_precip,Value_202007_precip,Value_202008_precip,Value_20209_precip,Value_202010_precip,Value_202011_precip,Value_202012_precip
0,AL-001,Autauga County,50.7,52.5,65.2,63.4,70.2,78.2,82.2,81.6,...,6.55,8.32,3.06,4.94,5.36,4.53,5.12,5.67,3.66,3.09
1,AL-003,Baldwin County,54.8,56.6,68.4,67.3,72.6,79.4,81.8,81.8,...,0.89,3.64,2.57,7.28,9.73,6.95,11.71,4.29,2.23,3.85
2,AL-005,Barbour County,51.6,52.5,65.8,64.2,70.3,77.4,80.9,80.8,...,5.09,6.45,4.53,4.46,3.96,5.94,13.99,2.81,3.11,2.34
3,AL-007,Bibb County,48.6,49.7,62.4,60.8,68.2,76.1,80.5,79.8,...,7.91,8.49,2.53,3.56,4.71,4.48,2.46,6.59,3.17,3.47
4,AL-009,Blount County,47.1,48.1,60.4,58.8,66.8,75.5,79.6,78.8,...,9.56,8.75,3.81,3.88,3.89,5.08,3.32,6.64,2.15,5.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3132,AK-230,Skagway Municipality,9.5,18.6,18.3,27.8,41.6,44.9,49.4,47.0,...,3.04,3.28,1.66,3.71,3.86,6.99,4.43,7.16,7.98,30.89
3133,AK-240,Southeast Fairbanks Census Area,-16.6,-2.9,6.5,26.0,44.6,50.3,52.4,50.5,...,2.06,1.17,0.70,6.04,3.05,4.73,2.05,2.04,0.68,0.40
3134,AK-275,Wrangell City and Borough,23.6,30.3,30.0,36.6,47.2,49.7,53.2,52.7,...,8.10,9.94,4.64,10.21,11.08,20.54,9.02,12.74,21.29,31.63
3135,AK-282,Yakutat City and Borough,11.8,22.8,22.8,30.8,41.9,44.6,49.4,48.3,...,7.25,10.54,8.45,10.15,12.29,23.22,16.93,20.97,15.40,27.63


In [ ]:
# from COVID, extract weekly change in cumulative confirmed cases and deaths
# many government regions only reliably report cases on a weekly basis,
# batching their reports, producing COVID spikes on Sundays/Mondays
# to smooth this out we look at weekly data
# this data tells us how many people contracted & died of COVID each week

# extract relevant columns
# keep geographic identifiers
fields = list(COVID)[:3]
# loop through columns, adding regularly-interspersed
# changes in cumulative cases
for i in range(15, len(list(COVID)), 140):
  fields.append(list(COVID)[i])
# filter dataframe
weekly_change_cases = pd.DataFrame(COVID, columns=fields)
weekly_change_cases

,FIPS,Name of Area,Qualifying Name,Change in Cumulative Confirmed Cases as of Dec 31 2020 from Previous Week,Change in Cumulative Confirmed Cases as of Dec 24 2020 from Previous Week,Change in Cumulative Confirmed Cases as of Dec 17 2020 from Previous Week,Change in Cumulative Confirmed Cases as of Dec 10 2020 from Previous Week,Change in Cumulative Confirmed Cases as of Dec 3 2020 from Previous Week,Change in Cumulative Confirmed Cases as of Nov 26 2020 from Previous Week,Change in Cumulative Confirmed Cases as of Nov 19 2020 from Previous Week,...,Change in Cumulative Confirmed Cases as of Mar 26 2020 from Previous Week,Change in Cumulative Confirmed Cases as of Mar 19 2020 from Previous Week,Change in Cumulative Confirmed Cases as of Mar 12 2020 from Previous Week,Change in Cumulative Confirmed Cases as of Mar 5 2020 from Previous Week,Change in Cumulative Confirmed Cases as of Feb 27 2020 from Previous Week,Change in Cumulative Confirmed Cases as of Feb 20 2020 from Previous Week,Change in Cumulative Confirmed Cases as of Feb 13 2020 from Previous Week,Change in Cumulative Confirmed Cases as of Feb 6 2020 from Previous Week,Change in Cumulative Confirmed Cases as of Jan 30 2020 from Previous Week,Change in Cumulative Confirmed Cases as of Jan 23 2020 from Previous Week
0,Geo_FIPS,Geo_NAME,Geo_QNAME,ORG_I123120,ORG_I122420,ORG_I121720,ORG_I121020,ORG_I120320,ORG_I112620,ORG_I111920,...,ORG_I032620,ORG_I031920,ORG_I031220,ORG_I030520,ORG_I022720,ORG_I022020,ORG_I021320,ORG_I020620,ORG_I013020,ORG_I012320
1,01001,Autauga County,"Autauga County, Alabama",248,372,384,293,189,150,169,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01003,Baldwin County,"Baldwin County, Alabama",1080,1157,1076,947,765,643,479,...,4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01005,Barbour County,"Barbour County, Alabama",124,81,51,44,44,25,32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01007,Bibb County,"Bibb County, Alabama",123,163,189,107,82,159,45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3276,72999,Unknown County,"Unknown County, Puerto Rico",219.0,327.0,185.0,207.0,219.0,199.0,99.0,...,58.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3277,78010,St. Croix County,"St. Croix County, Virgin Islands",30.0,46.0,86.0,62.0,20.0,17.0,28.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3278,78020,St. John County,"St. John County, Virgin Islands",8.0,15.0,21.0,36.0,15.0,4.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3279,78030,St. Thomas County,"St. Thomas County, Virgin Islands",26.0,28.0,38.0,46.0,33.0,31.0,22.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# extract relevant columns
# keep geographic identifiers
fields = list(COVID)[:3]
# loop through columns, adding regularly-interspersed
# changes in cumulative deaths
for i in range(15, len(list(COVID)), 140):
  fields.append(list(COVID)[i + 4])
# filter dataframe
weekly_change_deaths = pd.DataFrame(COVID, columns=fields)
weekly_change_deaths

,FIPS,Name of Area,Qualifying Name,Change in Cumulative Deaths as of Dec 31 2020 from Previous Week,Change in Cumulative Deaths as of Dec 24 2020 from Previous Week,Change in Cumulative Deaths as of Dec 17 2020 from Previous Week,Change in Cumulative Deaths as of Dec 10 2020 from Previous Week,Change in Cumulative Deaths as of Dec 3 2020 from Previous Week,Change in Cumulative Deaths as of Nov 26 2020 from Previous Week,Change in Cumulative Deaths as of Nov 19 2020 from Previous Week,...,Change in Cumulative Deaths as of Mar 26 2020 from Previous Week,Change in Cumulative Deaths as of Mar 19 2020 from Previous Week,Change in Cumulative Deaths as of Mar 12 2020 from Previous Week,Change in Cumulative Deaths as of Mar 5 2020 from Previous Week,Change in Cumulative Deaths as of Feb 27 2020 from Previous Week,Change in Cumulative Deaths as of Feb 20 2020 from Previous Week,Change in Cumulative Deaths as of Feb 13 2020 from Previous Week,Change in Cumulative Deaths as of Feb 6 2020 from Previous Week,Change in Cumulative Deaths as of Jan 30 2020 from Previous Week,Change in Cumulative Deaths as of Jan 23 2020 from Previous Week
0,Geo_FIPS,Geo_NAME,Geo_QNAME,ORG_M123120,ORG_M122420,ORG_M121720,ORG_M121020,ORG_M120320,ORG_M112620,ORG_M111920,...,ORG_M032620,ORG_M031920,ORG_M031220,ORG_M030520,ORG_M022720,ORG_M022020,ORG_M021320,ORG_M020620,ORG_M013020,ORG_M012320
1,01001,Autauga County,"Autauga County, Alabama",2,3,1,0,0,3,7,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01003,Baldwin County,"Baldwin County, Alabama",10,6,4,4,39,14,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01005,Barbour County,"Barbour County, Alabama",0,2,0,1,19,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01007,Bibb County,"Bibb County, Alabama",4,0,2,2,21,-1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3276,72999,Unknown County,"Unknown County, Puerto Rico",80.0,100.0,85.0,83.0,86.0,87.0,73.0,...,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3277,78010,St. Croix County,"St. Croix County, Virgin Islands",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3278,78020,St. John County,"St. John County, Virgin Islands",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3279,78030,St. Thomas County,"St. Thomas County, Virgin Islands",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# extract cumulative cases through same methodology as above
fields = list(COVID)[:3]
for i in range(7, len(list(COVID)), 140):
  fields.append(list(COVID)[i])

weekly_total_cases = pd.DataFrame(COVID, columns=fields)
weekly_total_cases

,FIPS,Name of Area,Qualifying Name,Cumulative Number of Confirmed Cases as of Dec 31 2020,Cumulative Number of Confirmed Cases as of Dec 24 2020,Cumulative Number of Confirmed Cases as of Dec 17 2020,Cumulative Number of Confirmed Cases as of Dec 10 2020,Cumulative Number of Confirmed Cases as of Dec 3 2020,Cumulative Number of Confirmed Cases as of Nov 26 2020,Cumulative Number of Confirmed Cases as of Nov 19 2020,...,Cumulative Number of Confirmed Cases as of Mar 26 2020,Cumulative Number of Confirmed Cases as of Mar 19 2020,Cumulative Number of Confirmed Cases as of Mar 12 2020,Cumulative Number of Confirmed Cases as of Mar 5 2020,Cumulative Number of Confirmed Cases as of Feb 27 2020,Cumulative Number of Confirmed Cases as of Feb 20 2020,Cumulative Number of Confirmed Cases as of Feb 13 2020,Cumulative Number of Confirmed Cases as of Feb 6 2020,Cumulative Number of Confirmed Cases as of Jan 30 2020,Cumulative Number of Confirmed Cases as of Jan 23 2020
0,Geo_FIPS,Geo_NAME,Geo_QNAME,ORG_A123120,ORG_A122420,ORG_A121720,ORG_A121020,ORG_A120320,ORG_A112620,ORG_A111920,...,ORG_A032620,ORG_A031920,ORG_A031220,ORG_A030520,ORG_A022720,ORG_A022020,ORG_A021320,ORG_A020620,ORG_A013020,ORG_A012320
1,01001,Autauga County,"Autauga County, Alabama",4190,3942,3570,3186,2893,2704,2554,...,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01003,Baldwin County,"Baldwin County, Alabama",13601,12521,11364,10288,9341,8576,7933,...,5,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01005,Barbour County,"Barbour County, Alabama",1514,1390,1309,1258,1214,1170,1145,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01007,Bibb County,"Bibb County, Alabama",1834,1711,1548,1359,1252,1170,1011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3276,72999,Unknown County,"Unknown County, Puerto Rico",3875.0,3656.0,3329.0,3144.0,2937.0,2718.0,2519.0,...,64.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3277,78010,St. Croix County,"St. Croix County, Virgin Islands",829.0,799.0,753.0,667.0,605.0,585.0,568.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3278,78020,St. John County,"St. John County, Virgin Islands",174.0,166.0,151.0,130.0,94.0,79.0,75.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3279,78030,St. Thomas County,"St. Thomas County, Virgin Islands",1028.0,1002.0,974.0,936.0,890.0,857.0,826.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# extract cumulative deaths through same methodology as above
fields = list(COVID)[:3]
for i in range(7, len(list(COVID)), 140):
  fields.append(list(COVID)[i + 1])

weekly_total_deaths = pd.DataFrame(COVID, columns=fields)
weekly_total_deaths

,FIPS,Name of Area,Qualifying Name,Cumulative Number of Deaths as of Dec 31 2020,Cumulative Number of Deaths as of Dec 24 2020,Cumulative Number of Deaths as of Dec 17 2020,Cumulative Number of Deaths as of Dec 10 2020,Cumulative Number of Deaths as of Dec 3 2020,Cumulative Number of Deaths as of Nov 26 2020,Cumulative Number of Deaths as of Nov 19 2020,...,Cumulative Number of Deaths as of Mar 26 2020,Cumulative Number of Deaths as of Mar 19 2020,Cumulative Number of Deaths as of Mar 12 2020,Cumulative Number of Deaths as of Mar 5 2020,Cumulative Number of Deaths as of Feb 27 2020,Cumulative Number of Deaths as of Feb 20 2020,Cumulative Number of Deaths as of Feb 13 2020,Cumulative Number of Deaths as of Feb 6 2020,Cumulative Number of Deaths as of Jan 30 2020,Cumulative Number of Deaths as of Jan 23 2020
0,Geo_FIPS,Geo_NAME,Geo_QNAME,ORG_B123120,ORG_B122420,ORG_B121720,ORG_B121020,ORG_B120320,ORG_B112620,ORG_B111920,...,ORG_B032620,ORG_B031920,ORG_B031220,ORG_B030520,ORG_B022720,ORG_B022020,ORG_B021320,ORG_B020620,ORG_B013020,ORG_B012320
1,01001,Autauga County,"Autauga County, Alabama",48,46,43,42,42,42,39,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01003,Baldwin County,"Baldwin County, Alabama",161,151,145,141,137,98,84,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01005,Barbour County,"Barbour County, Alabama",32,32,30,30,29,10,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01007,Bibb County,"Bibb County, Alabama",46,42,42,40,38,17,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3276,72999,Unknown County,"Unknown County, Puerto Rico",1503.0,1423.0,1323.0,1238.0,1155.0,1069.0,982.0,...,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3277,78010,St. Croix County,"St. Croix County, Virgin Islands",7.0,7.0,7.0,7.0,7.0,7.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3278,78020,St. John County,"St. John County, Virgin Islands",1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3279,78030,St. Thomas County,"St. Thomas County, Virgin Islands",15.0,15.0,15.0,15.0,15.0,15.0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# get overall 2020 cases and deaths for COVID
yearly_cases = COVID[["FIPS", "Name of Area", "Qualifying Name", "Cumulative Number of Confirmed Cases as of Dec 31 2020"]]
yearly_deaths = COVID[["FIPS", "Name of Area", "Qualifying Name", "Cumulative Number of Deaths as of Dec 31 2020"]]
yearly_cases

,FIPS,Name of Area,Qualifying Name,Cumulative Number of Confirmed Cases as of Dec 31 2020
0,Geo_FIPS,Geo_NAME,Geo_QNAME,ORG_A123120
1,01001,Autauga County,"Autauga County, Alabama",4190
2,01003,Baldwin County,"Baldwin County, Alabama",13601
3,01005,Barbour County,"Barbour County, Alabama",1514
4,01007,Bibb County,"Bibb County, Alabama",1834
...,...,...,...,...
3276,72999,Unknown County,"Unknown County, Puerto Rico",3875.0
3277,78010,St. Croix County,"St. Croix County, Virgin Islands",829.0
3278,78020,St. John County,"St. John County, Virgin Islands",174.0
3279,78030,St. Thomas County,"St. Thomas County, Virgin Islands",1028.0


In [ ]:
# some nulls recorded as -999999; filter these out
hospitals = hospitals.replace(-999999, None)
hospitals

,hospital_pk,collection_week,state,ccn,hospital_name,address,city,zip,hospital_subtype,fips_code,...,previous_day_admission_pediatric_covid_confirmed_unknown_7_day_sum,staffed_icu_pediatric_patients_confirmed_covid_7_day_avg,staffed_icu_pediatric_patients_confirmed_covid_7_day_coverage,staffed_icu_pediatric_patients_confirmed_covid_7_day_sum,staffed_pediatric_icu_bed_occupancy_7_day_avg,staffed_pediatric_icu_bed_occupancy_7_day_coverage,staffed_pediatric_icu_bed_occupancy_7_day_sum,total_staffed_pediatric_icu_beds_7_day_avg,total_staffed_pediatric_icu_beds_7_day_coverage,total_staffed_pediatric_icu_beds_7_day_sum
0,100217,2020/07/10,FL,100217,SEBASTIAN RIVER MEDICAL CENTER,13695 US HWY 1,SEBASTIAN,32978.0,Short Term,12061.0,...,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN
1,100232,2020/07/03,FL,100232,HCA FLORIDA PUTNAM HOSPITAL,611 ZEAGLER DR,PALATKA,32177.0,Short Term,12107.0,...,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN
2,102029,2020/05/01,FL,102029,SELECT SPECIALTY HOSPITAL-FORT MYERS,3050 CHAMPION RING ROAD,FORT MYERS,33905.0,Long Term,12071.0,...,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN
3,102030,2020/06/05,FL,102030,SELECT SPECIALTY HOSPITAL DAYTONA BEACH,301 MEMORIAL MEDICAL PARKWAY,DAYTONA BEACH,32117.0,Long Term,12127.0,...,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN
4,110107,2020/07/31,GA,110107,ATRIUM HEALTH NAVICENT THE MEDICAL CENTER,777 HEMLOCK STREET,MACON,31201.0,Short Term,13021.0,...,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
761856,400012,2022/06/24,PR,400012,HOSPITAL ONCOLOGICO DR ISAAC GONZALEZ MARTINEZ,BO. MONACILLOS CARR 22 CENTRO MEDICO DE PUERTO...,SAN JUAN,935.0,Short Term,72127.0,...,0.0,0.0,5,0.0,0.0,5,0.0,0.0,5,0.0
761857,451386,2020/12/11,TX,451386,MEMORIAL HOSPITAL,224 E SECOND STREET,DUMAS,79029.0,Critical Access Hospitals,48341.0,...,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN
761858,500124,2022/11/18,WA,500124,EVERGREENHEALTH MEDICAL CENTER,12040 NE 128TH STREET,KIRKLAND,98034.0,Short Term,53033.0,...,0.0,0.0,7,0.0,0.0,7,0.0,0.0,7,0.0
761859,670128,2021/10/15,TX,670128,BAYLOR SCOTT & WHITE MEDICAL CENTER – PFLUGERV...,2600 EAST PFLUGERVILLE PARKWAY,PFLUGERVILLE,78660.0,Short Term,48453.0,...,NaN,NaN,0,NaN,0.0,7,0.0,0.0,7,0.0


In [ ]:
# create handy functions to extract year, month, and day from date string
# will help us aggregate and combine data according to date
def split_date(date):
  slash1 = date.find("/")
  slash2 = date.find("/", slash1 + 1)
  year = int(date[0:slash1])
  month = int(date[slash1 + 1:slash2])
  day = int(date[slash2 + 1:])
  return [year, month, day]

def year_slash(date):
  return split_date(date)[0]

def month_slash(date):
  return split_date(date)[1]

def day_slash(date):
  return split_date(date)[2]

def split_date_dash(date):
  slash1 = date.find("-")
  slash2 = date.find("-", slash1 + 1)
  year = int(date[0:slash1])
  month = int(date[slash1 + 1:slash2])
  day = int(date[slash2 + 1:])
  return [year, month, day]

def year_dash(date):
  return split_date_dash(date)[0]

def month_dash(date):
  return split_date_dash(date)[1]

def day_dash(date):
  return split_date_dash(date)[2]

In [ ]:
# extract 2020-only data from hospitals
hospitals = hospitals[hospitals["collection_week"].apply(year_slash) == 2020]
hospitals

,hospital_pk,collection_week,state,ccn,hospital_name,address,city,zip,hospital_subtype,fips_code,...,previous_day_admission_pediatric_covid_confirmed_unknown_7_day_sum,staffed_icu_pediatric_patients_confirmed_covid_7_day_avg,staffed_icu_pediatric_patients_confirmed_covid_7_day_coverage,staffed_icu_pediatric_patients_confirmed_covid_7_day_sum,staffed_pediatric_icu_bed_occupancy_7_day_avg,staffed_pediatric_icu_bed_occupancy_7_day_coverage,staffed_pediatric_icu_bed_occupancy_7_day_sum,total_staffed_pediatric_icu_beds_7_day_avg,total_staffed_pediatric_icu_beds_7_day_coverage,total_staffed_pediatric_icu_beds_7_day_sum
0,100217,2020/07/10,FL,100217,SEBASTIAN RIVER MEDICAL CENTER,13695 US HWY 1,SEBASTIAN,32978.0,Short Term,12061.0,...,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN
1,100232,2020/07/03,FL,100232,HCA FLORIDA PUTNAM HOSPITAL,611 ZEAGLER DR,PALATKA,32177.0,Short Term,12107.0,...,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN
2,102029,2020/05/01,FL,102029,SELECT SPECIALTY HOSPITAL-FORT MYERS,3050 CHAMPION RING ROAD,FORT MYERS,33905.0,Long Term,12071.0,...,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN
3,102030,2020/06/05,FL,102030,SELECT SPECIALTY HOSPITAL DAYTONA BEACH,301 MEMORIAL MEDICAL PARKWAY,DAYTONA BEACH,32117.0,Long Term,12127.0,...,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN
4,110107,2020/07/31,GA,110107,ATRIUM HEALTH NAVICENT THE MEDICAL CENTER,777 HEMLOCK STREET,MACON,31201.0,Short Term,13021.0,...,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
761833,420051,2020/12/18,SC,420051,MCLEOD REGIONAL MEDICAL CENTER-PEE DEE,555 E CHEVES ST BOX 8700,FLORENCE,29506.0,Short Term,45041.0,...,NaN,NaN,0,NaN,25.1,7,176.0,31.0,7,217.0
761841,450236,2020/09/25,TX,450236,CHRISTUS MOTHER FRANCES HOSPITAL SULPHUR SPRINGS,115 AIRPORT RD,SULPHUR SPRINGS,75482.0,Short Term,48223.0,...,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN
761846,420066,2020/08/14,SC,420066,LAKE CITY COMMUNITY HOSPITAL,258 N RON MCNAIR BLVD,LAKE CITY,29560.0,Short Term,45041.0,...,NaN,0.0,7,0.0,0.0,7,0.0,0.0,7,0.0
761851,452068,2020/09/04,TX,452068,KPC PROMISE HOSPITAL OF WICHITA FALLS LLC,1103 GRACE STREET,WICHITA FALLS,76301.0,Long Term,48485.0,...,NaN,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN


In [ ]:
# extracting hospital columns into dataframe to manually search column names
# for information which we want to extract
df = pd.DataFrame()
df["hospital_columns"] = hospitals.columns
df

,hospital_columns
0,hospital_pk
1,collection_week
2,state
3,ccn
4,hospital_name
...,...
123,staffed_pediatric_icu_bed_occupancy_7_day_cove...
124,staffed_pediatric_icu_bed_occupancy_7_day_sum
125,total_staffed_pediatric_icu_beds_7_day_avg
126,total_staffed_pediatric_icu_beds_7_day_coverage


In [ ]:
# extract relevant columns
hospitals_aggreg = hospitals[["collection_week", "fips_code", "total_beds_7_day_sum",
           "inpatient_beds_used_7_day_sum", "total_icu_beds_7_day_sum", "icu_beds_used_7_day_sum"]]

# drop nulls
hospitals_aggreg = hospitals_aggreg.dropna()

# aggregate - sum by FIPS (county) and week
hospitals_aggreg = hospitals_aggreg.groupby(["fips_code", "collection_week"]).sum()
hospitals_aggreg

total_beds_7_day_sum  \
fips_code collection_week                         
1001.0    2020/07/17                      617.0   
          2020/07/24                      594.0   
          2020/07/31                      574.0   
          2020/08/07                      574.0   
          2020/08/14                      574.0   
...                                         ...   
78020.0   2020/11/27                      924.0   
          2020/12/04                      924.0   
          2020/12/11                      924.0   
          2020/12/18                      924.0   
          2020/12/25                      924.0   

                           inpatient_beds_used_7_day_sum  \
fips_code collection_week                                  
1001.0    2020/07/17                               380.0   
          2020/07/24                               351.0   
          2020/07/31                               363.0   
          2020/08/07                               369.0   
          2020/08/14                               384.0   
...                                                  ...   
78020.0   2020/11/27                               351.0   
          2020/12/04                               359.0   
          2020/12/11                               383.0   
          2020/12/18                               371.0   
          2020/12/25                               354.0   

                           total_icu_beds_7_day_sum  icu_beds_used_7_day_sum  
fips_code collection_week                                                     
1001.0    2020/07/17                           42.0                     12.0  
          2020/07/24                           42.0                     41.0  
          2020/07/31                           42.0                     40.0  
          2020/08/07                           42.0                     41.0  
          2020/08/14                           42.0                     36.0  
...                                             ...                      ...  
78020.0   2020/11/27                           56.0                     44.0  
          2020/12/04                           56.0                     48.0  
          2020/12/11                           56.0                     42.0  
          2020/12/18                           56.0                     41.0  
          2020/12/25                           56.0                     22.0  

[52088 rows x 4 columns]

In [ ]:
# we will construct two measures for hospital capacity
# beds used (sum of inpatient & ICU beds) / all beds = how strained is bed capacity?
# ICU beds used / ICU beds total = how strained is the ICU department?

hospitals_aggreg["beds"] = (hospitals_aggreg["inpatient_beds_used_7_day_sum"] + hospitals_aggreg["icu_beds_used_7_day_sum"])
hospitals_aggreg["beds"] /= hospitals_aggreg["total_beds_7_day_sum"]
hospitals_aggreg["beds"]

hospitals_aggreg["ICU"] = hospitals_aggreg["icu_beds_used_7_day_sum"] / hospitals_aggreg["total_icu_beds_7_day_sum"]
hospitals_aggreg["ICU"]

fips_code  collection_week
1001.0     2020/07/17         0.285714
           2020/07/24         0.976190
           2020/07/31         0.952381
           2020/08/07         0.976190
           2020/08/14         0.857143
                                ...   
78020.0    2020/11/27         0.785714
           2020/12/04         0.857143
           2020/12/11         0.750000
           2020/12/18         0.732143
           2020/12/25         0.392857
Name: ICU, Length: 52088, dtype: float64

In [ ]:
# extract constructed measures
hospitals_capacity = hospitals_aggreg[["beds", "ICU"]].reset_index()
hospitals_capacity

,fips_code,collection_week,beds,ICU
0,1001.0,2020/07/17,0.635332,0.285714
1,1001.0,2020/07/24,0.659933,0.976190
2,1001.0,2020/07/31,0.702091,0.952381
3,1001.0,2020/08/07,0.714286,0.976190
4,1001.0,2020/08/14,0.731707,0.857143
...,...,...,...,...
52083,78020.0,2020/11/27,0.427489,0.785714
52084,78020.0,2020/12/04,0.440476,0.857143
52085,78020.0,2020/12/11,0.459957,0.750000
52086,78020.0,2020/12/18,0.445887,0.732143


In [ ]:
# create a function that assembles a county's FIPS from its state and county code
# useful for aggregating dataframes (like daily_AQI) that lack explicit FIPS codes
def assemble_FIPS(row):
  # take a row of dataframe as input, extract state and county codes from them
  state_code = row["state_code"]
  county_code = row["county_code"]
  # since we'll convert FIPS to numeric, we don't care about leading zeroes for states
  state = str(state_code)
  county = str(county_code)
  # for FIPS, county codes must be 3 characters long
  # add leading zero(es) as needed to county code
  county = '0' * (3 - len(county)) + county
  # return county data as numeric for clear compatibility
  return int(state + county)

In [ ]:
daily_AQI["state_code"] = daily_AQI["State Code"]
daily_AQI["county_code"] = daily_AQI["County Code"]

# apply assemble_FIPS function
daily_AQI["FIPS"] = daily_AQI.apply(assemble_FIPS, axis=1)

In [ ]:
daily_AQI_clean = daily_AQI[["FIPS", "Date", "AQI"]]
daily_AQI_clean

,FIPS,Date,AQI
0,1003,2020-01-01,48
1,1003,2020-01-04,13
2,1003,2020-01-07,14
3,1003,2020-01-10,39
4,1003,2020-01-13,29
...,...,...,...
324333,56045,2020-12-27,32
324334,56045,2020-12-28,30
324335,56045,2020-12-29,33
324336,56045,2020-12-30,33


In [ ]:
# given a date, convert to a numbered week

# ok, so the weeks start on Jan 23 2020
# there should be a way to extract from the columns or some shit
# then, if it falls in between (or 7 days before - special case),
# round down with the numeration
# don't want to do by integer division b/c months are irregularly-spanned

# first, we must compile the bins of weeks
# we do this using the weeks for COVID case and deaths data,
# as this is ultimately what we want to predict,
# and our input data must be compatible with the output

# from COVID weeks data get the weeks in numeric range
weeks = list(weekly_total_cases.columns)[3:]
dates = []

for week in weeks:
  week_start = week.find("as of ")
  week_end = week.find(" 2020")
  week = week[week_start + len("as of "):week_end]
  week = week.split()
  dates.append(week)
  # nice, this works, explain how works with code later - you will review all this + comment before submission ofc

dates_num = []
for date in dates:
  month = date[0]
  map_months = {"Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4, "May": 5, "Jun": 6, "Jul": 7, "Aug": 8, "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12}
  month_no = str(map_months[month])
  day = str(date[1])
  day = "0" * (2 - len(day)) + day
  dates_num.append(int(month_no + day))

# sort the list ascending to get positions
# the week number of this will be index + 1?
# think about the indexing properly.
# we want Dec 31 to be the last week, don't want Dec 31 itself to fall into a new week
# right because it's as of, that's like the culmination of the week
# so [whatever - jan 23] [jan 24 - jan 30] [whatever - dec 24] [dec 25 - dec 31]
# so we do strictly greater than comparisons
# let's append before we sort the end of the week before jan 23
# so jan 16
# ok, so there is a one-index situation
# if it's greater than index 0, and less than equal to index 1, then it's week 1
# greater than index 1, less than equal to index 2, then it's week 2
# and so on
dates_num.append(123 - 7)
dates_num.sort()
dates_num

[116,
 123,
 130,
 206,
 213,
 220,
 227,
 305,
 312,
 319,
 326,
 402,
 409,
 416,
 423,
 430,
 507,
 514,
 521,
 528,
 604,
 611,
 618,
 625,
 702,
 709,
 716,
 723,
 730,
 806,
 813,
 820,
 827,
 903,
 910,
 917,
 924,
 1001,
 1008,
 1015,
 1022,
 1029,
 1105,
 1112,
 1119,
 1126,
 1203,
 1210,
 1217,
 1224,
 1231]

In [ ]:
def number_week_dash(date):
  # earlier work has given us conveniently dates_num list, which we will use

  month = str(month_dash(date))
  day = str(day_dash(date))
  day = "0" * (2 - len(day)) + day
  numbered_date = int(month + day)

  for i in range(len(dates_num)):
    if numbered_date <= dates_num[i]:
      return i

def number_week(date):
  # earlier work has given us conveniently dates_num list, which we will use

  month = str(month_slash(date))
  day = str(day_slash(date))
  day = "0" * (2 - len(day)) + day
  numbered_date = int(month + day)

  for i in range(len(dates_num)):
    if numbered_date <= dates_num[i]:
      return i

In [ ]:
daily_AQI_clean["week_num"] = daily_AQI_clean["Date"].apply(number_week_dash)
daily_AQI_clean

<ipython-input-63-e967c2803480>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  daily_AQI_clean["week_num"] = daily_AQI_clean["Date"].apply(number_week_dash)


,FIPS,Date,AQI,week_num
0,1003,2020-01-01,48,0
1,1003,2020-01-04,13,0
2,1003,2020-01-07,14,0
3,1003,2020-01-10,39,0
4,1003,2020-01-13,29,0
...,...,...,...,...
324333,56045,2020-12-27,32,50
324334,56045,2020-12-28,30,50
324335,56045,2020-12-29,33,50
324336,56045,2020-12-30,33,50


In [ ]:
weekly_AQI = daily_AQI_clean[["FIPS", "week_num", "AQI"]]
weekly_AQI = weekly_AQI.groupby(["FIPS", "week_num"]).mean()
weekly_AQI = weekly_AQI.reset_index()
weekly_AQI

,FIPS,week_num,AQI
0,1003,0,26.333333
1,1003,1,24.000000
2,1003,2,18.500000
3,1003,3,22.333333
4,1003,4,27.500000
...,...,...,...
48653,80026,15,53.714286
48654,80026,16,74.857143
48655,80026,17,51.571429
48656,80026,18,55.142857


In [ ]:
hospitals_capacity["week_num"] = hospitals_capacity["collection_week"].apply(number_week)
hospitals_capacity["FIPS"] = hospitals_capacity["fips_code"]
weekly_hospitals = hospitals_capacity[["FIPS", "week_num", "beds", "ICU"]]
# doing this to make sure there aren't any weird signpost things,
# where two collection weeks for whatever reason fall in the same week bucket
# luckily there aren't
weekly_hospitals = weekly_hospitals.groupby(["FIPS", "week_num"]).mean()
weekly_hospitals = weekly_hospitals.reset_index()
weekly_hospitals

,FIPS,week_num,beds,ICU
0,1001.0,27,0.635332,0.285714
1,1001.0,28,0.659933,0.976190
2,1001.0,29,0.702091,0.952381
3,1001.0,30,0.714286,0.976190
4,1001.0,31,0.731707,0.857143
...,...,...,...,...
52083,78020.0,46,0.427489,0.785714
52084,78020.0,47,0.440476,0.857143
52085,78020.0,48,0.459957,0.750000
52086,78020.0,49,0.445887,0.732143


In [ ]:
weekly = weekly_hospitals.merge(weekly_AQI, how="outer", on=("FIPS", "week_num"), validate="one_to_one")
weekly

,FIPS,week_num,beds,ICU,AQI
0,1001.0,27,0.635332,0.285714,NaN
1,1001.0,28,0.659933,0.976190,NaN
2,1001.0,29,0.702091,0.952381,NaN
3,1001.0,30,0.714286,0.976190,NaN
4,1001.0,31,0.731707,0.857143,NaN
...,...,...,...,...,...
81074,80026.0,15,NaN,NaN,53.714286
81075,80026.0,16,NaN,NaN,74.857143
81076,80026.0,17,NaN,NaN,51.571429
81077,80026.0,18,NaN,NaN,55.142857


In [ ]:
# because it's a many-to-many merge, we'll have lots of additional duplicates
# which will then be harmlessly filtered out
# we'll leave the code like so, because we can spare the compute
state_codes = coarse[["State Postal Abbreviation", "States"]].merge(daily_AQI[[
    "State Name", "State Code"]], left_on="States", right_on="State Name",
    validate="many_to_many")
state_codes = state_codes.groupby("State Postal Abbreviation").mean().reset_index()
state_codes

,State Postal Abbreviation,State Code
0,al,1.0
1,ar,5.0
2,az,4.0
3,ca,6.0
4,co,8.0
5,ct,9.0
6,de,10.0
7,fl,12.0
8,ga,13.0
9,ia,19.0


In [ ]:
# convert state codes to uppercase to match their comb_temp_precip dataframe
state_codes["State Postal Abbreviation"] = state_codes["State Postal Abbreviation"].str.upper()
state_codes

,State Postal Abbreviation,State Code
0,AL,1.0
1,AR,5.0
2,AZ,4.0
3,CA,6.0
4,CO,8.0
5,CT,9.0
6,DE,10.0
7,FL,12.0
8,GA,13.0
9,IA,19.0


In [ ]:
# convert state codes dataframe to dictionary,
# by first creating a series whose values are the state codes,
# and whose indices are the uppercase state postal abbreviations
# then convert this to dictionary mapping indices to values
state_code_dict = pd.Series(state_codes["State Code"].values, index = state_codes["State Postal Abbreviation"]).to_dict()
state_code_dict

{'AL': 1.0,
 'AR': 5.0,
 'AZ': 4.0,
 'CA': 6.0,
 'CO': 8.0,
 'CT': 9.0,
 'DE': 10.0,
 'FL': 12.0,
 'GA': 13.0,
 'IA': 19.0,
 'ID': 16.0,
 'IL': 17.0,
 'IN': 18.0,
 'KS': 20.0,
 'KY': 21.0,
 'LA': 22.0,
 'MA': 25.0,
 'MD': 24.0,
 'ME': 23.0,
 'MI': 26.0,
 'MN': 27.0,
 'MO': 29.0,
 'MS': 28.0,
 'MT': 30.0,
 'NC': 37.0,
 'ND': 38.0,
 'NE': 31.0,
 'NH': 33.0,
 'NJ': 34.0,
 'NM': 35.0,
 'NV': 32.0,
 'NY': 36.0,
 'OH': 39.0,
 'OK': 40.0,
 'OR': 41.0,
 'PA': 42.0,
 'RI': 44.0,
 'SC': 45.0,
 'SD': 46.0,
 'TN': 47.0,
 'TX': 48.0,
 'UT': 49.0,
 'VA': 51.0,
 'VT': 50.0,
 'WA': 53.0,
 'WI': 55.0,
 'WV': 54.0,
 'WY': 56.0}

In [ ]:
# now, for comb_temp_precip, we need to convert location ID to proper FIPS
# ok, then be simple, just merge it into coarse for yearly
# then clean to yearly
# also use that yearly as the basis for incorporating the weekly data + adjacency

# first we need to find a dictionary that maps state abbreviation to state code
# we did this above, comment that code properly later

def mixedCode_FIPS(code):
  dash = code.find("-")
  state = code[:dash]
  # if the county's state is outside the dataframe, return None
  # enables us to easily filter out these out-of-dataset observations later
  if state not in state_code_dict:
    return None
  state = str(int(state_code_dict[state]))
  county = code[dash + 1:]
  return int(state + county)

In [ ]:
comb_temp_precip["FIPS_num"] = comb_temp_precip["Location ID"].apply(mixedCode_FIPS)
comb_temp_precip = comb_temp_precip.dropna()
comb_temp_precip

,Location ID,Location_temp,Value_202001_temp,Value_202002_temp,Value_202003_temp,Value_202004_temp,Value_202005_temp,Value_202006_temp,Value_202007_temp,Value_202008_temp,...,Value_202004_precip,Value_202005_precip,Value_202006_precip,Value_202007_precip,Value_202008_precip,Value_20209_precip,Value_202010_precip,Value_202011_precip,Value_202012_precip,FIPS_num
0,AL-001,Autauga County,50.7,52.5,65.2,63.4,70.2,78.2,82.2,81.6,...,8.32,3.06,4.94,5.36,4.53,5.12,5.67,3.66,3.09,1001.0
1,AL-003,Baldwin County,54.8,56.6,68.4,67.3,72.6,79.4,81.8,81.8,...,3.64,2.57,7.28,9.73,6.95,11.71,4.29,2.23,3.85,1003.0
2,AL-005,Barbour County,51.6,52.5,65.8,64.2,70.3,77.4,80.9,80.8,...,6.45,4.53,4.46,3.96,5.94,13.99,2.81,3.11,2.34,1005.0
3,AL-007,Bibb County,48.6,49.7,62.4,60.8,68.2,76.1,80.5,79.8,...,8.49,2.53,3.56,4.71,4.48,2.46,6.59,3.17,3.47,1007.0
4,AL-009,Blount County,47.1,48.1,60.4,58.8,66.8,75.5,79.6,78.8,...,8.75,3.81,3.88,3.89,5.08,3.32,6.64,2.15,5.47,1009.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3102,WY-037,Sweetwater County,20.8,18.0,32.6,39.1,51.1,59.0,67.2,68.4,...,0.64,0.47,1.31,0.38,0.14,0.51,0.26,0.38,0.52,56037.0
3103,WY-039,Teton County,19.5,15.1,27.3,31.6,43.0,48.7,56.2,58.8,...,3.53,2.56,3.04,0.94,0.43,1.32,2.40,4.11,2.48,56039.0
3104,WY-041,Uinta County,20.8,18.3,30.7,37.9,48.3,54.9,63.0,64.5,...,0.47,0.47,1.92,0.16,0.20,0.68,0.45,0.64,0.49,56041.0
3105,WY-043,Washakie County,26.2,21.9,35.9,40.2,54.2,64.1,70.5,70.9,...,1.04,0.91,1.03,0.16,0.32,0.71,0.82,0.20,0.41,56043.0


**Step 3: Data Preparation & Artificial Intelligence (Yearly)**

In [ ]:
# first, finalize yearly data
# one county gets dropped in this, it really isn't the end of the world if no precip data, that's all fine - just one county
# merge comb_temp_precip with yearly

yearly = coarse.merge(comb_temp_precip, on="FIPS_num", validate="one_to_one")
yearly

,FIPS,Area Name,Qualifying Name,State Postal Abbreviation,Total Population,Population Density (Per Sq. Mile),Total Population: Male,Total Population: Female,Median Age:,Gini Index,...,Value_202003_precip,Value_202004_precip,Value_202005_precip,Value_202006_precip,Value_202007_precip,Value_202008_precip,Value_20209_precip,Value_202010_precip,Value_202011_precip,Value_202012_precip
0,01001,Autauga County,"Autauga County, Alabama",al,55380,93.16273,26934,28446,38.2,0.4542,...,6.55,8.32,3.06,4.94,5.36,4.53,5.12,5.67,3.66,3.09
1,01003,Baldwin County,"Baldwin County, Alabama",al,212830,133.8703,103496,109334,43,0.4587,...,0.89,3.64,2.57,7.28,9.73,6.95,11.71,4.29,2.23,3.85
2,01005,Barbour County,"Barbour County, Alabama",al,25361,28.65624,13421,11940,40.4,0.4883,...,5.09,6.45,4.53,4.46,3.96,5.94,13.99,2.81,3.11,2.34
3,01007,Bibb County,"Bibb County, Alabama",al,22493,36.13558,12150,10343,40.9,0.4487,...,7.91,8.49,2.53,3.56,4.71,4.48,2.46,6.59,3.17,3.47
4,01009,Blount County,"Blount County, Alabama",al,57681,89.45139,28495,29186,40.7,0.457,...,9.56,8.75,3.81,3.88,3.89,5.08,3.32,6.64,2.15,5.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3098,56037,Sweetwater County,"Sweetwater County, Wyoming",wy,43521,4.173883,22471,21050,35.3,0.4065,...,0.74,0.64,0.47,1.31,0.38,0.14,0.51,0.26,0.38,0.52
3099,56039,Teton County,"Teton County, Wyoming",wy,23280,5.824592,12325,10955,39.3,0.5003,...,4.33,3.53,2.56,3.04,0.94,0.43,1.32,2.40,4.11,2.48
3100,56041,Uinta County,"Uinta County, Wyoming",wy,20479,9.837535,10406,10073,35.8,0.3861,...,1.28,0.47,0.47,1.92,0.16,0.20,0.68,0.45,0.64,0.49
3101,56043,Washakie County,"Washakie County, Wyoming",wy,8027,3.585605,4065,3962,42.9,0.3868,...,0.32,1.04,0.91,1.03,0.16,0.32,0.71,0.82,0.20,0.41


In [ ]:
# eliminate second header row from yearly_cases and yearly_deaths
yearly_cases = yearly_cases.iloc[1:]
yearly_deaths = yearly_deaths.iloc[1:]

In [ ]:
yearly_deaths

,FIPS,Name of Area,Qualifying Name,Cumulative Number of Deaths as of Dec 31 2020
1,01001,Autauga County,"Autauga County, Alabama",48
2,01003,Baldwin County,"Baldwin County, Alabama",161
3,01005,Barbour County,"Barbour County, Alabama",32
4,01007,Bibb County,"Bibb County, Alabama",46
5,01009,Blount County,"Blount County, Alabama",63
...,...,...,...,...
3276,72999,Unknown County,"Unknown County, Puerto Rico",1503.0
3277,78010,St. Croix County,"St. Croix County, Virgin Islands",7.0
3278,78020,St. John County,"St. John County, Virgin Islands",1.0
3279,78030,St. Thomas County,"St. Thomas County, Virgin Islands",15.0


In [ ]:
# we will now implement spatial autoregression
# (https://www.sciencedirect.com/topics/earth-and-planetary-sciences/spatial-autoregressive-model)
# to do so, we'll incorporate COVID cases/deaths from physically-bordering counties
# as part of our COVID cases/deaths predictions for a given county
# (the idea being that, independent of a county's attributes,
# mere proximity to COVID alters a county's COVID experience by disease spread)

# first, narrow down COVID cases/deaths data to include only counties left in dataset

# convert FIPS data to numeric to avoid inconsistencies with
# string/integer data type and number of leading zeroes
yearly_cases["FIPS"] = yearly_cases["FIPS"].astype(int)
yearly_deaths["FIPS"] = yearly_deaths["FIPS"].astype(int)

# inner-merge yearly_cases with the FIPS column of yearly dataframe
# to narrow down to only counties in dataset
narrow_yearly_cases = yearly_cases.merge(yearly[[
    "FIPS_num"]], left_on="FIPS", right_on="FIPS_num", how="right", validate="one_to_one")
narrow_yearly_cases = narrow_yearly_cases.set_index("FIPS_num")
narrow_yearly_deaths = yearly_deaths.merge(yearly[[
    "FIPS_num"]], left_on="FIPS", right_on="FIPS_num", how="right", validate="one_to_one")
narrow_yearly_deaths = narrow_yearly_deaths.set_index("FIPS_num")

# second, narrow down adjacency matrix to include only counties left in dataset
# adjacency matrix uses county data with leading zeroes,
# so convert yearly FIPS series to string (earlier counties correctly have leading zeroes,
# later instances are numeric)
# and create a new dataframe out of only those rows and columns in adjacency
# that are also counties in our yearly dataframe
# convert labels to numeric to avoid compatibility issues
narrow_adjacency = pd.DataFrame(adjacency, columns=yearly[
    "FIPS"].astype(str).values, index=yearly["FIPS"].astype(str).values)
# convert narrow adjacency matrix's labels to numeric
adjacency.columns = adjacency.columns.astype(int)
adjacency.index = adjacency.index.astype(int)
narrow_adjacency = pd.DataFrame(adjacency, columns=yearly[
    "FIPS_num"].astype(int).values, index=yearly["FIPS_num"].astype(int).values)
narrow_adjacency

<ipython-input-75-faeaa72ee7bb>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yearly_cases["FIPS"] = yearly_cases["FIPS"].astype(int)
<ipython-input-75-faeaa72ee7bb>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yearly_deaths["FIPS"] = yearly_deaths["FIPS"].astype(int)


,1001,1003,1005,1007,1009,1011,1013,1015,1017,1019,...,56027,56029,56031,56033,56035,56037,56039,56041,56043,56045
1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1005,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
56039,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
56041,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
56043,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# interestingly, there is exactly one county missing from original adjacency matrix
# county 46102 - which is one of 2 counties in the US to lack a county seat,
# and was renamed in 2014, after my adjacency dataset was produced

narrow_adjacency.isna().sum().sort_values(ascending=False)

46102    3103
1001        1
39137       1
39119       1
39121       1
         ... 
21159       1
21161       1
21163       1
21165       1
56045       1
Length: 3103, dtype: int64

In [ ]:
# I will have to fill this data in by hand,
# consulting https://en.wikipedia.org/wiki/Oglala_Lakota_County,_South_Dakota
# and Wikipedia lists of South Dakota and Nebraska counties
narrow_adjacency = narrow_adjacency.fillna(0)
# fill in adjacent counties
# (1) in 46102's column
narrow_adjacency[46102][46103] = 1
narrow_adjacency[46102][46071] = 1
narrow_adjacency[46102][46007] = 1
narrow_adjacency[46102][46047] = 1
narrow_adjacency[46102][46033] = 1
narrow_adjacency[46102][28161] = 1
narrow_adjacency[46102][28045] = 1
# (2) in their own columns
narrow_adjacency[46103][46102] = 1
narrow_adjacency[46071][46102] = 1
narrow_adjacency[46007][46102] = 1
narrow_adjacency[46047][46102] = 1
narrow_adjacency[46033][46102] = 1
narrow_adjacency[28161][46102] = 1
narrow_adjacency[28045][46102] = 1
narrow_adjacency.isna().sum().sum()

0

In [ ]:
# strangely, COVID data for all NYC counties is missing
narrow_yearly_cases.isna().sum()
narrow_yearly_cases[narrow_yearly_cases["Cumulative Number of Confirmed Cases as of Dec 31 2020"].isna()]
yearly_cases[yearly_cases["FIPS"] == 36005]
COVID[COVID["FIPS"] == 36005]

,FIPS,Name of Area,Qualifying Name,Nation,State,County,Metropolitan Statistical Area,Cumulative Number of Confirmed Cases as of Dec 31 2020,Cumulative Number of Deaths as of Dec 31 2020,Number of Currently Hospitalized as of Dec 31 2020,...,Change in Cumulative Confirmed Cases as of Jan 21 2020 from Previous Month,Percent Change in Cumulative Confirmed Cases as of Jan 21 2020 from Previous Month,Change in Cumulative Deaths as of Jan 21 2020 from Previous Week,Percent Change in Cumulative Deaths as of Jan 21 2020 from Previous Week,Change in Cumulative Deaths as of Jan 21 2020 from Previous Month,Percent Change in Cumulative Deaths as of Jan 21 2020 from Previous Month,Cumulative Percent of Population with Confirmed Cases as of Jan 21 2020,Cumulative Percent of Population Deaths as of Jan 21 2020,"Cumulative Confirmed Cases Rate per 100,000 as of Jan 21 2020","Cumulative Death Rate per 100,000 as of Jan 21 2020"
1862,36005,Bronx County,"Bronx County, New York",0,36,5,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# NYC COVID data is missing even in the original file
# perhaps because of Cuomo-De Blasio disputes?
# at any rate, I have no choice but to filter out these counties too

narrow_yearly_cases = narrow_yearly_cases.dropna()
narrow_yearly_cases

narrow_yearly_deaths = narrow_yearly_deaths.dropna()
narrow_yearly_deaths

yearly = yearly.merge(narrow_yearly_cases[["FIPS"]], left_on="FIPS_num", right_on="FIPS",
             validate="one_to_one")
yearly

narrow_adjacency = pd.DataFrame(narrow_adjacency, columns=yearly[
    "FIPS_num"].astype(int).values, index=yearly["FIPS_num"].astype(int).values)
narrow_adjacency.isna().sum().sum(), narrow_adjacency

(0,
        1001   1003   1005   1007   1009   1011   1013   1015   1017   1019   \
 1001     0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
 1003     0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
 1005     0.0    0.0    0.0    0.0    0.0    1.0    0.0    0.0    0.0    0.0   
 1007     0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
 1009     0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
 ...      ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
 56037    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
 56039    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
 56041    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
 56043    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
 56045    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
 
        ...  56027  56029  56031  

In [ ]:
# convert cases and deaths data to per capita

# create a series with counties as index and populations as values
pops = yearly[["FIPS_num", "Total Population"]].set_index("FIPS_num")
pops = pops["Total Population"].sort_index().astype(float)
pops

# sort index of cases/deaths data to ensure compatibility
narrow_yearly_cases = narrow_yearly_cases.sort_index()
narrow_yearly_deaths = narrow_yearly_deaths.sort_index()

# divide cases by populations
# convert this series to a float series
# (because there used to be a double header which we eliminated,
# the series remained as general-purpose object, which posed difficulties later on)
narrow_yearly_cases["Cumulative Number of Confirmed Cases as of Dec 31 2020"] = narrow_yearly_cases["Cumulative Number of Confirmed Cases as of Dec 31 2020"].astype(float)
narrow_yearly_cases["pc_cases"] = narrow_yearly_cases["Cumulative Number of Confirmed Cases as of Dec 31 2020"].divide(pops)
narrow_yearly_cases

# divide deaths by population
narrow_yearly_deaths["Cumulative Number of Deaths as of Dec 31 2020"] = narrow_yearly_deaths["Cumulative Number of Deaths as of Dec 31 2020"].astype(float)
narrow_yearly_deaths["pc_deaths"] = narrow_yearly_deaths["Cumulative Number of Deaths as of Dec 31 2020"].divide(pops)
narrow_yearly_deaths

,FIPS,Name of Area,Qualifying Name,Cumulative Number of Deaths as of Dec 31 2020,pc_deaths
FIPS_num,,,,,
1001,1001,Autauga County,"Autauga County, Alabama",48.0,0.000867
1003,1003,Baldwin County,"Baldwin County, Alabama",161.0,0.000756
1005,1005,Barbour County,"Barbour County, Alabama",32.0,0.001262
1007,1007,Bibb County,"Bibb County, Alabama",46.0,0.002045
1009,1009,Blount County,"Blount County, Alabama",63.0,0.001092
...,...,...,...,...,...
56037,56037,Sweetwater County,"Sweetwater County, Wyoming",16.0,0.000368
56039,56039,Teton County,"Teton County, Wyoming",4.0,0.000172
56041,56041,Uinta County,"Uinta County, Wyoming",7.0,0.000342


In [ ]:
# now, create a function that performs spatial autoregression on cases
def SAR_cases(county):
  # count the number of adjacent counties by summing the county's adjacency column
  adj_count = narrow_adjacency[county].sum()
  # find dot-product between adjacent counties and per capita cases
  # (cases of all non-adjacent counties will get zeroed out)
  # we compute this average of averages to upweight smaller-population adjacent counties,
  # as whether a large-population or small-population adjacent county is overrun with COVID
  # might have similar effect on an adjacent county
  adj_cases = narrow_adjacency[county].dot(narrow_yearly_cases["pc_cases"])
  return adj_cases / adj_count

def SAR_deaths(county):
  # count the number of adjacent counties by summing the county's adjacency column
  adj_count = narrow_adjacency[county].sum()
  # find dot-product between adjacent counties and per capita cases
  # (cases of all non-adjacent counties will get zeroed out)
  # we compute this average of averages to upweight smaller-population adjacent counties,
  # as whether a large-population or small-population adjacent county is overrun with COVID
  # might have similar effect on an adjacent county
  adj_cases = narrow_adjacency[county].dot(narrow_yearly_deaths["pc_deaths"])
  return adj_cases / adj_count

In [ ]:
# to train our COVID cases AI predictors,
# create a yearly cases input dataframe
yearly_cases_input = yearly
yearly_cases_input["adj_cases"] = yearly["FIPS_num"].apply(SAR_cases)
yearly_cases_input

# (create yearly deaths input a bit later
# to avoid repeating input filtering on cases that's applicable to both)

# create a yearly cases output dataframe similarly
yearly_cases_output = narrow_yearly_cases
yearly_cases_output

yearly_deaths_output = narrow_yearly_deaths
yearly_deaths_output

,FIPS,Name of Area,Qualifying Name,Cumulative Number of Deaths as of Dec 31 2020,pc_deaths
FIPS_num,,,,,
1001,1001,Autauga County,"Autauga County, Alabama",48.0,0.000867
1003,1003,Baldwin County,"Baldwin County, Alabama",161.0,0.000756
1005,1005,Barbour County,"Barbour County, Alabama",32.0,0.001262
1007,1007,Bibb County,"Bibb County, Alabama",46.0,0.002045
1009,1009,Blount County,"Blount County, Alabama",63.0,0.001092
...,...,...,...,...,...
56037,56037,Sweetwater County,"Sweetwater County, Wyoming",16.0,0.000368
56039,56039,Teton County,"Teton County, Wyoming",4.0,0.000172
56041,56041,Uinta County,"Uinta County, Wyoming",7.0,0.000342


In [ ]:
# perform one final round of cleaning on inputs and outputs
# to prepare for sklearn models

yearly_cases_output = yearly_cases_output[["pc_cases"]]
yearly_deaths_output = yearly_deaths_output[["pc_deaths"]]

# for inputs: (1) normalize for population
# (2) generally, if two measures are perfectly correlated (ex. % men and % women),
# eliminate one of them to mitigate multicollinearity
# (3) collapse measures as needed
yearly_cases_input["%male"] = yearly_cases_input["Total Population: Male"].astype(float) / yearly_cases_input["Total Population"].astype(float)
yearly_cases_input["%65+yrs"] = (yearly_cases_input[
    "Total Population: 85 Years and Over"].astype(float) +  yearly_cases_input[
        "Total Population: 65 to 74 Years"].astype(float) + yearly_cases_input[
            "Total Population: 75 to 84 Years"].astype(float)) / yearly_cases_input["Total Population"].astype(float)
yearly_cases_input["%85+yrs"] = yearly_cases_input["Total Population: 85 Years and Over"].astype(float) / yearly_cases_input["Total Population"].astype(float)
yearly_cases_input["%white"] = (yearly_cases_input[
    "Total Population: Not Hispanic or Latino: White Alone"].astype(float) +  yearly_cases_input[
        "Total Population: Hispanic or Latino: White Alone"].astype(float)) / yearly_cases_input["Total Population"].astype(float)
yearly_cases_input["%black"] = (yearly_cases_input[
    "Total Population: Not Hispanic or Latino: Black or African American Alone"].astype(float) +  yearly_cases_input[
        "Total Population: Hispanic or Latino: Black or African American Alone"].astype(float)) / yearly_cases_input["Total Population"].astype(float)
yearly_cases_input["%under_poverty_line"] = (yearly_cases_input[
    "Population for Whom Poverty Status Is Determined: Under .50"].astype(float) +  yearly_cases_input[
        "Population for Whom Poverty Status Is Determined: .50 to .74"].astype(float) + yearly_cases_input[
            "Population for Whom Poverty Status Is Determined: .75 to .99"].astype(float)) / yearly_cases_input["Population for Whom Poverty Status Is Determined:"].astype(float)
yearly_cases_input["%college"] = (yearly_cases_input[
    "Population 25 Years and Over: Some College"].astype(float) +  yearly_cases_input[
        "Population 25 Years and Over: Bachelor's Degree"].astype(float) + yearly_cases_input[
            "Population 25 Years and Over: Master's Degree"].astype(float) + yearly_cases_input[
                "Population 25 Years and Over: Professional School Degree"].astype(float) + yearly_cases_input[
                    "Population 25 Years and Over: Doctorate Degree"].astype(float)) / yearly_cases_input["Total Population"].astype(float)
yearly_cases_input["Homeownership"] = yearly_cases_input["Owner Occupied Housing Units"].astype(float) / yearly_cases_input["Housing Units"].astype(float)
yearly_cases_input["median_age"] = yearly_cases_input["Median Age:"]
yearly_cases_input["pop_density"] = yearly_cases_input["Population Density (Per Sq. Mile)"]
yearly_cases_input = yearly_cases_input.loc[:, "NEVER_masks":]
yearly_cases_input = yearly_cases_input.drop(["States", "Location ID", "Location_temp", "FIPS_y"], axis=1)
yearly_cases_input

yearly_deaths_input = yearly_cases_input.drop("adj_cases", axis=1)
yearly_deaths_input["adj_deaths"] = yearly["FIPS_num"].apply(SAR_deaths)
yearly_deaths_input

,NEVER_masks,RARELY_masks,SOMETIMES_masks,FREQUENTLY_masks,ALWAYS_masks,smokers,diabetes,obesity,inactive,heartdeaths,...,%65+yrs,%85+yrs,%white,%black,%under_poverty_line,%college,Homeownership,median_age,pop_density,adj_deaths
0,0.053,0.074,0.134,0.295,0.444,18.0,9.6,30.9,26.6,198.0,...,0.149567,0.015962,0.767913,0.190285,0.151852,0.370621,0.667518,38.2,93.16273,0.001548
1,0.083,0.059,0.098,0.323,0.436,16.1,7.9,27.7,22.8,188.6,...,0.199836,0.019020,0.862054,0.092647,0.103541,0.448348,0.533408,43,133.8703,0.001056
2,0.067,0.121,0.120,0.201,0.491,24.8,11.3,24.8,27.5,278.5,...,0.185718,0.015969,0.468002,0.475770,0.306687,0.266393,0.473737,40.4,28.65624,0.001105
3,0.020,0.034,0.096,0.278,0.572,22.4,10.2,39.3,27.8,256.1,...,0.159338,0.019651,0.767883,0.222914,0.181272,0.245676,0.558302,40.9,36.13558,0.001187
4,0.053,0.114,0.180,0.194,0.459,21.0,9.5,35.8,25.5,225.6,...,0.179019,0.018290,0.954595,0.016088,0.135515,0.324700,0.675205,40.7,89.45139,0.001132
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3093,0.061,0.295,0.230,0.146,0.268,17.5,7.6,33.8,21.4,160.1,...,0.114244,0.011075,0.933848,0.011535,0.114554,0.383861,0.598452,35.3,4.173883,0.000710
3094,0.095,0.157,0.160,0.247,0.340,12.2,4.2,15.4,12.0,94.9,...,0.140378,0.014519,0.892826,0.012457,0.061497,0.597595,0.393414,39.3,5.824592,0.000719
3095,0.098,0.278,0.154,0.207,0.264,18.6,7.3,28.3,23.8,180.6,...,0.130133,0.014649,0.934225,0.001123,0.113431,0.324235,0.647495,35.8,9.837535,0.000265
3096,0.204,0.155,0.069,0.285,0.287,16.7,8.8,24.5,19.8,160.9,...,0.210913,0.023172,0.897471,0.000374,0.105055,0.414601,0.666839,42.9,3.585605,0.000965


In [ ]:
# core yearly variable data for data exploration
# also useful for implementing recursive time-series learning
yearly_new = yearly_cases_input.drop("adj_cases", axis=1)
yearly_new

,NEVER_masks,RARELY_masks,SOMETIMES_masks,FREQUENTLY_masks,ALWAYS_masks,smokers,diabetes,obesity,inactive,heartdeaths,...,%male,%65+yrs,%85+yrs,%white,%black,%under_poverty_line,%college,Homeownership,median_age,pop_density
0,0.053,0.074,0.134,0.295,0.444,18.0,9.6,30.9,26.6,198.0,...,0.486349,0.149567,0.015962,0.767913,0.190285,0.151852,0.370621,0.667518,38.2,93.16273
1,0.083,0.059,0.098,0.323,0.436,16.1,7.9,27.7,22.8,188.6,...,0.486285,0.199836,0.019020,0.862054,0.092647,0.103541,0.448348,0.533408,43,133.8703
2,0.067,0.121,0.120,0.201,0.491,24.8,11.3,24.8,27.5,278.5,...,0.529198,0.185718,0.015969,0.468002,0.475770,0.306687,0.266393,0.473737,40.4,28.65624
3,0.020,0.034,0.096,0.278,0.572,22.4,10.2,39.3,27.8,256.1,...,0.540168,0.159338,0.019651,0.767883,0.222914,0.181272,0.245676,0.558302,40.9,36.13558
4,0.053,0.114,0.180,0.194,0.459,21.0,9.5,35.8,25.5,225.6,...,0.494010,0.179019,0.018290,0.954595,0.016088,0.135515,0.324700,0.675205,40.7,89.45139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3093,0.061,0.295,0.230,0.146,0.268,17.5,7.6,33.8,21.4,160.1,...,0.516325,0.114244,0.011075,0.933848,0.011535,0.114554,0.383861,0.598452,35.3,4.173883
3094,0.095,0.157,0.160,0.247,0.340,12.2,4.2,15.4,12.0,94.9,...,0.529424,0.140378,0.014519,0.892826,0.012457,0.061497,0.597595,0.393414,39.3,5.824592
3095,0.098,0.278,0.154,0.207,0.264,18.6,7.3,28.3,23.8,180.6,...,0.508130,0.130133,0.014649,0.934225,0.001123,0.113431,0.324235,0.647495,35.8,9.837535
3096,0.204,0.155,0.069,0.285,0.287,16.7,8.8,24.5,19.8,160.9,...,0.506416,0.210913,0.023172,0.897471,0.000374,0.105055,0.414601,0.666839,42.9,3.585605


**Step 5: Weekly AI**

In [ ]:
# now, we're going to implement time-series recursive learning
# the plan is to train our model on the first 45 weeks of the year
# then test it on the remaining 5
# (county, week) will be the primary key tuple for our training data,
# so we'll have 45 observations per county
# the goal is for the model to predict the next week
# so for training, it'll train on the previous week's COVID data,
# plus general county-level attributes (demographics, etc.) that we assume constant,
# plus weekly data

# for testing, we'll have the model predict the next week (week 46, with week 45 data),
# check its error, then use the predicted week 47 data to predict week 46,
# and iterate until we've reached the end of 2020

In [ ]:
# drop weather data, as you don't want a time-series model
# predicting the present based on future data
yearly_forweeks = yearly_new.drop(yearly_new.iloc[:, 40:64], axis=1)
yearly_forweeks

,NEVER_masks,RARELY_masks,SOMETIMES_masks,FREQUENTLY_masks,ALWAYS_masks,smokers,diabetes,obesity,inactive,heartdeaths,...,%male,%65+yrs,%85+yrs,%white,%black,%under_poverty_line,%college,Homeownership,median_age,pop_density
0,0.053,0.074,0.134,0.295,0.444,18.0,9.6,30.9,26.6,198.0,...,0.486349,0.149567,0.015962,0.767913,0.190285,0.151852,0.370621,0.667518,38.2,93.16273
1,0.083,0.059,0.098,0.323,0.436,16.1,7.9,27.7,22.8,188.6,...,0.486285,0.199836,0.019020,0.862054,0.092647,0.103541,0.448348,0.533408,43,133.8703
2,0.067,0.121,0.120,0.201,0.491,24.8,11.3,24.8,27.5,278.5,...,0.529198,0.185718,0.015969,0.468002,0.475770,0.306687,0.266393,0.473737,40.4,28.65624
3,0.020,0.034,0.096,0.278,0.572,22.4,10.2,39.3,27.8,256.1,...,0.540168,0.159338,0.019651,0.767883,0.222914,0.181272,0.245676,0.558302,40.9,36.13558
4,0.053,0.114,0.180,0.194,0.459,21.0,9.5,35.8,25.5,225.6,...,0.494010,0.179019,0.018290,0.954595,0.016088,0.135515,0.324700,0.675205,40.7,89.45139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3093,0.061,0.295,0.230,0.146,0.268,17.5,7.6,33.8,21.4,160.1,...,0.516325,0.114244,0.011075,0.933848,0.011535,0.114554,0.383861,0.598452,35.3,4.173883
3094,0.095,0.157,0.160,0.247,0.340,12.2,4.2,15.4,12.0,94.9,...,0.529424,0.140378,0.014519,0.892826,0.012457,0.061497,0.597595,0.393414,39.3,5.824592
3095,0.098,0.278,0.154,0.207,0.264,18.6,7.3,28.3,23.8,180.6,...,0.508130,0.130133,0.014649,0.934225,0.001123,0.113431,0.324235,0.647495,35.8,9.837535
3096,0.204,0.155,0.069,0.285,0.287,16.7,8.8,24.5,19.8,160.9,...,0.506416,0.210913,0.023172,0.897471,0.000374,0.105055,0.414601,0.666839,42.9,3.585605


In [ ]:
# for ease, move FIPS_num column to front and rename as FIPS
col = comb_temp_precip.pop("FIPS_num")
comb_temp_precip.insert(0, "FIPS", col)
comb_temp_precip

,FIPS,Location ID,Location_temp,Value_202001_temp,Value_202002_temp,Value_202003_temp,Value_202004_temp,Value_202005_temp,Value_202006_temp,Value_202007_temp,...,Value_202003_precip,Value_202004_precip,Value_202005_precip,Value_202006_precip,Value_202007_precip,Value_202008_precip,Value_20209_precip,Value_202010_precip,Value_202011_precip,Value_202012_precip
0,1001.0,AL-001,Autauga County,50.7,52.5,65.2,63.4,70.2,78.2,82.2,...,6.55,8.32,3.06,4.94,5.36,4.53,5.12,5.67,3.66,3.09
1,1003.0,AL-003,Baldwin County,54.8,56.6,68.4,67.3,72.6,79.4,81.8,...,0.89,3.64,2.57,7.28,9.73,6.95,11.71,4.29,2.23,3.85
2,1005.0,AL-005,Barbour County,51.6,52.5,65.8,64.2,70.3,77.4,80.9,...,5.09,6.45,4.53,4.46,3.96,5.94,13.99,2.81,3.11,2.34
3,1007.0,AL-007,Bibb County,48.6,49.7,62.4,60.8,68.2,76.1,80.5,...,7.91,8.49,2.53,3.56,4.71,4.48,2.46,6.59,3.17,3.47
4,1009.0,AL-009,Blount County,47.1,48.1,60.4,58.8,66.8,75.5,79.6,...,9.56,8.75,3.81,3.88,3.89,5.08,3.32,6.64,2.15,5.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3102,56037.0,WY-037,Sweetwater County,20.8,18.0,32.6,39.1,51.1,59.0,67.2,...,0.74,0.64,0.47,1.31,0.38,0.14,0.51,0.26,0.38,0.52
3103,56039.0,WY-039,Teton County,19.5,15.1,27.3,31.6,43.0,48.7,56.2,...,4.33,3.53,2.56,3.04,0.94,0.43,1.32,2.40,4.11,2.48
3104,56041.0,WY-041,Uinta County,20.8,18.3,30.7,37.9,48.3,54.9,63.0,...,1.28,0.47,0.47,1.92,0.16,0.20,0.68,0.45,0.64,0.49
3105,56043.0,WY-043,Washakie County,26.2,21.9,35.9,40.2,54.2,64.1,70.5,...,0.32,1.04,0.91,1.03,0.16,0.32,0.71,0.82,0.20,0.41


In [ ]:
# use the .melt() method to turn wide dataframe into a long one,
# first dealing with precipitation variable
comb_temp_precip = comb_temp_precip.melt(id_vars=comb_temp_precip.columns[:15], value_name="precip")
comb_temp_precip

,FIPS,Location ID,Location_temp,Value_202001_temp,Value_202002_temp,Value_202003_temp,Value_202004_temp,Value_202005_temp,Value_202006_temp,Value_202007_temp,Value_202008_temp,Value_20209_temp,Value_202010_temp,Value_202011_temp,Value_202012_temp,variable,precip
0,1001.0,AL-001,Autauga County,50.7,52.5,65.2,63.4,70.2,78.2,82.2,81.6,76.0,68.4,59.6,46.8,Value_202001_precip,7.54
1,1003.0,AL-003,Baldwin County,54.8,56.6,68.4,67.3,72.6,79.4,81.8,81.8,77.2,71.2,64.1,51.0,Value_202001_precip,5.09
2,1005.0,AL-005,Barbour County,51.6,52.5,65.8,64.2,70.3,77.4,80.9,80.8,74.3,67.9,60.4,47.2,Value_202001_precip,8.93
3,1007.0,AL-007,Bibb County,48.6,49.7,62.4,60.8,68.2,76.1,80.5,79.8,74.0,66.1,57.3,44.8,Value_202001_precip,7.33
4,1009.0,AL-009,Blount County,47.1,48.1,60.4,58.8,66.8,75.5,79.6,78.8,72.6,64.8,55.6,43.5,Value_202001_precip,7.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37279,56037.0,WY-037,Sweetwater County,20.8,18.0,32.6,39.1,51.1,59.0,67.2,68.4,55.2,43.2,31.1,20.4,Value_202012_precip,0.52
37280,56039.0,WY-039,Teton County,19.5,15.1,27.3,31.6,43.0,48.7,56.2,58.8,48.8,36.3,23.8,16.2,Value_202012_precip,2.48
37281,56041.0,WY-041,Uinta County,20.8,18.3,30.7,37.9,48.3,54.9,63.0,64.5,53.0,43.3,30.1,21.2,Value_202012_precip,0.49
37282,56043.0,WY-043,Washakie County,26.2,21.9,35.9,40.2,54.2,64.1,70.5,70.9,58.6,42.0,34.5,26.6,Value_202012_precip,0.41


In [ ]:
# given a variable name describing its month, convert to week
# this will create some future contamination (as the model has all Jan 2020 data
# even for the first week of January)
# but I don't want to have to pass in null weather data for week 1
# a future extension of this project might be to avoid this source of future-contamination
def monthToWeek(varname):
  date_start = varname.find("_")
  date_end = varname.find("_", date_start + 1)
  month = int(varname[date_start + len("_2020"):date_end])
  result = []
  for i in range(len(dates_num)):
    dates_mo = int(str(dates_num[i])[:-2])
    if month == dates_mo:
      result.append(i)
  return result

# test it out - it works!
monthToWeek("a_202012_b")

[46, 47, 48, 49, 50]

In [ ]:
# convert months to list of weeks with the function,
# use explode to get multiple rows
precip_weekly = comb_temp_precip[["FIPS", "variable", "precip"]]
precip_weekly["variable"] = precip_weekly["variable"].apply(monthToWeek)
precip_weekly = precip_weekly.explode("variable")
precip_weekly

<ipython-input-90-71a115dcd8b6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_weekly["variable"] = precip_weekly["variable"].apply(monthToWeek)


,FIPS,variable,precip
0,1001.0,0,7.54
0,1001.0,1,7.54
0,1001.0,2,7.54
1,1003.0,0,5.09
1,1003.0,1,5.09
...,...,...,...
37283,56045.0,46,0.30
37283,56045.0,47,0.30
37283,56045.0,48,0.30
37283,56045.0,49,0.30


In [ ]:
# repeat the process for temperature data

comb_temp_precip = comb_temp_precip.drop(["variable", "precip"], axis=1)
# even after dropping columns, we're left with extra rows we must deal with
# mean() aggregation drops all non-numeric rows
comb_temp_precip = comb_temp_precip.groupby("FIPS").mean().reset_index()

In [ ]:
# use the .melt() method to turn wide dataframe into a long one,
# now dealing with temperature variable

comb_temp_precip = comb_temp_precip.melt(id_vars=comb_temp_precip.columns[:1], value_name="temp")

temp_weekly = comb_temp_precip[["FIPS", "variable", "temp"]]
temp_weekly["variable"] = temp_weekly["variable"].apply(monthToWeek)
temp_weekly = temp_weekly.explode("variable")
temp_weekly

,FIPS,variable,temp
0,1001.0,0,50.7
0,1001.0,1,50.7
0,1001.0,2,50.7
1,1003.0,0,54.8
1,1003.0,1,54.8
...,...,...,...
37283,56045.0,46,29.1
37283,56045.0,47,29.1
37283,56045.0,48,29.1
37283,56045.0,49,29.1


In [ ]:
# merge exploded precip and temp
weekly_weather = precip_weekly.merge(temp_weekly, validate="one_to_one", on=("FIPS", "variable"))
# luckily, there are no nulls
weekly_weather.isna().sum().sum()
weekly_weather
# 158457 / 37284 = 4.25 -> 51 weeks (exactly the right timescale here!)

,FIPS,variable,precip,temp
0,1001.0,0,7.54,50.7
1,1001.0,1,7.54,50.7
2,1001.0,2,7.54,50.7
3,1003.0,0,5.09,54.8
4,1003.0,1,5.09,54.8
...,...,...,...,...
158452,56045.0,46,0.30,29.1
158453,56045.0,47,0.30,29.1
158454,56045.0,48,0.30,29.1
158455,56045.0,49,0.30,29.1


In [ ]:
# name columns properly
weekly_weather = weekly_weather.rename(columns={"variable": "week_num"})

In [ ]:
weekly = weekly.merge(weekly_weather, on=("FIPS", "week_num"), validate="one_to_one", how="outer")

/usr/local/lib/python3.9/dist-packages/pandas/core/reshape/merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)


In [ ]:
# fill nulls with average values for the column
# better than filling in with zero
# infinite value for beds means that the hospital contains no beds
# fill with a value of 1 to indicate "many beds used" (low spare capacity)
# import numpy to handle infinities
import numpy as np

# find values to fill
weekly["beds"].replace(np.inf, 1, inplace=True)
beds_fill = weekly["beds"].mean()
ICU_fill = weekly["ICU"].mean()
AQI_fill = weekly["AQI"].mean()
precip_fill = weekly["precip"].mean()
temp_fill = weekly["temp"].mean()
beds_fill, ICU_fill, AQI_fill, precip_fill, temp_fill

(0.4937151284223621,
 0.5917070438352693,
 37.004388245260245,
 3.4782574452375092,
 56.33767772960488)

In [ ]:
weekly["beds"].fillna(beds_fill, inplace=True)
weekly["ICU"].fillna(ICU_fill, inplace=True)
weekly["AQI"].fillna(AQI_fill, inplace=True)
weekly["precip"].fillna(precip_fill, inplace=True)
weekly["temp"].fillna(temp_fill, inplace=True)

# no nulls left!
weekly.isna().sum()

FIPS        0
week_num    0
beds        0
ICU         0
AQI         0
precip      0
temp        0
dtype: int64

In [ ]:
narrow_weekly = weekly.merge(yearly[["FIPS_num"]], left_on="FIPS", right_on="FIPS_num", validate="many_to_one").drop("FIPS_num", axis=1)
# exactly right number of rows and such, 51 per county, as it should be
yearly_new["FIPS"] = yearly["FIPS_num"]
narrow_weekly = narrow_weekly.merge(yearly_new, on="FIPS", validate="many_to_one")
# cols = 7 + 75 - 1 = 81, that's correct, and number of rows is right too
narrow_weekly

,FIPS,week_num,beds,ICU,AQI_x,precip,temp,NEVER_masks,RARELY_masks,SOMETIMES_masks,...,%male,%65+yrs,%85+yrs,%white,%black,%under_poverty_line,%college,Homeownership,median_age,pop_density
0,1001.0,27.0,0.635332,0.285714,37.004388,5.36,82.2,0.053,0.074,0.134,...,0.486349,0.149567,0.015962,0.767913,0.190285,0.151852,0.370621,0.667518,38.2,93.16273
1,1001.0,28.0,0.659933,0.976190,37.004388,5.36,82.2,0.053,0.074,0.134,...,0.486349,0.149567,0.015962,0.767913,0.190285,0.151852,0.370621,0.667518,38.2,93.16273
2,1001.0,29.0,0.702091,0.952381,37.004388,4.53,81.6,0.053,0.074,0.134,...,0.486349,0.149567,0.015962,0.767913,0.190285,0.151852,0.370621,0.667518,38.2,93.16273
3,1001.0,30.0,0.714286,0.976190,37.004388,4.53,81.6,0.053,0.074,0.134,...,0.486349,0.149567,0.015962,0.767913,0.190285,0.151852,0.370621,0.667518,38.2,93.16273
4,1001.0,31.0,0.731707,0.857143,37.004388,4.53,81.6,0.053,0.074,0.134,...,0.486349,0.149567,0.015962,0.767913,0.190285,0.151852,0.370621,0.667518,38.2,93.16273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157993,55078.0,46,0.493715,0.591707,37.004388,0.53,24.7,0.042,0.103,0.310,...,0.503949,0.129662,0.007459,0.108820,0.002852,0.353137,0.284774,0.405760,32.0,12.74556
157994,55078.0,47,0.493715,0.591707,37.004388,0.53,24.7,0.042,0.103,0.310,...,0.503949,0.129662,0.007459,0.108820,0.002852,0.353137,0.284774,0.405760,32.0,12.74556
157995,55078.0,48,0.493715,0.591707,37.004388,0.53,24.7,0.042,0.103,0.310,...,0.503949,0.129662,0.007459,0.108820,0.002852,0.353137,0.284774,0.405760,32.0,12.74556
157996,55078.0,49,0.493715,0.591707,37.004388,0.53,24.7,0.042,0.103,0.310,...,0.503949,0.129662,0.007459,0.108820,0.002852,0.353137,0.284774,0.405760,32.0,12.74556


In [ ]:
# filter out second header row
weekly_change_cases = weekly_change_cases.iloc[1:]
weekly_total_cases = weekly_total_cases.iloc[1:]
weekly_change_deaths = weekly_change_deaths.iloc[1:]
weekly_total_deaths = weekly_total_deaths.iloc[1:]

In [ ]:
# convert variable names to week values
def varToWeek(varname):
  date_start = varname.find("as of ")
  date_end = varname.find(" 2020")
  date = (varname[date_start + len("as of "):date_end])
  date = date.split()
  day = str(date[1])
  day = "0" * (2 - len(day)) + day
  month = str(map_months[date[0]])
  numbered_date = int(month + day)

  for i in range(len(dates_num)):
    if numbered_date <= dates_num[i]:
      return i

In [ ]:
# for weekly data, narrow it down to include only counties left in main dataset

# convert FIPS data to numeric to avoid inconsistencies with
# string/integer data type and number of leading zeroes
weekly_change_cases["FIPS"] = weekly_change_cases["FIPS"].astype(int)
weekly_total_cases["FIPS"] = weekly_total_cases["FIPS"].astype(int)
weekly_change_deaths["FIPS"] = weekly_change_deaths["FIPS"].astype(int)
weekly_total_deaths["FIPS"] = weekly_total_deaths["FIPS"].astype(int)

# inner-merge with FIPS column of yearly dataframe to narrow down counties
narrow_change_cases = weekly_change_cases.merge(yearly[[
    "FIPS_num"]], left_on="FIPS", right_on="FIPS_num", how="right", validate="one_to_one").fillna(0)
# there are a lot of nulls in the dataset, because COVID began later in some counties than others
# these nulls indicate no COVID; fill in values with zero
narrow_change_cases = narrow_change_cases.set_index("FIPS_num")
# normalize data for population
narrow_change_cases.loc[
    :,"Change in Cumulative Confirmed Cases as of Dec 31 2020 from Previous Week":] = narrow_change_cases.loc[
        :, "Change in Cumulative Confirmed Cases as of Dec 31 2020 from Previous Week":].astype(float)
narrow_change_cases.loc[
    :,"Change in Cumulative Confirmed Cases as of Dec 31 2020 from Previous Week":] = narrow_change_cases.loc[
        :,"Change in Cumulative Confirmed Cases as of Dec 31 2020 from Previous Week":].divide(pops, axis=0)
# melt data to convert long to wide, then convert variable names to numbered weeks
narrow_change_cases = narrow_change_cases.melt(id_vars=["FIPS", "Name of Area", "Qualifying Name"], var_name="week_num", value_name="pc_cases")
narrow_change_cases["week_num"] = narrow_change_cases["week_num"].apply(varToWeek)
narrow_change_cases

# similarly, for the others
narrow_total_cases = weekly_total_cases.merge(yearly[[
    "FIPS_num"]], left_on="FIPS", right_on="FIPS_num", how="right", validate="one_to_one").fillna(0)
narrow_total_cases = narrow_total_cases.set_index("FIPS_num")
# normalize data for population
narrow_total_cases.loc[
    :,"Cumulative Number of Confirmed Cases as of Dec 31 2020":] = narrow_total_cases.loc[
        :, "Cumulative Number of Confirmed Cases as of Dec 31 2020":].astype(float)
narrow_total_cases.loc[
    :,"Cumulative Number of Confirmed Cases as of Dec 31 2020":] = narrow_total_cases.loc[
        :,"Cumulative Number of Confirmed Cases as of Dec 31 2020":].divide(pops, axis=0)
# melt data to convert long to wide, then convert variable names to numbered weeks
narrow_total_cases = narrow_total_cases.melt(id_vars=["FIPS", "Name of Area", "Qualifying Name"], var_name="week_num", value_name="pc_cases")
narrow_total_cases["week_num"] = narrow_total_cases["week_num"].apply(varToWeek)
narrow_total_cases

narrow_change_deaths = weekly_change_deaths.merge(yearly[[
    "FIPS_num"]], left_on="FIPS", right_on="FIPS_num", how="right", validate="one_to_one").fillna(0)
narrow_change_deaths = narrow_change_deaths.set_index("FIPS_num")
# normalize data for population
narrow_change_deaths.loc[
    :,"Change in Cumulative Deaths as of Dec 31 2020 from Previous Week":] = narrow_change_deaths.loc[
        :, "Change in Cumulative Deaths as of Dec 31 2020 from Previous Week":].astype(float)
narrow_change_deaths.loc[
    :,"Change in Cumulative Deaths as of Dec 31 2020 from Previous Week":] = narrow_change_deaths.loc[
        :,"Change in Cumulative Deaths as of Dec 31 2020 from Previous Week":].divide(pops, axis=0)
# melt data to convert long to wide, then convert variable names to numbered weeks
narrow_change_deaths = narrow_change_deaths.melt(id_vars=["FIPS", "Name of Area", "Qualifying Name"], var_name="week_num", value_name="pc_deaths")
narrow_change_deaths["week_num"] = narrow_change_deaths["week_num"].apply(varToWeek)
narrow_change_deaths

narrow_total_deaths = weekly_total_deaths.merge(yearly[[
    "FIPS_num"]], left_on="FIPS", right_on="FIPS_num", how="right", validate="one_to_one").fillna(0)
narrow_total_deaths = narrow_total_deaths.set_index("FIPS_num")
# normalize data for population
narrow_total_deaths.loc[
    :,"Cumulative Number of Deaths as of Dec 31 2020":] = narrow_total_deaths.loc[
        :, "Cumulative Number of Deaths as of Dec 31 2020":].astype(float)
narrow_total_deaths.loc[
    :,"Cumulative Number of Deaths as of Dec 31 2020":] = narrow_total_deaths.loc[
        :,"Cumulative Number of Deaths as of Dec 31 2020":].divide(pops, axis=0)
# melt data to convert long to wide, then convert variable names to numbered weeks
narrow_total_deaths = narrow_total_deaths.melt(id_vars=["FIPS", "Name of Area", "Qualifying Name"], var_name="week_num", value_name="pc_deaths")
narrow_total_deaths["week_num"] = narrow_total_deaths["week_num"].apply(varToWeek)
narrow_total_deaths

<ipython-input-101-82365c9bc5b8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weekly_change_cases["FIPS"] = weekly_change_cases["FIPS"].astype(int)
<ipython-input-101-82365c9bc5b8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weekly_total_cases["FIPS"] = weekly_total_cases["FIPS"].astype(int)
<ipython-input-101-82365c9bc5b8>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

,FIPS,Name of Area,Qualifying Name,week_num,pc_deaths
0,1001,Autauga County,"Autauga County, Alabama",50,0.000867
1,1003,Baldwin County,"Baldwin County, Alabama",50,0.000756
2,1005,Barbour County,"Barbour County, Alabama",50,0.001262
3,1007,Bibb County,"Bibb County, Alabama",50,0.002045
4,1009,Blount County,"Blount County, Alabama",50,0.001092
...,...,...,...,...,...
154895,56037,Sweetwater County,"Sweetwater County, Wyoming",1,0.000000
154896,56039,Teton County,"Teton County, Wyoming",1,0.000000
154897,56041,Uinta County,"Uinta County, Wyoming",1,0.000000
154898,56043,Washakie County,"Washakie County, Wyoming",1,0.000000


In [ ]:
narrow_total_deaths_input = narrow_total_deaths
narrow_total_deaths_input["week_num"] -= 1
narrow_total_deaths_input

,FIPS,Name of Area,Qualifying Name,week_num,pc_deaths
0,1001,Autauga County,"Autauga County, Alabama",49,0.000867
1,1003,Baldwin County,"Baldwin County, Alabama",49,0.000756
2,1005,Barbour County,"Barbour County, Alabama",49,0.001262
3,1007,Bibb County,"Bibb County, Alabama",49,0.002045
4,1009,Blount County,"Blount County, Alabama",49,0.001092
...,...,...,...,...,...
154895,56037,Sweetwater County,"Sweetwater County, Wyoming",0,0.000000
154896,56039,Teton County,"Teton County, Wyoming",0,0.000000
154897,56041,Uinta County,"Uinta County, Wyoming",0,0.000000
154898,56043,Washakie County,"Washakie County, Wyoming",0,0.000000


In [ ]:
all_total_deaths = narrow_weekly.merge(narrow_total_deaths_input, on=("FIPS", "week_num"), how="outer", validate="one_to_one").drop(["Name of Area", "Qualifying Name"], axis=1)
# very important to sort by FIPS and weeks to ensure compatibility with output data!
all_total_deaths = all_total_deaths.sort_values(["FIPS", "week_num"])
# cut off to week 45
train_total_deaths = all_total_deaths[all_total_deaths["week_num"] <= 45].drop(["FIPS", "week_num"], axis=1)
# no nulls!
train_total_deaths.isna().sum().sum()
train_total_deaths

# do this for output too
# just for the training output, since we used the previous week (downshifted),
# we need to upshift so that it's the same for both
train_output_total_deaths = narrow_total_deaths[narrow_total_deaths["week_num"] <= 45].sort_values(["FIPS", "week_num"])
train_output_total_deaths = train_output_total_deaths[["pc_deaths"]]
# nice, we have that data, just do it for all four!

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

reg = LinearRegression()
reg.fit(train_total_deaths, train_output_total_deaths)
predict = reg.predict(train_total_deaths)
R = reg.score(train_total_deaths, train_output_total_deaths)
RMSE = np.sqrt(mean_squared_error(train_output_total_deaths, predict))

# very great scores, almost perfectly fitting the data
# (linear regression has enough features to do this)
# not trivial tho - sorting issue, it's not just automatically great number of columns
# is it robust to future time, however?
RMSE_list = []
R_list = []
RMSE_list.append(RMSE)
R_list.append(R)
R_list, RMSE_list

([1.0], [3.4367050254414957e-16])

In [ ]:
# I don't know how sklearn interacts with recursion
# so I will code the recursion iteratively
# this is the toughest part conceptually

# for week 46 (45 + 1)
# for non-pc_deaths, we simply take all data up to and including week 46
# for pc_deaths we will keep all the training data, which consists of all the previous weeks up to week 45 (so lags by one week)
# the predictions of pc_deaths are of week 45, or the previous week for week 46
# append model's predictions to the model to avoid future-peeking (add 1 week = 3098 rows of data)
# then pass this data into the same regressor, have it predict the next row, and calculate metrics
# rinse and repeat

recursive_data = train_output_total_deaths

# deal with the 5 thing
for i in range(1, 1 + 4):
  input = all_total_deaths[all_total_deaths["week_num"] <= 45 + i].drop("pc_deaths", axis=1)
  # reset index to ensure compatibility
  input = input.reset_index().drop("index", axis=1)

  prev_predicts = pd.DataFrame(predict).iloc[-3098:]
  prev_predicts = prev_predicts.rename(columns={0:"pc_deaths"})

  # resetting index
  recursive_data = pd.concat([recursive_data, prev_predicts]).reset_index()[["pc_deaths"]]
  # no nulls, correct length

  # nulls seem to accumulate w input, something is faulty
  input["pc_deaths"] = recursive_data["pc_deaths"]
  input = input.sort_values(["FIPS", "week_num"])
  input = input.drop(["FIPS", "week_num"], axis=1)
  input

  # deal with this thing
  output = narrow_total_deaths[narrow_total_deaths["week_num"] <= 45 + i].sort_values(["FIPS", "week_num"])[["pc_deaths"]]
  output

  # works for one iteration, just need to string this together
  predict = reg.predict(input)
  R = reg.score(input, output)
  RMSE = np.sqrt(mean_squared_error(output, predict))
  RMSE_list.append(RMSE)
  R_list.append(R)
  print(R, RMSE)

RMSE_list, R_list

-0.8168047839812951 0.0006318196014830013
-0.770558524144155 0.00065269487886536
-0.7251003856670211 0.0006746394643844825
-0.6599424074912503 0.0006923942104522154


([3.4367050254414957e-16,
  0.0006318196014830013,
  0.00065269487886536,
  0.0006746394643844825,
  0.0006923942104522154],
 [1.0,
  -0.8168047839812951,
  -0.770558524144155,
  -0.7251003856670211,
  -0.6599424074912503])

In [ ]:
output[output["pc_deaths"] > 0.0007]

,pc_deaths
18588,0.000704
15490,0.000758
12392,0.000758
9294,0.000758
6196,0.000776
...,...
15488,0.000997
12390,0.001370
9292,0.001370
6194,0.001495


**Step X: Data Exploration and Visualization**






WEEKLY CASES PER REGION THROUHGOUT THE YEAR

In [ ]:
weekly_change_cases['Qualifying Name'].replace('[-.A-Za-z0-9 \']+[,ß\'±?]', '', regex=True, inplace=True) #front
graphweekly = weekly_change_cases
graphweekly = graphweekly.drop('FIPS', axis=1)
graphweekly = graphweekly.drop('Name of Area', axis=1)
graphweekly = graphweekly.drop(0, axis = 0).fillna(0)

fields = list(graphweekly)[1:]
fields

graphweeklydf1 = graphweekly[['Qualifying Name']]
graphweeklydf2 = graphweekly[fields].astype(int)


graphweeklynew = pd.concat([graphweeklydf1, graphweeklydf2], axis = 1)
graphweeklynew = graphweeklynew.groupby('Qualifying Name').sum()
graphweeklynew

In [ ]:
regionlist = ['South', 'West', 'Overseas Territory', 'West', 'South', 'West', 'West', 'Northeast', 'South', 'South', 'South', 'South', 'Overseas Territory', 'West', 'West', 'Midwest', 'Midwest', 'Midwest', 'Midwest', 'South', 'South', 'Northeast', 'South', 'Northeast', 'Midwest', 'Midwest', 'South', 'Midwest', 'West', 'Midwest', 'West', 'Northeast', 'Northeast', 'West', 'Northeast', 'South', 'Midwest', 'Overseas Territory', 'Midwest', 'South', 'West', 'Northeast', 'Overseas Territory', 'Northeast', 'South', 'Midwest', 'South', 'South', 'West', 'Northeast', 'Overseas Territory', 'South', 'West', 'South', 'Midwest', 'West']
graphweeklynew['Region'] = regionlist

regiongraph = graphweeklynew.groupby('Region').sum()
regiongraph

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize = (10, 6))

copy = regiongraph
#copy.set_index('Qualifying Name', inplace=True)

# Plot the data for each state
#fig, ax = plt.subplots(figsize=(12,8))
for state in copy.index.unique():
    ax.plot(copy.loc[state], label=state)

# Set the title and axis labels
ax.set_title('COVID Cases by Region Per Week')
ax.set_xlabel('Week')
ax.set_ylabel('Number of Cases')

# Add a legend
ax.legend()

ax.invert_xaxis()
plt.xticks(visible=False)
# Show the plot
plt.show()

WEEKLY DEATHS PER REGION THROUHGOUT THE YEAR

In [ ]:
weekly_change_deaths['Qualifying Name'].replace('[-.A-Za-z0-9 \']+[,ß\'±?]', '', regex=True, inplace=True)
weekly_change_deaths
graphweeklydeaths = weekly_change_deaths
graphweeklydeaths = graphweeklydeaths.drop('FIPS', axis=1)
graphweeklydeaths = graphweeklydeaths.drop('Name of Area', axis=1)
graphweeklydeaths = graphweeklydeaths.drop(0, axis = 0).fillna(0)

fields2 = list(graphweeklydeaths)[1:]
fields2
graphweeklydeathsdf1 = graphweeklydeaths[['Qualifying Name']]
graphweeklydeathsdf2 = graphweeklydeaths[fields2].astype(int)

graphweeklydeathsnew = pd.concat([graphweeklydeathsdf1, graphweeklydeathsdf2], axis = 1)
graphweeklydeathsnew = graphweeklydeathsnew.groupby('Qualifying Name').sum()
graphweeklydeathsnew

In [ ]:
regionlist2 = ['South', 'West', 'Overseas Territory', 'West', 'South', 'West', 'West', 'Northeast', 'South', 'South', 'South', 'South', 'Overseas Territory', 'West', 'West', 'Midwest', 'Midwest', 'Midwest', 'Midwest', 'South', 'South', 'Northeast', 'South', 'Northeast', 'Midwest', 'Midwest', 'South', 'Midwest', 'West', 'Midwest', 'West', 'Northeast', 'Northeast', 'West', 'Northeast', 'South', 'Midwest', 'Overseas Territory', 'Midwest', 'South', 'West', 'Northeast', 'Overseas Territory', 'Northeast', 'South', 'Midwest', 'South', 'South', 'West', 'Northeast', 'Overseas Territory', 'South', 'West', 'South', 'Midwest', 'West']
graphweeklydeathsnew['Region'] = regionlist2

regiondeathsgraph = graphweeklydeathsnew.groupby('Region').sum()
regiondeathsgraph

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize = (10, 6))

copy = regiondeathsgraph
#copy.set_index('Qualifying Name', inplace=True)

# Plot the data for each state
#fig, ax = plt.subplots(figsize=(12,8))
for state in copy.index.unique():
    ax.plot(copy.loc[state], label=state)

# Set the title and axis labels
ax.set_title('Covid Deaths by Region Per Week')
ax.set_xlabel('Week')
ax.set_ylabel('Number of Deaths')

# Add a legend

ax.invert_xaxis()
ax.legend()
plt.xticks(visible=False)
# Show the plot
plt.show()

TOTAL CASES PER REGION THROUHGOUT THE YEAR

In [ ]:
weekly_total_cases['Qualifying Name'].replace('[-.A-Za-z0-9 \']+[,ß\'±?]', '', regex=True, inplace=True) #front
graphweeklytotalcases = weekly_total_cases
graphweeklytotalcases = graphweeklytotalcases.drop('FIPS', axis=1)
graphweeklytotalcases = graphweeklytotalcases.drop('Name of Area', axis=1)
graphweeklytotalcases = graphweeklytotalcases.drop(0, axis = 0).fillna(0)

fields = list(graphweeklytotalcases)[1:]
fields

graphweeklytotalcasesdf1 = graphweeklytotalcases[['Qualifying Name']]
graphweeklytotalcasesdf2 = graphweeklytotalcases[fields].astype(int)


graphweeklytotalcasesnew = pd.concat([graphweeklytotalcasesdf1, graphweeklytotalcasesdf2], axis = 1)
graphweeklytotalcasesnew = graphweeklytotalcasesnew.groupby('Qualifying Name').sum()
graphweeklytotalcasesnew

In [ ]:
regionlist3 = ['South', 'West', 'Overseas Territory', 'West', 'South', 'West', 'West', 'Northeast', 'South', 'South', 'South', 'South', 'Overseas Territory', 'West', 'West', 'Midwest', 'Midwest', 'Midwest', 'Midwest', 'South', 'South', 'Northeast', 'South', 'Northeast', 'Midwest', 'Midwest', 'South', 'Midwest', 'West', 'Midwest', 'West', 'Northeast', 'Northeast', 'West', 'Northeast', 'South', 'Midwest', 'Overseas Territory', 'Midwest', 'South', 'West', 'Northeast', 'Overseas Territory', 'Northeast', 'South', 'Midwest', 'South', 'South', 'West', 'Northeast', 'Overseas Territory', 'South', 'West', 'South', 'Midwest', 'West']
graphweeklytotalcasesnew['Region'] = regionlist3

totalcasesgraph = graphweeklytotalcasesnew.groupby('Region').sum()
totalcasesgraph

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize = (10, 6))

copy = totalcasesgraph
#copy.set_index('Qualifying Name', inplace=True)

# Plot the data for each state
#fig, ax = plt.subplots(figsize=(12,8))
for state in copy.index.unique():
    ax.plot(copy.loc[state], label=state)

# Set the title and axis labels
ax.set_title('Total Covid Cases Across by Region across 2020')
ax.set_xlabel('Week')
ax.set_ylabel('Number of Cases')

# Add a legend
ax.legend()
ax.invert_xaxis()
plt.xticks(visible=False)
# Show the plot
plt.show()

TOTAL DEATHS PER REGION THROUHGOUT THE YEAR

In [ ]:
weekly_total_deaths['Qualifying Name'].replace('[-.A-Za-z0-9 \']+[,ß\'±?]', '', regex=True, inplace=True) #front
graphweeklytotaldeaths = weekly_total_deaths
graphweeklytotaldeaths = graphweeklytotaldeaths.drop('FIPS', axis=1)
graphweeklytotaldeaths = graphweeklytotaldeaths.drop('Name of Area', axis=1)
graphweeklytotaldeaths = graphweeklytotaldeaths.drop(0, axis = 0).fillna(0)

fields = list(graphweeklytotaldeaths)[1:]
fields

graphweeklytotaldeathsdf1 = graphweeklytotaldeaths[['Qualifying Name']]
graphweeklytotaldeathsdf2 = graphweeklytotaldeaths[fields].astype(int)


graphweeklytotaldeathsnew = pd.concat([graphweeklytotaldeathsdf1, graphweeklytotaldeathsdf2], axis = 1)
graphweeklytotaldeathsnew = graphweeklytotaldeathsnew.groupby('Qualifying Name').sum()
graphweeklytotaldeathsnew

In [ ]:
regionlist4 = ['South', 'West', 'Overseas Territory', 'West', 'South', 'West', 'West', 'Northeast', 'South', 'South', 'South', 'South', 'Overseas Territory', 'West', 'West', 'Midwest', 'Midwest', 'Midwest', 'Midwest', 'South', 'South', 'Northeast', 'South', 'Northeast', 'Midwest', 'Midwest', 'South', 'Midwest', 'West', 'Midwest', 'West', 'Northeast', 'Northeast', 'West', 'Northeast', 'South', 'Midwest', 'Overseas Territory', 'Midwest', 'South', 'West', 'Northeast', 'Overseas Territory', 'Northeast', 'South', 'Midwest', 'South', 'South', 'West', 'Northeast', 'Overseas Territory', 'South', 'West', 'South', 'Midwest', 'West']
graphweeklytotaldeathsnew['Region'] = regionlist4

totaldeathsgraph = graphweeklytotaldeathsnew.groupby('Region').sum()
totaldeathsgraph

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize = (10, 6))

copy = totaldeathsgraph
#copy.set_index('Qualifying Name', inplace=True)

# Plot the data for each state
#fig, ax = plt.subplots(figsize=(12,8))
for state in copy.index.unique():
    ax.plot(copy.loc[state], label=state)

# Set the title and axis labels
ax.set_title('Total Covid Deaths by Region across 2020')
ax.set_xlabel('Week')
ax.set_ylabel('Number of Deaths')

# Add a legend
ax.legend()
ax.invert_xaxis()
plt.xticks(visible=False)
# Show the plot
plt.show()

SCATTERPLOTS

In [ ]:
df1 = yearly_new
df2 = yearly_cases_output

# Concatenate the DataFrames horizontally
concatenated_df = pd.concat([df1, df2], axis=1)

# Calculate the correlation matrix
correlation_matrix1 = concatenated_df.corr(method='pearson')
correlation_matrix1

In [ ]:
import matplotlib.pyplot as plt
# find the highest 4 absolute values in the 'values' column
highest_abs1 = correlation_matrix1['pc_cases'].abs().nlargest(6)

# find the lowest 4 absolute values in the 'values' column
lowest_abs1 = correlation_matrix1['pc_cases'].abs().nsmallest(5)

print('Highest 4 absolute values:')
print(highest_abs1)

print('Lowest 4 absolute values:')
print(lowest_abs1)

In [ ]:
merged_indices1 = pd.concat([highest_abs1, lowest_abs1])
ax = merged_indices1[1:].plot.bar()

# set the title of the bar graph
ax.set_title('Variables Most and Least Correlated to Cases')

In [ ]:
df1 = yearly_new
df2 = yearly_deaths_output

# Concatenate the DataFrames horizontally
concatenated_df = pd.concat([df1, df2], axis=1)

# Calculate the correlation matrix
correlation_matrix2 = concatenated_df.corr(method='pearson')
correlation_matrix2

In [ ]:
# find the highest 4 absolute values in the 'values' column
highest_abs2 = correlation_matrix2['pc_deaths'].abs().nlargest(6)

# find the lowest 4 absolute values in the 'values' column
lowest_abs2 = correlation_matrix2['pc_deaths'].abs().nsmallest(5)

print('Highest 4 absolute values:')
print(highest_abs2)

print('Lowest 4 absolute values:')
print(lowest_abs2)

In [ ]:
merged_indices2 = pd.concat([highest_abs2, lowest_abs2])
ax2 = merged_indices2[1:].plot.bar()

ax2.set_title('Variables Most and Least Correlated to Deaths')

In [ ]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np

df_cases = correlation_matrix1[['pc_cases']]
df_deaths = correlation_matrix2[['pc_deaths']]

# combine dataframes
df_combined = pd.concat([df_cases, df_deaths], axis=1)

# create scatterplot

fig, ax = plt.subplots()
df_combined.plot.scatter(x='pc_cases', y='pc_deaths', ax=ax)


df_cases.dropna(inplace=True)
df_cases.replace([np.inf, -np.inf], np.nan, inplace=True)
df_cases.dropna(inplace=True)

df_deaths.replace([np.inf, -np.inf], np.nan, inplace=True)
df_deaths.dropna(inplace=True)
# fit linear regression

reg = LinearRegression().fit(df_cases, df_deaths)
ax.plot(df_cases, reg.predict(df_cases), color='black')

# set axis labels and title
ax.set_xlabel('Correlation with Cases')
ax.set_ylabel('Correlation with Deaths')
ax.set_title("Variables' Correlation with Cases vs Correlation with Deaths")

ax.set_ylim(-0.25, 0.4)
ax.set_xlim(-0.33, 0.2)


# display plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np

df_cases2 = yearly_cases_output[['pc_cases']]
df_deaths2 = yearly_deaths_output[['pc_deaths']]

# combine dataframes
df_combined = pd.concat([df_cases2, df_deaths2], axis=1)

# create scatterplot

fig, ax = plt.subplots()
df_combined.plot.scatter(x='pc_cases', y='pc_deaths', ax=ax)


df_cases2.dropna(inplace=True)
df_cases2.replace([np.inf, -np.inf], np.nan, inplace=True)
df_cases2.dropna(inplace=True)

df_deaths2.replace([np.inf, -np.inf], np.nan, inplace=True)
df_deaths2.dropna(inplace=True)
# fit linear regression

reg = LinearRegression().fit(df_cases2, df_deaths2)
ax.plot(df_cases2, reg.predict(df_cases2), color='black')

# set axis labels and title
ax.set_xlabel('Cases')
ax.set_ylabel('Deaths')
ax.set_title("Deaths and Cases Per Capita for US Counties")


# display plot
plt.show()

K-NEAREST NEIGHBOURS FOR CASES

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
x_train = yearly_cases_input
y_train = yearly_cases_output

from sklearn.model_selection import GridSearchCV

pipeline = make_pipeline(StandardScaler(), KNeighborsRegressor(metric="euclidean"))

grid_cv = GridSearchCV(
            pipeline,
            param_grid={
                      "kneighborsregressor__n_neighbors": range(1, 25),
                      "kneighborsregressor__metric": ["euclidean"],
                      },
            scoring="neg_mean_squared_error", cv=10)
grid_cv.fit(x_train, y_train)
grid_cv.best_params_

In [ ]:
ks, train_mses = range(1, 25), []
for k in ks:
  pipeline = make_pipeline(
      KNeighborsRegressor(n_neighbors=k, metric="euclidean"))
  pipeline.fit(x_train, y_train)
  y_train_ = pipeline.predict(x_train)
  train_mses.append(mean_squared_error(y_train, y_train_)**0.5)

In [ ]:
ks, test_mses = range(1,25), []
for k in ks:

  pipeline = make_pipeline(StandardScaler(),
      KNeighborsRegressor(n_neighbors=k, metric="euclidean"))
  test_mses.append((-cross_val_score(pipeline,
                    x_train,
                    y_train,
                    scoring = "neg_mean_squared_error",
                    cv = 10).mean())**0.5)

In [ ]:
pd.Series(train_mses, index = ks).plot.line(legend=True, label = 'Train MSE for Cases')
pd.Series(test_mses, index = ks).plot.line(legend=True, label = 'Test MSE for Cases')

LINEAR REGRESSION FOR CASES

In [ ]:
from sklearn import linear_model
Lreg = linear_model.Lasso(alpha = 0.5)
pipeline = make_pipeline(StandardScaler(), Lreg)

lasso_error = (-cross_val_score(pipeline,
                  x_train, y_train, scoring="neg_mean_squared_error",
                  cv=10).mean()) ** .5
lasso_error

LASSO REGRESSION FOR CASES

In [ ]:
from sklearn import linear_model
Lreg = linear_model.Lasso(alpha = 0.5)
pipeline = make_pipeline(StandardScaler(), Lreg)

lasso_error = (-cross_val_score(pipeline,
                  x_train, y_train, scoring="neg_mean_squared_error",
                  cv=10).mean()) ** .5
lasso_error

K-NEAREST NEIGHBOURS FOR DEATHS

In [ ]:
x_train = yearly_deaths_input
y_train = yearly_deaths_output

ks, train_mses = range(1, 25), []
for k in ks:
  pipeline = make_pipeline(
      KNeighborsRegressor(n_neighbors=k, metric="euclidean"))
  pipeline.fit(x_train, y_train)
  y_train_ = pipeline.predict(x_train)
  train_mses.append(mean_squared_error(y_train, y_train_)**0.5)

In [ ]:
ks, test_mses = range(1,25), []
for k in ks:

  pipeline = make_pipeline(StandardScaler(),
      KNeighborsRegressor(n_neighbors=k, metric="euclidean"))
  test_mses.append((-cross_val_score(pipeline,
                    x_train,
                    y_train,
                    scoring = "neg_mean_squared_error",
                    cv = 10).mean())**0.5)

In [ ]:
pd.Series(train_mses, index = ks).plot.line(legend=True, label = 'Train MSE for Deaths')
pd.Series(test_mses, index = ks).plot.line(legend=True, label = 'Test MSE for Deaths')



LINEAR REGRESSION FOR DEATHS

In [ ]:
from sklearn import linear_model
reg2 = linear_model.LinearRegression()
pipeline = make_pipeline(StandardScaler(), reg2)
lin2_error = (-cross_val_score(pipeline,
                  x_train, y_train, scoring="neg_mean_squared_error",
                  cv=10).mean()) ** .5
lin2_error

LASSO REGRESSION FOR DEATHS

In [ ]:
from sklearn import linear_model
Lreg2 = linear_model.Lasso(alpha = 0.5)
pipeline = make_pipeline(StandardScaler(), Lreg2)

lasso_error2 = (-cross_val_score(pipeline,
                  x_train, y_train, scoring="neg_mean_squared_error",
                  cv=10).mean()) ** .5
lasso_error2